# TIPE

## 1. Autoencodeurs - Apprentissage non supervisé

On commence par charger les noms des fichiers en mémoire : <br>
On sépare en 3 sous-lots de paires image label<br>
* Lot d'entrainement
* Lot de test
* Lot de validation
<br>


Pour économiser la mémoire du pc la fonction retournant le batch d'entrainement chargera effectivement les images en mémoire et libèrera la place mémoire précédement occupée (cf cellule plus bas)

### Backup Google Colab

In [ ]:
print("Vérification GPU")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print()
print()
from google.colab import files
import tensorflow as tf
import numpy as np
import scipy
from PIL import Image

# Ajoute images
entrainement = 60/100
test = 20/100
# validation = le reste : 20%
tailleImage = 399

images = []
noise = []
for file in uploaded.keys():
    if file.endswith(".jpg"):
      if file[0] == "c":
        images.append(file)
      else:
        noise.append(file)
indice = np.arange(len(images))
np.random.shuffle(indice)
train_dataset_label = []
train_dataset_item = []
test_dataset_label = []
test_dataset_item = []
validation_dataset_label = []
validation_dataset_item = []
print(len(images)," images disponibles")
print("On partage de 0 à ",str(int(entrainement*len(images)))," pour l'entrainement")
print("On partage de ",str(int(entrainement*len(images))+1)," à ",str(int((1-test)*len(images)))," pour la validation")
print("On partage de ",int((1-test)*len(images)+1)," à ",str(len(images))," pour le test")

afficherIndices = False
afficheIndiceImage = False #Affiche indice des images mélangées, sinon l'indice i 
for i in range(len(images)):
    if i <= int(entrainement*len(images)):
        train_dataset_label.append(np.array(Image.open(images[indice[i]]),np.uint8))
        train_dataset_item.append(np.array(Image.open(noise[indice[i]]),np.uint8))
        if afficherIndices == True:
            print("Indice ",str(indice[i])," associé à train") if afficheIndiceImage == True else print("Indice ",str(i)," associé à train")
        
    elif i > int((entrainement+test)*len(images)):
        i_test = i - int((entrainement+test)*len(images))
        test_dataset_label.append(np.array(Image.open(images[indice[i]]),np.uint8))
        test_dataset_item.append(np.array(Image.open(noise[indice[i]]),np.uint8))
        if afficherIndices == True:
            print("Indice ",str(indice[i])," associé à test") if afficheIndiceImage == True else print("Indice ",str(i)," associé à test")
        
    else:
        i_validation = i - int((entrainement)*len(images))
        validation_dataset_label.append(np.array(Image.open(images[indice[i]]),np.uint8))
        validation_dataset_item.append(np.array(Image.open(noise[indice[i]]),np.uint8))
        if afficherIndices == True:
            print("Indice ",str(indice[i])," associé à validation") if afficheIndiceImage == True else print("Indice ",str(i)," associé à validation")
        
print("Data stored in memory !")

### Code local

In [1]:
print("Vérification GPU")
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print()
print()
import os
import tensorflow as tf
import numpy as np
import scipy
from PIL import Image

os.chdir("D:/Photos")
# Ajoute images
entrainement = 60/100
test = 20/100
# validation = le reste : 20%
tailleImage = 399

images = []
noise = []

i = 1
while os.path.isfile("clean/"+str(i)+".jpg"):
    images.append("clean/"+str(i)+".jpg")
    i += 1

i = 1
while os.path.isfile("noise/"+str(i)+".jpg"):
    noise.append("noise/"+str(i)+".jpg")
    i += 1
    
indice = np.arange(len(images))
np.random.shuffle(indice)

train_dataset_label = [images[i] for i in indice[:int(len(indice)*entrainement)+1]]
train_dataset_item = [noise[i] for i in indice[:int(len(indice)*entrainement)+1]]

test_dataset_label = [images[i] for i in indice[int(len(indice)*entrainement):int(len(indice)*entrainement)+int(len(indice)*test)+1]]
test_dataset_item = [noise[i] for i in indice[int(len(indice)*entrainement):int(len(indice)*entrainement)+int(len(indice)*test)+1]]

validation_dataset_label = [images[i] for i in indice[int(len(indice)*entrainement)+int(len(indice)*test):]]
validation_dataset_item = [noise[i] for i in indice[int(len(indice)*entrainement)+int(len(indice)*test):]]

print(len(images)," images disponibles")
print("On partage de 0 à ",str(int(entrainement*len(images)))," pour l'entrainement")
print("On partage de ",str(int(entrainement*len(images))+1)," à ",str(int((1-test)*len(images)))," pour la validation")
print("On partage de ",int((1-test)*len(images)+1)," à ",str(len(images))," pour le test")
print("Path stored in memory !")

Vérification GPU
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7094546054700314070
]


489  images disponibles
On partage de 0 à  293  pour l'entrainement
On partage de  294  à  391  pour la validation
On partage de  392  à  489  pour le test
Path stored in memory !


In [2]:
def next_batch(batch_size, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    print("Size of {}".format(len(data)))
    idx = np.arange(len(data))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [np.array(Image.open(data[i]),np.uint8) for i in idx]
    labels_shuffle = [np.array(Image.open(labels[i]),np.uint8) for i in idx]
    print()

    return np.array(data_shuffle), np.array(labels_shuffle)

Puis la construction du modèle

In [3]:
n_epochs = 400
n_batches_per_epoch = 3
batch_size = 7

height = 399
width = 399
channels = 3

In [ ]:
tf.reset_default_graph()

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")

    conv1 = tf.layers.conv2d(X, filters=5, kernel_size=15,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=10, kernel_size=15,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv2")
    conv3 = tf.layers.conv2d(conv2, filters=7, kernel_size=15,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv3")
    conv4 = tf.layers.conv2d(conv3, filters=7, kernel_size=5,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv4")
    output = tf.layers.conv2d(conv4, filters=3, kernel_size=2,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv7")
    
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - y))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    import os
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))

epoch :  0
Size of 294



## Backup avant modification architecture autoencodeur

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai7T'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

noyaux = [20, 17, 15, 12, 10, 7, 5, 3, 5, 7, 10, 12, 15, 17, 20]
filtre = [3, 5, 7, 10, 12, 15, 17, 20, 17, 15, 12, 10, 7, 5, 3]

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")

    conv1 = tf.layers.conv2d(X, filters=filtre[0], kernel_size=noyaux[0],
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=filtre[1], kernel_size=noyaux[1],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv2')
    conv3 = tf.layers.conv2d(conv2, filters=filtre[2], kernel_size=noyaux[2],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv3')
    conv4 = tf.layers.conv2d(conv3, filters=filtre[3], kernel_size=noyaux[3],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv4')
    conv5 = tf.layers.conv2d(conv4, filters=filtre[4], kernel_size=noyaux[4],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv5')
    conv6 = tf.layers.conv2d(conv5, filters=filtre[5], kernel_size=noyaux[5],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv6')
    conv7 = tf.layers.conv2d(conv6, filters=filtre[6], kernel_size=noyaux[6],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv7')
    conv8 = tf.layers.conv2d(conv7, filters=filtre[7], kernel_size=noyaux[7],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv8')
    conv9 = tf.layers.conv2d(conv8, filters=filtre[8], kernel_size=noyaux[8],
                             strides=1, padding='SAME',
                             activation=tf.nn.relu, name='conv9')
    conv10 = tf.layers.conv2d(conv9, filters=filtre[9], kernel_size=noyaux[9],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv10')
    conv11 = tf.layers.conv2d(conv10, filters=filtre[10], kernel_size=noyaux[10],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv11')
    conv12 = tf.layers.conv2d(conv11, filters=filtre[11], kernel_size=noyaux[11],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv12')
    conv13 = tf.layers.conv2d(conv12, filters=filtre[12], kernel_size=noyaux[12],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv13')
    conv14 = tf.layers.conv2d(conv13, filters=filtre[13], kernel_size=noyaux[13],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv14')
    output = tf.layers.conv2d(conv14, filters=filtre[14], kernel_size=noyaux[14],
                              strides=1, padding='SAME',
                              activation=tf.nn.relu, name='conv15')
    
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver = tf.summary.scalar("Loss",loss)
        summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 100
    n_batches_per_epoch = 3
    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: y_batch, y: X_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver.eval(feed_dict={X: y_batch, y: X_batch})
                  summary_writer.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))
        summary_writer.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer.close()

## Stratégie de réglage des hyperparamètres

|Implémenté| Essai | Fonctionnement | Nombre de couches | Types de couches | Nb de filtres | Taille du noyau | Choix | Résultats |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
|Oui| 1| Autoencodeur sur image | 5 | Uniquement convolution | Aléatoire | Cst à 15 puis décroissant jusqu'à 2 | Nombre de batch pour avoir réglièrement la 'précision' ; Se base // image de référence |`tf.square(output - y)` descend jusqu'à 64 mais fluctue toujours à 2100 à epooque 99 |
|Oui| 2| Autoencodeur sur image | 5 | Uniquement convolution | Croissant de 3 à 15 | Décroissant de 15 à 3 |Monotonie inverse filtre/noyau <b>Mais besoin même D -> Rupture à la fin </b>; Se base // image <b>bruitée</b> |`tf.square(output - X)` reste stable de l'epoch 36 à 93 à ~200 |
|Oui| 3| Autoencodeur sur image | 5 | Uniquement convolution | Croissant de 3 à 31 | Décroissant de 15 à 3 |Donner une plus gde variation du nb de filtres |Arrêté au bout d'une 1/2 h sans dépasser epoch 0 ; après chargement progressif des images fonctionne, le + près|
|Oui| 4 | Autoencodeur sur image | 5 | Uniquement convolution | Croissant de 3 à 31 | Décroissant de 15 à 3 |Prendre le meilleur modèle et le tester avec `tf.square(output - y)` | `tf.square(output - y)` Ne converge pas du tout|
|Oui| 5 | Autoencodeur sur image | 15 | Uniquement convolution | Croissant de 3 à 20 | Décroissant de 20 à 3 |Réduire les noyaux de convo pr plus efficacité et augmenter le nb de couches| `tf.square(output - y)` Ne converge pas du tout|
|Oui| 6 | Autoencodeur sur image | 15 | Uniquement convolution | Symétrique de 3 à 20 | Symétrique de 3 à 20 |Utiliser la symétrie préconisée ds livre et la tester avec `tf.square(output - y)` | `tf.square(output - y)` ne converge pas du tout (Pas arrivé jusqu'à la fin)|
|Oui| 7 | Autoencodeur sur image | 5 | Uniquement convolution | Croissant de 3 à 31 | Décroissant de 15 à 3 |Prendre le meilleur modèle et le tester avec `tf.square(output - y)` | `tf.square(output - y)` Ne converge pas du tout|
|Oui| 7T | Autoencodeur sur image | 5 | Uniquement convolution | Croissante de 3 à 15 |Décroissante de 15 à 3  |Couche de max_pooling|Erreur plus basse mais `tf.square(output - X)` avec cette choix les images non bruitées n'apprend pas réellement |
|Oui| 8 | Autoencodeur sur image | 5 | Convolution et couche intégralement connectée au milieu | Symétrique de 3 à 20 | Symétrique de 3 à 20 |Couche de max_pooling + Intégralement connectée|Erreur plus basse mais `tf.square(output - X)` avec cette choix les images non bruitées n'apprend pas réellement |
|Oui| 9 | DNN sur image | 5 | Couche intégralement connectée | - | - |-|-|-|-|-|Intégralement connectée|- |

Changement de perspective : le modèle n'apprend pas puisque globalement les étoiles ne sont pas détectées. Le bruit est détecté. Il s'agirait alors d'extraire le bruit des images et de le soustraire à l'image bruitée.<br>
Ou bien recentrer les images sur les galaxies bruitées/ débruitées pour :<br>
<ul>
    <li>Déterminer si on a un objet</li>
    <li>Corriger en fonction</li>
</ul>

|Implémenté| Essai | Fonctionnement | Nombre de couches | Types de couches | Nb de filtres | Taille du noyau| Choix | Résultats |
| --- | --- | ---| ---| ---| ---| ---| --- | --- |
|Oui| 9 | Autoencodeur sur image | 5 | Couche intégralement connectée | - | - |Que couche dense pour reproduire image débruitée|Converge et même surajuste|
|Oui| 10 | Autoencodeur sur image | 5 | Couche intégralement connectée | - | - |Couche dense pour reproduire à partir des images bruitées les images débruitées|Ne converge pas vraiment|
|Oui| 11 | Autoencodeur sur image | 5 | Couche intégralement connectée | - | - |Même architecture mais en laissant apprendre plus longtemps|Toujours pas|
|Oui| 12 | Autoencodeur sur image | 5 | Couche intégralement connectée | - | - |Plus de couches en revenant à 300 iterations (suffisait en essai 11|-|
|Oui| 13 | Autoencodeur sur image | 5 | Uniquement convolution | Constant à 3 | - | Symétrique de 15 à 3 |Entrainement couche par couche | `tf.square(output - y)` |

|Oui| 13 | Autoencodeur sur image | 5 | Uniquement convolution | Constant à 3 | Symétrique de 15 à 3 | Relu |7| 3|99|-|-|---| Entrainement couche par couche | `tf.square(output - y)` |

Essai sur les tailles de noyau de convolution sur 100 epoch

|Implémenté| Essai | Fonctionnement | Nb de filtres | Taille du noyau| Choix | Résultats |
| --- | --- | ---| ---| ---| ---| ---|
|Oui| C1 | Autoencodeur sur image | 3 | 2 |-|-|
|Oui| C2 | Autoencodeur sur image | 3 | 4 |-|-|
|Oui| C3 | Autoencodeur sur image | 3 | 6 |-|-|
|Oui| C4 | Autoencodeur sur image | 3 | 8 |-|-|
|Oui| C5 | Autoencodeur sur image | 3 | 10 |-|-|
|Oui| C6 | Autoencodeur sur image | 3 | 12 |-|-|
|Oui| C7 | Autoencodeur sur image | 3 | 14 |-|-|
|Oui| C8 | Autoencodeur sur image | 3 | 16 |-|-|

On observe que plus le noyau est petit, plus la couche apprend. A l'inverse plus le noyau est grand, moins le système apprend. Néanmoins, les deux aboutissent à une même précision

En combinant les deux 'brutalement' (sans entrainement couche par couche) :

In [5]:
print("""|Implémenté| Essai | Fonctionnement | Nombre de couches | Nb de filtres | Taille du noyau| Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |""")
i = 0
for c1 in range(1,9):
    for c2 in range(1,9):
        print("""|Oui| Cdeux"""+str(i)+""" | Autoencodeur sur image | 3 | """+str(c1*2)+""", """+str(c2*2)+""" |-|-|""")
        i += 1

|Implémenté| Essai | Fonctionnement | Nombre de couches | Nb de filtres | Taille du noyau| Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |
|Oui| Cdeux0 | Autoencodeur sur image | 3 | 2, 2 |-|-|
|Oui| Cdeux1 | Autoencodeur sur image | 3 | 2, 4 |-|-|
|Oui| Cdeux2 | Autoencodeur sur image | 3 | 2, 6 |-|-|
|Oui| Cdeux3 | Autoencodeur sur image | 3 | 2, 8 |-|-|
|Oui| Cdeux4 | Autoencodeur sur image | 3 | 2, 10 |-|-|
|Oui| Cdeux5 | Autoencodeur sur image | 3 | 2, 12 |-|-|
|Oui| Cdeux6 | Autoencodeur sur image | 3 | 2, 14 |-|-|
|Oui| Cdeux7 | Autoencodeur sur image | 3 | 2, 16 |-|-|
|Oui| Cdeux8 | Autoencodeur sur image | 3 | 4, 2 |-|-|
|Oui| Cdeux9 | Autoencodeur sur image | 3 | 4, 4 |-|-|
|Oui| Cdeux10 | Autoencodeur sur image | 3 | 4, 6 |-|-|
|Oui| Cdeux11 | Autoencodeur sur image | 3 | 4, 8 |-|-|
|Oui| Cdeux12 | Autoencodeur sur image | 3 | 4, 10 |-|-|
|Oui| Cdeux13 | Autoencodeur sur image | 3 | 4, 12 |-|-|
|Oui| Cdeux14 | Autoencodeur sur image | 3 | 4, 14 |-|-|
|Oui| 

|Implémenté| Essai | Fonctionnement | Nombre de couches | Nb de filtres | Taille du noyau| Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |
|Oui| Cdeux0 | Autoencodeur sur image | 3 | 2, 2 |-|-|
|Oui| Cdeux1 | Autoencodeur sur image | 3 | 2, 4 |-|-|
|Oui| Cdeux2 | Autoencodeur sur image | 3 | 2, 6 |-|-|
|Oui| Cdeux3 | Autoencodeur sur image | 3 | 2, 8 |-|-|
|Oui| Cdeux4 | Autoencodeur sur image | 3 | 2, 10 |-|-|
|Oui| Cdeux5 | Autoencodeur sur image | 3 | 2, 12 |-|-|
|Oui| Cdeux6 | Autoencodeur sur image | 3 | 2, 14 |-|-|
|Oui| Cdeux7 | Autoencodeur sur image | 3 | 2, 16 |-|-|
|Oui| Cdeux8 | Autoencodeur sur image | 3 | 4, 2 |-|-|
|Oui| Cdeux9 | Autoencodeur sur image | 3 | 4, 4 |-|-|
|Oui| Cdeux10 | Autoencodeur sur image | 3 | 4, 6 |-|-|
|Oui| Cdeux11 | Autoencodeur sur image | 3 | 4, 8 |-|-|
|Oui| Cdeux12 | Autoencodeur sur image | 3 | 4, 10 |-|-|
|Oui| Cdeux13 | Autoencodeur sur image | 3 | 4, 12 |-|-|
|Oui| Cdeux14 | Autoencodeur sur image | 3 | 4, 14 |-|-|
|Oui| Cdeux15 | Autoencodeur sur image | 3 | 4, 16 |-|-|
|Oui| Cdeux16 | Autoencodeur sur image | 3 | 6, 2 |-|-|
|Oui| Cdeux17 | Autoencodeur sur image | 3 | 6, 4 |-|-|
|Oui| Cdeux18 | Autoencodeur sur image | 3 | 6, 6 |-|-|
|Oui| Cdeux19 | Autoencodeur sur image | 3 | 6, 8 |-|-|
|Oui| Cdeux20 | Autoencodeur sur image | 3 | 6, 10 |-|-|
|Oui| Cdeux21 | Autoencodeur sur image | 3 | 6, 12 |-|-|
|Oui| Cdeux22 | Autoencodeur sur image | 3 | 6, 14 |-|-|
|Oui| Cdeux23 | Autoencodeur sur image | 3 | 6, 16 |-|-|
|Oui| Cdeux24 | Autoencodeur sur image | 3 | 8, 2 |-|-|
|Oui| Cdeux25 | Autoencodeur sur image | 3 | 8, 4 |-|-|
|Oui| Cdeux26 | Autoencodeur sur image | 3 | 8, 6 |-|-|
|Oui| Cdeux27 | Autoencodeur sur image | 3 | 8, 8 |-|-|
|Oui| Cdeux28 | Autoencodeur sur image | 3 | 8, 10 |-|-|
|Oui| Cdeux29 | Autoencodeur sur image | 3 | 8, 12 |-|-|
|Oui| Cdeux30 | Autoencodeur sur image | 3 | 8, 14 |-|-|
|Oui| Cdeux31 | Autoencodeur sur image | 3 | 8, 16 |-|-|
|Oui| Cdeux32 | Autoencodeur sur image | 3 | 10, 2 |-|-|
|Oui| Cdeux33 | Autoencodeur sur image | 3 | 10, 4 |-|-|
|Oui| Cdeux34 | Autoencodeur sur image | 3 | 10, 6 |-|-|
|Oui| Cdeux35 | Autoencodeur sur image | 3 | 10, 8 |-|-|
|Oui| Cdeux36 | Autoencodeur sur image | 3 | 10, 10 |-|-|
|Oui| Cdeux37 | Autoencodeur sur image | 3 | 10, 12 |-|-|
|Oui| Cdeux38 | Autoencodeur sur image | 3 | 10, 14 |-|-|
|Oui| Cdeux39 | Autoencodeur sur image | 3 | 10, 16 |-|-|
|Oui| Cdeux40 | Autoencodeur sur image | 3 | 12, 2 |-|-|
|Oui| Cdeux41 | Autoencodeur sur image | 3 | 12, 4 |-|-|
|Oui| Cdeux42 | Autoencodeur sur image | 3 | 12, 6 |-|-|
|Oui| Cdeux43 | Autoencodeur sur image | 3 | 12, 8 |-|-|
|Oui| Cdeux44 | Autoencodeur sur image | 3 | 12, 10 |-|-|
|Oui| Cdeux45 | Autoencodeur sur image | 3 | 12, 12 |-|-|
|Oui| Cdeux46 | Autoencodeur sur image | 3 | 12, 14 |-|-|
|Oui| Cdeux47 | Autoencodeur sur image | 3 | 12, 16 |-|-|
|Oui| Cdeux48 | Autoencodeur sur image | 3 | 14, 2 |-|-|
|Oui| Cdeux49 | Autoencodeur sur image | 3 | 14, 4 |-|-|
|Oui| Cdeux50 | Autoencodeur sur image | 3 | 14, 6 |-|-|
|Oui| Cdeux51 | Autoencodeur sur image | 3 | 14, 8 |-|-|
|Oui| Cdeux52 | Autoencodeur sur image | 3 | 14, 10 |-|-|
|Oui| Cdeux53 | Autoencodeur sur image | 3 | 14, 12 |-|-|
|Oui| Cdeux54 | Autoencodeur sur image | 3 | 14, 14 |-|-|
|Oui| Cdeux55 | Autoencodeur sur image | 3 | 14, 16 |-|-|
|Oui| Cdeux56 | Autoencodeur sur image | 3 | 16, 2 |-|-|
|Oui| Cdeux57 | Autoencodeur sur image | 3 | 16, 4 |-|-|
|Oui| Cdeux58 | Autoencodeur sur image | 3 | 16, 6 |-|-|
|Oui| Cdeux59 | Autoencodeur sur image | 3 | 16, 8 |-|-|
|Oui| Cdeux60 | Autoencodeur sur image | 3 | 16, 10 |-|-|
|Oui| Cdeux61 | Autoencodeur sur image | 3 | 16, 12 |-|-|
|Oui| Cdeux62 | Autoencodeur sur image | 3 | 16, 14 |-|-|
|Oui| Cdeux63 | Autoencodeur sur image | 3 | 16, 16 |-|-|

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 0
c1,c2 = 2,2
while c1 <= 16:
  c2 = 2
  while c2 <= 16:
    for p in (2,3):
      print("Essai "+str(i)+" pour conv0 avec " + str(c1) + " et conv1 avec " + str(c2) + "et pour couche de pooling "+str(p))
      print(str(i/127)+"% réalisé")
      nom = 'EssaiCdeuxPooling'+str(i)

      tf.reset_default_graph()

      root_logdir = "tf_logs"
      logdir = "{}/run-{}/".format(root_logdir,nom)

      with tf.Graph().as_default():
          X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
          y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
          print(X.get_shape())


          conv0 = tf.layers.conv2d(X, filters=3, kernel_size=c1,
                  strides=1, padding='SAME',
 	 	 	            activation=tf.nn.relu, name='conv0')
          size0 = conv0.get_shape()
          print('Shape of conv0 : '+str(size0))
          maxPool0 = tf.layers.max_pooling2d(conv0, pool_size=(p,p), strides=(2,2), padding='same')
          print('Shape of maxPool0 : '+str(maxPool0.get_shape()))

          conv1 = tf.layers.conv2d(maxPool0, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv1')
          size1 = conv1.get_shape()
          print('Shape of conv1 : '+str(size1))
          conv2 = tf.layers.conv2d(conv1, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv2')
          print('Shape of conv2 : '+str(conv2.get_shape()))
          resize0 = tf.image.resize_images(conv2, size=size0[1:3], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
          print('Shape of resize0 : '+str(resize0.get_shape()))
          conv3 = tf.layers.conv2d(resize0, filters=3, kernel_size=c1,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv3')
          print('Shape of conv3 : '+str(conv3.get_shape()))


          with tf.name_scope('Optimizer'):
              optimizer = tf.train.AdamOptimizer()
          with tf.name_scope("phase0"):
              loss0 = tf.reduce_mean(tf.square(conv3 - y))
              training_op0 = optimizer.minimize(loss0)

          with tf.name_scope("init"):
              init = tf.global_variables_initializer()
          with tf.name_scope("enregistrement"):
              summary_writer = tbc.get_writer()
              loss_saver0 = tf.summary.scalar("Loss",loss0)
              kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
              bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
              kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
              bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
              
              kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
              bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
              kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
              bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
              
              kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
              bias_saver0 = tf.summary.histogram("Bias0",bias0)
              kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
              bias_saver1 = tf.summary.histogram("Bias1",bias1)
              kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
              bias_saver2 = tf.summary.histogram("Bias2",bias2)
              kernel_saver3 = tf.summary.histogram("Kernel3",kernel2)
              bias_saver3 = tf.summary.histogram("Bias3",bias2)
              
              print(os.path.split(X.name))
              summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
              saver = tf.train.Saver()

          with tf.name_scope('Avant-Apres_'+nom):
              orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
              fin = tf.summary.image('output',tf.cast(conv1,dtype=tf.uint8))

          n_epochs = 100
          n_batches_per_epoch = 3
          Lloss = []
          def arret(lastLoss, seuil, duree, decroissance = False):
              global Lloss
              if len(Lloss) < duree:
                  Lloss.append(lastLoss)
                  return False
              else:
                  Lloss = Lloss[1:duree] + [lastLoss]
                  L = Lloss[:]
                  L.sort(reverse=True)
                  if L[0] <= seuil:
                    if decroissance == True:
                      return True if L == Lloss else False
                    else:
                      return True
                  return False

          import os
          with tf.Session() as sess:
              init.run()
              for epoch in range(n_epochs):
                  print("epoch : ",epoch)
                  for iteration in range(n_batches_per_epoch):
                      X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                      sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                      if iteration % 5 == 0:
                        summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)

                  loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
                  print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
                  if arret(loss_train0,100,6) == True:
                    break
              summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
              summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
              saver.save(sess, 'model/model'+nom+'.ckpt')
      summary_writer0.close()
    c2 += 2
    i += 1
  c1 += 2

Globalement à la fin le système reste imprécis et n'apprend plus

Après une repécision sur la théorie des couches de convolution il apparait que ces couches permettent de rééchantillonner les données, d'en extraire les caractéristique et d'en réduire la dimensionnalité. Néanmoins, pour cela une couche de pooling est nécessaire. On reprend donc la démarche entamée en essai 8 sans la couche intégralement connectée tout d'abord.

Comme cette piste expliquerait les résultats, et étant donné les résultats sur une seule couche peut prometteurs, l'entraienement de 2 couche par couche est laissé de côté.

<b>On fait varier les noyaux entre 2 et 16, avec 3 filtre par couche, pour 2 couches tout en faisant un pooling allant de 2 à 3 et un pas unitaire. On fait utilise le symétrique pour la reconstruction.</b></br>
Donc 2 étapes : </br>
<ol>
    <li>Encodage : réduction de l'image</li>
    <li>Décodage : reconstruction de l'image désirée</li>
</ol>

On aura des couches du type : (voir cellules cachées du dessous pour le détail des essais)

|Implémenté| Essai | Fonctionnement | Taille du noyau 1 | Pooling | Taille du noyau 2 | Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |
|Oui| CdeuxPooling0 | Autoencodeur sur image | 2| 2 |2|-|-|

In [1]:
print("""|Implémenté| Essai | Fonctionnement | Taille du noyau 1 | Pooling | Taille du noyau 2 | Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |""")
i = 0
for c1 in range(1,9):
    for c2 in range(1,9):
        for p in (2,3):
            print("""|Oui| CdeuxPooling"""+str(i)+""" | Autoencodeur sur image | """+str(c1*2)+"""| """+str(p)+""" |"""+str(c2*2)+"""|-|-|""")
            i += 1

|Implémenté| Essai | Fonctionnement | Taille du noyau 1 | Pooling | Taille du noyau 2 | Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |
|Oui| CdeuxPooling0 | Autoencodeur sur image | 2| 2 |2|-|-|
|Oui| CdeuxPooling1 | Autoencodeur sur image | 2| 3 |2|-|-|
|Oui| CdeuxPooling2 | Autoencodeur sur image | 2| 2 |4|-|-|
|Oui| CdeuxPooling3 | Autoencodeur sur image | 2| 3 |4|-|-|
|Oui| CdeuxPooling4 | Autoencodeur sur image | 2| 2 |6|-|-|
|Oui| CdeuxPooling5 | Autoencodeur sur image | 2| 3 |6|-|-|
|Oui| CdeuxPooling6 | Autoencodeur sur image | 2| 2 |8|-|-|
|Oui| CdeuxPooling7 | Autoencodeur sur image | 2| 3 |8|-|-|
|Oui| CdeuxPooling8 | Autoencodeur sur image | 2| 2 |10|-|-|
|Oui| CdeuxPooling9 | Autoencodeur sur image | 2| 3 |10|-|-|
|Oui| CdeuxPooling10 | Autoencodeur sur image | 2| 2 |12|-|-|
|Oui| CdeuxPooling11 | Autoencodeur sur image | 2| 3 |12|-|-|
|Oui| CdeuxPooling12 | Autoencodeur sur image | 2| 2 |14|-|-|
|Oui| CdeuxPooling13 | Autoencodeur sur image | 2| 3 |14|-

|Implémenté| Essai | Fonctionnement | Taille du noyau 1 | Pooling | Taille du noyau 2 | Choix | Résultats |
| --- | --- | ---| ---| ---| ---| --- | --- |
|Oui| CdeuxPooling0 | Autoencodeur sur image | 2| 2 |2|-|-|
|Oui| CdeuxPooling1 | Autoencodeur sur image | 2| 3 |2|-|-|
|Oui| CdeuxPooling2 | Autoencodeur sur image | 2| 2 |4|-|-|
|Oui| CdeuxPooling3 | Autoencodeur sur image | 2| 3 |4|-|-|
|Oui| CdeuxPooling4 | Autoencodeur sur image | 2| 2 |6|-|-|
|Oui| CdeuxPooling5 | Autoencodeur sur image | 2| 3 |6|-|-|
|Oui| CdeuxPooling6 | Autoencodeur sur image | 2| 2 |8|-|-|
|Oui| CdeuxPooling7 | Autoencodeur sur image | 2| 3 |8|-|-|
|Oui| CdeuxPooling8 | Autoencodeur sur image | 2| 2 |10|-|-|
|Oui| CdeuxPooling9 | Autoencodeur sur image | 2| 3 |10|-|-|
|Oui| CdeuxPooling10 | Autoencodeur sur image | 2| 2 |12|-|-|
|Oui| CdeuxPooling11 | Autoencodeur sur image | 2| 3 |12|-|-|
|Oui| CdeuxPooling12 | Autoencodeur sur image | 2| 2 |14|-|-|
|Oui| CdeuxPooling13 | Autoencodeur sur image | 2| 3 |14|-|-|
|Oui| CdeuxPooling14 | Autoencodeur sur image | 2| 2 |16|-|-|
|Oui| CdeuxPooling15 | Autoencodeur sur image | 2| 3 |16|-|-|
|Oui| CdeuxPooling16 | Autoencodeur sur image | 4| 2 |2|-|-|
|Oui| CdeuxPooling17 | Autoencodeur sur image | 4| 3 |2|-|-|
|Oui| CdeuxPooling18 | Autoencodeur sur image | 4| 2 |4|-|-|
|Oui| CdeuxPooling19 | Autoencodeur sur image | 4| 3 |4|-|-|
|Oui| CdeuxPooling20 | Autoencodeur sur image | 4| 2 |6|-|-|
|Oui| CdeuxPooling21 | Autoencodeur sur image | 4| 3 |6|-|-|
|Oui| CdeuxPooling22 | Autoencodeur sur image | 4| 2 |8|-|-|
|Oui| CdeuxPooling23 | Autoencodeur sur image | 4| 3 |8|-|-|
|Oui| CdeuxPooling24 | Autoencodeur sur image | 4| 2 |10|-|-|
|Oui| CdeuxPooling25 | Autoencodeur sur image | 4| 3 |10|-|-|
|Oui| CdeuxPooling26 | Autoencodeur sur image | 4| 2 |12|-|-|
|Oui| CdeuxPooling27 | Autoencodeur sur image | 4| 3 |12|-|-|
|Oui| CdeuxPooling28 | Autoencodeur sur image | 4| 2 |14|-|-|
|Oui| CdeuxPooling29 | Autoencodeur sur image | 4| 3 |14|-|-|
|Oui| CdeuxPooling30 | Autoencodeur sur image | 4| 2 |16|-|-|
|Oui| CdeuxPooling31 | Autoencodeur sur image | 4| 3 |16|-|-|
|Oui| CdeuxPooling32 | Autoencodeur sur image | 6| 2 |2|-|-|
|Oui| CdeuxPooling33 | Autoencodeur sur image | 6| 3 |2|-|-|
|Oui| CdeuxPooling34 | Autoencodeur sur image | 6| 2 |4|-|-|
|Oui| CdeuxPooling35 | Autoencodeur sur image | 6| 3 |4|-|-|
|Oui| CdeuxPooling36 | Autoencodeur sur image | 6| 2 |6|-|-|
|Oui| CdeuxPooling37 | Autoencodeur sur image | 6| 3 |6|-|-|
|Oui| CdeuxPooling38 | Autoencodeur sur image | 6| 2 |8|-|-|
|Oui| CdeuxPooling39 | Autoencodeur sur image | 6| 3 |8|-|-|
|Oui| CdeuxPooling40 | Autoencodeur sur image | 6| 2 |10|-|-|
|Oui| CdeuxPooling41 | Autoencodeur sur image | 6| 3 |10|-|-|
|Oui| CdeuxPooling42 | Autoencodeur sur image | 6| 2 |12|-|-|
|Oui| CdeuxPooling43 | Autoencodeur sur image | 6| 3 |12|-|-|
|Oui| CdeuxPooling44 | Autoencodeur sur image | 6| 2 |14|-|-|
|Oui| CdeuxPooling45 | Autoencodeur sur image | 6| 3 |14|-|-|
|Oui| CdeuxPooling46 | Autoencodeur sur image | 6| 2 |16|-|-|
|Oui| CdeuxPooling47 | Autoencodeur sur image | 6| 3 |16|-|-|
|Oui| CdeuxPooling48 | Autoencodeur sur image | 8| 2 |2|-|-|
|Oui| CdeuxPooling49 | Autoencodeur sur image | 8| 3 |2|-|-|
|Oui| CdeuxPooling50 | Autoencodeur sur image | 8| 2 |4|-|-|
|Oui| CdeuxPooling51 | Autoencodeur sur image | 8| 3 |4|-|-|
|Oui| CdeuxPooling52 | Autoencodeur sur image | 8| 2 |6|-|-|
|Oui| CdeuxPooling53 | Autoencodeur sur image | 8| 3 |6|-|-|
|Oui| CdeuxPooling54 | Autoencodeur sur image | 8| 2 |8|-|-|
|Oui| CdeuxPooling55 | Autoencodeur sur image | 8| 3 |8|-|-|
|Oui| CdeuxPooling56 | Autoencodeur sur image | 8| 2 |10|-|-|
|Oui| CdeuxPooling57 | Autoencodeur sur image | 8| 3 |10|-|-|
|Oui| CdeuxPooling58 | Autoencodeur sur image | 8| 2 |12|-|-|
|Oui| CdeuxPooling59 | Autoencodeur sur image | 8| 3 |12|-|-|
|Oui| CdeuxPooling60 | Autoencodeur sur image | 8| 2 |14|-|-|
|Oui| CdeuxPooling61 | Autoencodeur sur image | 8| 3 |14|-|-|
|Oui| CdeuxPooling62 | Autoencodeur sur image | 8| 2 |16|-|-|
|Oui| CdeuxPooling63 | Autoencodeur sur image | 8| 3 |16|-|-|
|Oui| CdeuxPooling64 | Autoencodeur sur image | 10| 2 |2|-|-|
|Oui| CdeuxPooling65 | Autoencodeur sur image | 10| 3 |2|-|-|
|Oui| CdeuxPooling66 | Autoencodeur sur image | 10| 2 |4|-|-|
|Oui| CdeuxPooling67 | Autoencodeur sur image | 10| 3 |4|-|-|
|Oui| CdeuxPooling68 | Autoencodeur sur image | 10| 2 |6|-|-|
|Oui| CdeuxPooling69 | Autoencodeur sur image | 10| 3 |6|-|-|
|Oui| CdeuxPooling70 | Autoencodeur sur image | 10| 2 |8|-|-|
|Oui| CdeuxPooling71 | Autoencodeur sur image | 10| 3 |8|-|-|
|Oui| CdeuxPooling72 | Autoencodeur sur image | 10| 2 |10|-|-|
|Oui| CdeuxPooling73 | Autoencodeur sur image | 10| 3 |10|-|-|
|Oui| CdeuxPooling74 | Autoencodeur sur image | 10| 2 |12|-|-|
|Oui| CdeuxPooling75 | Autoencodeur sur image | 10| 3 |12|-|-|
|Oui| CdeuxPooling76 | Autoencodeur sur image | 10| 2 |14|-|-|
|Oui| CdeuxPooling77 | Autoencodeur sur image | 10| 3 |14|-|-|
|Oui| CdeuxPooling78 | Autoencodeur sur image | 10| 2 |16|-|-|
|Oui| CdeuxPooling79 | Autoencodeur sur image | 10| 3 |16|-|-|
|Oui| CdeuxPooling80 | Autoencodeur sur image | 12| 2 |2|-|-|
|Oui| CdeuxPooling81 | Autoencodeur sur image | 12| 3 |2|-|-|
|Oui| CdeuxPooling82 | Autoencodeur sur image | 12| 2 |4|-|-|
|Oui| CdeuxPooling83 | Autoencodeur sur image | 12| 3 |4|-|-|
|Oui| CdeuxPooling84 | Autoencodeur sur image | 12| 2 |6|-|-|
|Oui| CdeuxPooling85 | Autoencodeur sur image | 12| 3 |6|-|-|
|Oui| CdeuxPooling86 | Autoencodeur sur image | 12| 2 |8|-|-|
|Oui| CdeuxPooling87 | Autoencodeur sur image | 12| 3 |8|-|-|
|Oui| CdeuxPooling88 | Autoencodeur sur image | 12| 2 |10|-|-|
|Oui| CdeuxPooling89 | Autoencodeur sur image | 12| 3 |10|-|-|
|Oui| CdeuxPooling90 | Autoencodeur sur image | 12| 2 |12|-|-|
|Oui| CdeuxPooling91 | Autoencodeur sur image | 12| 3 |12|-|-|
|Oui| CdeuxPooling92 | Autoencodeur sur image | 12| 2 |14|-|-|
|Oui| CdeuxPooling93 | Autoencodeur sur image | 12| 3 |14|-|-|
|Oui| CdeuxPooling94 | Autoencodeur sur image | 12| 2 |16|-|-|
|Oui| CdeuxPooling95 | Autoencodeur sur image | 12| 3 |16|-|-|
|Oui| CdeuxPooling96 | Autoencodeur sur image | 14| 2 |2|-|-|
|Oui| CdeuxPooling97 | Autoencodeur sur image | 14| 3 |2|-|-|
|Oui| CdeuxPooling98 | Autoencodeur sur image | 14| 2 |4|-|-|
|Oui| CdeuxPooling99 | Autoencodeur sur image | 14| 3 |4|-|-|
|Oui| CdeuxPooling100 | Autoencodeur sur image | 14| 2 |6|-|-|
|Oui| CdeuxPooling101 | Autoencodeur sur image | 14| 3 |6|-|-|
|Oui| CdeuxPooling102 | Autoencodeur sur image | 14| 2 |8|-|-|
|Oui| CdeuxPooling103 | Autoencodeur sur image | 14| 3 |8|-|-|
|Oui| CdeuxPooling104 | Autoencodeur sur image | 14| 2 |10|-|-|
|Oui| CdeuxPooling105 | Autoencodeur sur image | 14| 3 |10|-|-|
|Oui| CdeuxPooling106 | Autoencodeur sur image | 14| 2 |12|-|-|
|Oui| CdeuxPooling107 | Autoencodeur sur image | 14| 3 |12|-|-|
|Oui| CdeuxPooling108 | Autoencodeur sur image | 14| 2 |14|-|-|
|Oui| CdeuxPooling109 | Autoencodeur sur image | 14| 3 |14|-|-|
|Oui| CdeuxPooling110 | Autoencodeur sur image | 14| 2 |16|-|-|
|Oui| CdeuxPooling111 | Autoencodeur sur image | 14| 3 |16|-|-|
|Oui| CdeuxPooling112 | Autoencodeur sur image | 16| 2 |2|-|-|
|Oui| CdeuxPooling113 | Autoencodeur sur image | 16| 3 |2|-|-|
|Oui| CdeuxPooling114 | Autoencodeur sur image | 16| 2 |4|-|-|
|Oui| CdeuxPooling115 | Autoencodeur sur image | 16| 3 |4|-|-|
|Oui| CdeuxPooling116 | Autoencodeur sur image | 16| 2 |6|-|-|
|Oui| CdeuxPooling117 | Autoencodeur sur image | 16| 3 |6|-|-|
|Oui| CdeuxPooling118 | Autoencodeur sur image | 16| 2 |8|-|-|
|Oui| CdeuxPooling119 | Autoencodeur sur image | 16| 3 |8|-|-|
|Oui| CdeuxPooling120 | Autoencodeur sur image | 16| 2 |10|-|-|
|Oui| CdeuxPooling121 | Autoencodeur sur image | 16| 3 |10|-|-|
|Oui| CdeuxPooling122 | Autoencodeur sur image | 16| 2 |12|-|-|
|Oui| CdeuxPooling123 | Autoencodeur sur image | 16| 3 |12|-|-|
|Oui| CdeuxPooling124 | Autoencodeur sur image | 16| 2 |14|-|-|
|Oui| CdeuxPooling125 | Autoencodeur sur image | 16| 3 |14|-|-|
|Oui| CdeuxPooling126 | Autoencodeur sur image | 16| 2 |16|-|-|
|Oui| CdeuxPooling127 | Autoencodeur sur image | 16| 3 |16|-|-|

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 0
c1,c2 = 2,2
while c1 <= 16:
  c2 = 2
  while c2 <= 16:
      print("Essai "+str(i)+" pour conv0 avec " + str(c1) + " et conv1 avec " + str(c2) + " et sans couche de pooling")
      print(str(i/127)+"% réalisé")
      nom = 'EssaiCdeux'+str(i)+'avecC0_'+str(c1)+'_C1_'+str(c2)

      tf.reset_default_graph()

      root_logdir = "tf_logs"
      logdir = "{}/run-{}/".format(root_logdir,nom)

      with tf.Graph().as_default():
          X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
          y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
          print(X.get_shape())


          conv0 = tf.layers.conv2d(X, filters=3, kernel_size=c1,
                  strides=1, padding='SAME',
 	 	 	            activation=tf.nn.relu, name='conv0')
          size0 = conv0.get_shape()
          print('Shape of conv0 : '+str(size0))

          conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv1')
          size1 = conv1.get_shape()
          print('Shape of conv1 : '+str(size1))


          with tf.name_scope('Optimizer'):
              optimizer = tf.train.AdamOptimizer()
          with tf.name_scope("phase0"):
              loss0 = tf.reduce_mean(tf.square(conv1 - y))
              training_op0 = optimizer.minimize(loss0)

          with tf.name_scope("init"):
              init = tf.global_variables_initializer()
          with tf.name_scope("enregistrement"):
              loss_saver0 = tf.summary.scalar("Loss",loss0)
              kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
              bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
              kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
              bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
              
              kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
              bias_saver0 = tf.summary.histogram("Bias0",bias0)
              kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
              bias_saver1 = tf.summary.histogram("Bias1",bias1)
              
              print(os.path.split(X.name))
              summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
              saver = tf.train.Saver()

          with tf.name_scope('Avant-Apres_'+nom):
              orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
              fin = tf.summary.image('output',tf.cast(conv1,dtype=tf.uint8))

          n_epochs = 100
          n_batches_per_epoch = 3
          Lloss = []
          def arret(lastLoss, seuil, duree, decroissance = False):
              global Lloss
              if len(Lloss) < duree:
                  Lloss.append(lastLoss)
                  return False
              else:
                  Lloss = Lloss[1:duree] + [lastLoss]
                  L = Lloss[:]
                  L.sort(reverse=True)
                  if L[0] <= seuil:
                    if decroissance == True:
                      return True if L == Lloss else False
                    else:
                      return True
                  return False

          import os
          with tf.Session() as sess:
              init.run()
              for epoch in range(n_epochs):
                  print("epoch : ",epoch)
                  for iteration in range(n_batches_per_epoch):
                      X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                      sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                      if iteration % 5 == 0:
                        summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)

                  loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
                  print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
                  if arret(loss_train0,100,6) == True:
                    break
              summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
              summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
              saver.save(sess, 'model/model'+nom+'.ckpt')
      summary_writer0.close()
      c2 += 2
      i += 1
  c1 += 2

V2

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 12
c1,c2 = 4,12
while c1 <= 16:
  c2 = 2
  while c2 <= 16:
    for p in (2,3):
      print("Essai "+str(i)+" pour conv0 avec " + str(c1) + " et conv1 avec " + str(c2) + "et pour couche de pooling "+str(p))
      print(str(i/127)+"% réalisé")
      nom = 'EssaiCdeuxPooling'+str(i)

      tf.reset_default_graph()

      root_logdir = "tf_logs"
      logdir = "{}/run-{}/".format(root_logdir,nom)

      with tf.Graph().as_default():
          X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
          y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
          print(X.get_shape())


          conv0 = tf.layers.conv2d(X, filters=3, kernel_size=c1,
                  strides=1, padding='SAME',
 	 	 	            activation=tf.nn.relu, name='conv0')
          size0 = conv0.get_shape()
          print('Shape of conv0 : '+str(size0))
          maxPool0 = tf.layers.max_pooling2d(conv0, pool_size=(p,p), strides=(2,2), padding='same')
          print('Shape of maxPool0 : '+str(maxPool0.get_shape()))

          conv1 = tf.layers.conv2d(maxPool0, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv1')
          size1 = conv1.get_shape()
          print('Shape of conv1 : '+str(size1))
          conv2 = tf.layers.conv2d(conv1, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv2')
          print('Shape of conv2 : '+str(conv2.get_shape()))
          resize0 = tf.image.resize_images(conv2, size=size0[1:3], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
          print('Shape of resize0 : '+str(resize0.get_shape()))
          conv3 = tf.layers.conv2d(resize0, filters=3, kernel_size=c1,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv3')
          print('Shape of conv3 : '+str(conv3.get_shape()))


          with tf.name_scope('Optimizer'):
              optimizer = tf.train.AdamOptimizer()
          with tf.name_scope("phase0"):
              loss0 = tf.reduce_mean(tf.square(conv3 - y))
              training_op0 = optimizer.minimize(loss0)

          with tf.name_scope("init"):
              init = tf.global_variables_initializer()
          with tf.name_scope("enregistrement"):
              loss_saver0 = tf.summary.scalar("Loss",loss0)
              kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
              bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
              kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
              bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
              
              kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
              bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
              kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
              bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
              
              kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
              bias_saver0 = tf.summary.histogram("Bias0",bias0)
              kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
              bias_saver1 = tf.summary.histogram("Bias1",bias1)
              kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
              bias_saver2 = tf.summary.histogram("Bias2",bias2)
              kernel_saver3 = tf.summary.histogram("Kernel3",kernel2)
              bias_saver3 = tf.summary.histogram("Bias3",bias2)
              
              print(os.path.split(X.name))
              summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
              saver = tf.train.Saver()

          with tf.name_scope('Avant-Apres_'+nom):
              orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
              fin = tf.summary.image('output',tf.cast(conv1,dtype=tf.uint8))

          n_epochs = 100
          n_batches_per_epoch = 3
          Lloss = []
          def arret(lastLoss, seuil, duree, decroissance = False):
              global Lloss
              if len(Lloss) < duree:
                  Lloss.append(lastLoss)
                  return False
              else:
                  Lloss = Lloss[1:duree] + [lastLoss]
                  L = Lloss[:]
                  L.sort(reverse=True)
                  if L[0] <= seuil:
                    if decroissance == True:
                      return True if L == Lloss else False
                    else:
                      return True
                  return False

          import os
          with tf.Session() as sess:
              init.run()
              for epoch in range(n_epochs):
                  print("epoch : ",epoch)
                  for iteration in range(n_batches_per_epoch):
                      X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                      sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                      if iteration % 5 == 0:
                        summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)

                  loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
                  print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
                  if arret(loss_train0,100,6) == True:
                    break
              summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
              summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
              saver.save(sess, 'model/model'+nom+'.ckpt')
      summary_writer0.close()
    c2 += 2
    i += 1
  c1 += 2

Globalement sauf changement (actuellement l'entrainement en est à l'essai 9), le système reste très imprécis et très instable.</br>
On ajoute une couche dense au milieu avec les couche de max_pooling déjà : même tableau que ci-dessus

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 0
c1,c2 = 2,2
while c1 <= 16:
  c2 = 2
  while c2 <= 16:
    for p in (2,3):
      print("Essai "+str(i)+" pour conv0 avec " + str(c1) + " et conv1 avec " + str(c2) + "et pour couche de pooling "+str(p))
      print(str(i/127)+"% réalisé")
      nom = 'EssaiCdeuxPoolingDense'+str(i)+'avecC0_'+str(c1)+'_C1_'+str(c2)+"_p_"+str(p)

      tf.reset_default_graph()

      root_logdir = "tf_logs"
      logdir = "{}/run-{}/".format(root_logdir,nom)

      with tf.Graph().as_default():
          X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
          y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
          print(X.get_shape())


          conv0 = tf.layers.conv2d(X, filters=3, kernel_size=c1,
                  strides=1, padding='SAME',
 	 	 	            activation=tf.nn.relu, name='conv0')
          size0 = conv0.get_shape()
          print('Shape of conv0 : '+str(size0))
          maxPool0 = tf.layers.max_pooling2d(conv0, pool_size=(p,p), strides=(2,2), padding='same')
          print('Shape of maxPool0 : '+str(maxPool0.get_shape()))

          conv1 = tf.layers.conv2d(maxPool0, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv1')
          size1 = conv1.get_shape()
          print('Shape of conv1 : '+str(size1))
          dense0 = tf.layers.dense(conv1,3)
          print('Shape of dense0 : '+str(dense0.get_shape()))
          conv2 = tf.layers.conv2d(dense0, filters=3, kernel_size=c2,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv2')
          print('Shape of conv2 : '+str(conv2.get_shape()))
          resize0 = tf.image.resize_images(conv2, size=size0[1:3], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
          print('Shape of resize0 : '+str(resize0.get_shape()))
          conv3 = tf.layers.conv2d(resize0, filters=3, kernel_size=c1,
                 strides=1, padding='SAME',
                 activation=tf.nn.relu, name='conv3')
          print('Shape of conv3 : '+str(conv3.get_shape()))


          with tf.name_scope('Optimizer'):
              optimizer = tf.train.AdamOptimizer()
          with tf.name_scope("phase0"):
              loss0 = tf.reduce_mean(tf.square(conv3 - y))
              training_op0 = optimizer.minimize(loss0)

          with tf.name_scope("init"):
              init = tf.global_variables_initializer()
          with tf.name_scope("enregistrement"):
              loss_saver0 = tf.summary.scalar("Loss",loss0)
              kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
              bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
              kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
              bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
              
              kernelD = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/kernel')
              biasD = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/bias')
              
              
              kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
              bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
              kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
              bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
              
              kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
              bias_saver0 = tf.summary.histogram("Bias0",bias0)
              kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
              bias_saver1 = tf.summary.histogram("Bias1",bias1)
              kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
              bias_saver2 = tf.summary.histogram("Bias2",bias2)
              kernel_saver3 = tf.summary.histogram("Kernel3",kernel2)
              bias_saver3 = tf.summary.histogram("Bias3",bias2)
              kernel_saverD = tf.summary.histogram("Kernel3",kernelD)
              bias_saverD = tf.summary.histogram("Bias3",biasD)
              
              print(os.path.split(X.name))
              summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
              saver = tf.train.Saver()

          with tf.name_scope('Avant-Apres_'+nom):
              orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
              fin = tf.summary.image('output',tf.cast(conv1,dtype=tf.uint8))

          n_epochs = 100
          n_batches_per_epoch = 3
          Lloss = []
          def arret(lastLoss, seuil, duree, decroissance = False):
              global Lloss
              if len(Lloss) < duree:
                  Lloss.append(lastLoss)
                  return False
              else:
                  Lloss = Lloss[1:duree] + [lastLoss]
                  L = Lloss[:]
                  L.sort(reverse=True)
                  if L[0] <= seuil:
                    if decroissance == True:
                      return True if L == Lloss else False
                    else:
                      return True
                  return False

          import os
          with tf.Session() as sess:
              init.run()
              for epoch in range(n_epochs):
                  print("epoch : ",epoch)
                  for iteration in range(n_batches_per_epoch):
                      X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                      sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                      if iteration % 5 == 0:
                        summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strKernelD = kernel_saverD.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_strBiasD = bias_saverD.eval(feed_dict={X: X_batch, y: y_batch})
                        summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strKernelD,epoch*n_batches_per_epoch+iteration)
                        summary_writer0.add_summary(summary_strBiasD,epoch*n_batches_per_epoch+iteration)

                  loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
                  print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
                  if arret(loss_train0,100,6) == True:
                    break
              summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
              summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
              saver.save(sess, 'model/model'+nom+'.ckpt')
      summary_writer0.close()
    c2 += 2
    i += 1
  c1 += 2

Maintenant essayons de voir l'impact du nombre de couches sur la précision. On met une taille de noyaux croissant de 2 à 15. On met le nombre de filtre à 3 fixe pour le moment. 
Sans réduction de taille et couche intégralement connectée. </br>
On fait varier le nombre de couches de 2 à 10.

In [6]:
print("""|Essai | Taille des noyaux |
| --- | ---|""")
i = 0
import numpy as np
for nb in range(2,10):
    tailles = np.linspace(2,15,nb,dtype=np.int)
    print("""| CtestCouches"""+str(i)+""" | """+str(tailles)+""" |""")
    i += 1

|Essai | Taille des noyaux |
| --- | ---|
| CtestCouches0 | [ 2 15] |
| CtestCouches1 | [ 2  8 15] |
| CtestCouches2 | [ 2  6 10 15] |
| CtestCouches3 | [ 2  5  8 11 15] |
| CtestCouches4 | [ 2  4  7  9 12 15] |
| CtestCouches5 | [ 2  4  6  8 10 12 15] |
| CtestCouches6 | [ 2  3  5  7  9 11 13 15] |
| CtestCouches7 | [ 2  3  5  6  8 10 11 13 15] |


|Essai | Taille des noyaux |
| --- | ---|
| CtestCouches0 | [ 2 15] |
| CtestCouches1 | [ 2  8 15] |
| CtestCouches2 | [ 2  6 10 15] |
| CtestCouches3 | [ 2  5  8 11 15] |
| CtestCouches4 | [ 2  4  7  9 12 15] |
| CtestCouches5 | [ 2  4  6  8 10 12 15] |
| CtestCouches6 | [ 2  3  5  7  9 11 13 15] |
| CtestCouches7 | [ 2  3  5  6  8 10 11 13 15] |
| CtestCouches8 | [ 2  2  2  2  2  2  2  2  2] |

In [35]:
nb = 4
nbResize = 5
plage = 2,15
plageResize = 3,15
inputSize = 399
import numpy as np
chaineTf = ''
seprateur = '///'
def convTf(indice,filtre,noyau,chaineTf,first=False,last=False):
    chaineTf += seprateur
    global lastName
    if last == True:
        chaineTf += 'output'
#         lastName = 'output'
    else:
        chaineTf += "conv"+str(indice)
    chaineTf += " = tf.layers.conv2d("
    if first == True:
        chaineTf += "X"
    else:
        chaineTf += lastName
    chaineTf+= ", filters="+str(filtre)+", kernel_size="+str(noyau)+",\n \t \t \t strides=1, padding='SAME',\n \t \t \t activation=tf.nn.relu, name='conv"+str(indice)+"')"
    lastName = "conv"+str(indice)
    if last == True:
        lastName = 'output'
    chaineTf += seprateur+"print('Shape of "+lastName+" : '+str("+lastName+".get_shape()))"
    return chaineTf
    
# filtre = np.linspace(plage[0],plage[1],nb//2+1 if nb % 2 == 1 else nb//2,dtype=np.int)
# noyaux = np.linspace(plage[1],plage[0],nb//2+1 if nb % 2 == 1 else nb//2,dtype=np.int)
# filtre = np.concatenate((filtre,np.flip(filtre,axis=0) if nb % 2 == 0 else np.flip(filtre[:-1],axis=0)),axis=0).tolist()
# noyaux = np.concatenate((noyaux,np.flip(noyaux,axis=0) if nb % 2 == 0 else np.flip(noyaux[:-1],axis=0)),axis=0).tolist()
filtre = np.linspace(3,3,nb,dtype=np.int)
noyaux = np.linspace(plage[0],plage[1],nb,dtype=np.int)
layers = [3 for i in range(15)]
print()
nom = ""
last = ''
lastName = 'X'
currentSize = inputSize
chaineTf = convTf(0,filtre[0],noyaux[0],chaineTf,first=True)

for i in range(1,len(filtre)-1):
    chaineTf = convTf(i,filtre[i],noyaux[i],chaineTf)
chaineTf = convTf(len(filtre)-1,filtre[-1],noyaux[-1],chaineTf,last=True)
chaineTf = chaineTf.split(seprateur)
for elem in chaineTf:
    print(elem)
print("""
with tf.name_scope('Optimizer'):
  optimizer = tf.train.AdamOptimizer()
with tf.name_scope("phase0"):
  loss0 = tf.reduce_mean(tf.square(conv3 - y))
  training_op0 = optimizer.minimize(loss0)

with tf.name_scope("init"):
  init = tf.global_variables_initializer()
with tf.name_scope("enregistrement"):
    loss_saver0 = tf.summary.scalar("Loss",loss0)""",end='')
for i in range(len(chaineTf)//2-1):
    print("""
    kernel"""+str(i)+""" = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv"""+str(i)+"""/kernel')[0]
    bias"""+str(i)+""" = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv"""+str(i)+"""/bias')[0]""",end='')

print("""
    kernel"""+str(len(chaineTf)//2-1)+""" = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/kernel')[0]
    bias"""+str(len(chaineTf)//2-1)+""" = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/bias')[0]""")
print()
for i in range(len(chaineTf)//2):
    print("""
    kernel_saver"""+str(i)+""" = tf.summary.histogram("Kernel"""+str(i)+"""",kernel"""+str(i)+""")
    bias_saver"""+str(i)+""" = tf.summary.histogram("Bias"""+str(i)+"""",bias"""+str(i)+""")""",end='')
    
print()
print("""

    print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

    with tf.name_scope('Avant-Apres_'+nom):
      orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
      fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
      init.run()
      for epoch in range(n_epochs):
          print("epoch : ",epoch)
          for iteration in range(n_batches_per_epoch):
              X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
              sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
              if iteration % 5 == 0:
                summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)""")
for i in range(len(chaineTf)//2):
    print("""
                summary_strKernel"""+str(i)+""" = kernel_saver"""+str(i)+""".eval(feed_dict={X: X_batch, y: y_batch})
                summary_strBias"""+str(i)+""" = bias_saver"""+str(i)+""".eval(feed_dict={X: X_batch, y: y_batch})
                summary_writer0.add_summary(summary_strKernel"""+str(i)+""",epoch*n_batches_per_epoch+iteration)
                summary_writer0.add_summary(summary_strBias"""+str(i)+""",epoch*n_batches_per_epoch+iteration)""",end='')
print("""
          loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
          print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
          if arret(loss_train0,100,6) == True:
              break
      summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
      summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
      saver.save(sess, 'model/model'+nom+'.ckpt')""")



conv0 = tf.layers.conv2d(X, filters=3, kernel_size=2,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv0')
print('Shape of conv0 : '+str(conv0.get_shape()))
conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=6,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv1')
print('Shape of conv1 : '+str(conv1.get_shape()))
conv2 = tf.layers.conv2d(conv1, filters=3, kernel_size=10,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv2')
print('Shape of conv2 : '+str(conv2.get_shape()))
output = tf.layers.conv2d(conv2, filters=3, kernel_size=15,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv3')
print('Shape of output : '+str(output.get_shape()))

with tf.name_scope('Optimizer'):
  optimizer = tf.train.AdamOptimizer()
with tf.name_scope("phase0"):
  loss0 = tf.reduce_mean(tf.square(conv3 - y))
  training_op0 = optimizer.minimize(loss0)

with tf.name_scope("init"):
  init = tf.global_variables_in

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 8
print("Essai avec 8 couches")
nom = 'CtestCouchesFiltres_10filtres_'+str(i)+'_'+str(2+i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    conv0 = tf.layers.conv2d(X, filters=3, kernel_size=2,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv0')
    print('Shape of conv0 : '+str(conv0.get_shape()))
    conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=2,
       strides=1, padding='SAME',
       activation=tf.nn.relu, name='conv1')
    print('Shape of conv1 : '+str(conv1.get_shape()))
    conv2 = tf.layers.conv2d(conv1, filters=9, kernel_size=2,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv2')
    print('Shape of conv2 : '+str(conv2.get_shape()))
    conv3 = tf.layers.conv2d(conv2, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv3')
    print('Shape of conv3 : '+str(conv3.get_shape()))
    conv4 = tf.layers.conv2d(conv3, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv4')
    print('Shape of conv4 : '+str(conv4.get_shape()))
    conv5 = tf.layers.conv2d(conv4, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv5')
    print('Shape of conv5 : '+str(conv5.get_shape()))
    conv6 = tf.layers.conv2d(conv5, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv6')
    print('Shape of conv6 : '+str(conv6.get_shape()))
    conv7 = tf.layers.conv2d(conv6, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv7')
    print('Shape of conv7 : '+str(conv7.get_shape()))
    output = tf.layers.conv2d(conv7, filters=3, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='output')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
      kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
      bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
      kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
      bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
      kernel4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv4/kernel')[0]
      bias4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv4/bias')[0]
      kernel5 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv5/kernel')[0]
      bias5 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv5/bias')[0]
      kernel6 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv6/kernel')[0]
      bias6 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv6/bias')[0]
      kernel7 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv7/kernel')[0]
      bias7 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv7/bias')[0]
      kernel8 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/kernel')[0]
      bias8 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)
      kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
      bias_saver2 = tf.summary.histogram("Bias2",bias2)
      kernel_saver3 = tf.summary.histogram("Kernel3",kernel3)
      bias_saver3 = tf.summary.histogram("Bias3",bias3)
      kernel_saver4 = tf.summary.histogram("Kernel4",kernel4)
      bias_saver4 = tf.summary.histogram("Bias4",bias4)
      kernel_saver5 = tf.summary.histogram("Kernel5",kernel5)
      bias_saver5 = tf.summary.histogram("Bias5",bias5)
      kernel_saver6 = tf.summary.histogram("Kernel6",kernel6)
      bias_saver6 = tf.summary.histogram("Bias6",bias6)
      kernel_saver7 = tf.summary.histogram("Kernel3",kernel7)
      bias_saver7 = tf.summary.histogram("Bias3",bias7)
      kernel_saver8 = tf.summary.histogram("Kernel8",kernel8)
      bias_saver8 = tf.summary.histogram("Bias8",bias8)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

      n_epochs = 100
      n_batches_per_epoch = 3
      Lloss = []
      def arret(lastLoss, seuil, duree, decroissance = False):
        global Lloss
        if len(Lloss) < duree:
            Lloss.append(lastLoss)
            return False
        else:
            Lloss = Lloss[1:duree] + [lastLoss]
            L = Lloss[:]
            L.sort(reverse=True)
            if L[0] <= seuil:
              if decroissance == True:
                return True if L == Lloss else False
              else:
                return True
            return False

      import os
      with tf.Session() as sess:
          init.run()
          for epoch in range(n_epochs):
              print("epoch : ",epoch)
              for iteration in range(n_batches_per_epoch):
                  X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                  sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                  if iteration % 5 == 0:
                    summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                    summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel4 = kernel_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias4 = bias_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel4,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias4,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel5 = kernel_saver5.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias5 = bias_saver5.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel5,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias5,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel6 = kernel_saver6.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias6 = bias_saver6.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel6,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias6,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel7 = kernel_saver7.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias7 = bias_saver7.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel7,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias7,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel8 = kernel_saver8.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias8 = bias_saver8.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel8,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias8,epoch*n_batches_per_epoch+iteration)
                    
              loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
              print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
              if arret(loss_train0,100,6) == True:
                  break
          summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
          summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
          saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

Comme cela ne change toujours rien, le nombre de filtre fixé à trois est peut-être la cause de cette imprécision subsistante. Donc test avec plus de filtres au milieu en sachant qu'il faut impérativement 3 filtres au début et à la fin pour correspondre aux 3 canaux RGB.

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 8
print("Essai avec 8 couches")
nom = 'CtestCouchesFiltres_10filtres_'+str(i)+'_'+str(2+i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    conv0 = tf.layers.conv2d(X, filters=3, kernel_size=2,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv0')
    print('Shape of conv0 : '+str(conv0.get_shape()))
    conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=2,
       strides=1, padding='SAME',
       activation=tf.nn.relu, name='conv1')
    print('Shape of conv1 : '+str(conv1.get_shape()))
    conv2 = tf.layers.conv2d(conv1, filters=9, kernel_size=2,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv2')
    print('Shape of conv2 : '+str(conv2.get_shape()))
    conv3 = tf.layers.conv2d(conv2, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv3')
    print('Shape of conv3 : '+str(conv3.get_shape()))
    conv4 = tf.layers.conv2d(conv3, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv4')
    print('Shape of conv4 : '+str(conv4.get_shape()))
    conv5 = tf.layers.conv2d(conv4, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv5')
    print('Shape of conv5 : '+str(conv5.get_shape()))
    conv6 = tf.layers.conv2d(conv5, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv6')
    print('Shape of conv6 : '+str(conv6.get_shape()))
    conv7 = tf.layers.conv2d(conv6, filters=9, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv7')
    print('Shape of conv7 : '+str(conv7.get_shape()))
    output = tf.layers.conv2d(conv7, filters=3, kernel_size=2,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='output')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
      kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
      bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
      kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
      bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
      kernel4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv4/kernel')[0]
      bias4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv4/bias')[0]
      kernel5 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv5/kernel')[0]
      bias5 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv5/bias')[0]
      kernel6 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv6/kernel')[0]
      bias6 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv6/bias')[0]
      kernel7 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv7/kernel')[0]
      bias7 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv7/bias')[0]
      kernel8 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/kernel')[0]
      bias8 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)
      kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
      bias_saver2 = tf.summary.histogram("Bias2",bias2)
      kernel_saver3 = tf.summary.histogram("Kernel3",kernel3)
      bias_saver3 = tf.summary.histogram("Bias3",bias3)
      kernel_saver4 = tf.summary.histogram("Kernel4",kernel4)
      bias_saver4 = tf.summary.histogram("Bias4",bias4)
      kernel_saver5 = tf.summary.histogram("Kernel5",kernel5)
      bias_saver5 = tf.summary.histogram("Bias5",bias5)
      kernel_saver6 = tf.summary.histogram("Kernel6",kernel6)
      bias_saver6 = tf.summary.histogram("Bias6",bias6)
      kernel_saver7 = tf.summary.histogram("Kernel3",kernel7)
      bias_saver7 = tf.summary.histogram("Bias3",bias7)
      kernel_saver8 = tf.summary.histogram("Kernel8",kernel8)
      bias_saver8 = tf.summary.histogram("Bias8",bias8)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

      n_epochs = 100
      n_batches_per_epoch = 3
      Lloss = []
      def arret(lastLoss, seuil, duree, decroissance = False):
        global Lloss
        if len(Lloss) < duree:
            Lloss.append(lastLoss)
            return False
        else:
            Lloss = Lloss[1:duree] + [lastLoss]
            L = Lloss[:]
            L.sort(reverse=True)
            if L[0] <= seuil:
              if decroissance == True:
                return True if L == Lloss else False
              else:
                return True
            return False

      import os
      with tf.Session() as sess:
          init.run()
          for epoch in range(n_epochs):
              print("epoch : ",epoch)
              for iteration in range(n_batches_per_epoch):
                  X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                  sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                  if iteration % 5 == 0:
                    summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                    summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel4 = kernel_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias4 = bias_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel4,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias4,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel5 = kernel_saver5.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias5 = bias_saver5.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel5,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias5,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel6 = kernel_saver6.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias6 = bias_saver6.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel6,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias6,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel7 = kernel_saver7.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias7 = bias_saver7.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel7,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias7,epoch*n_batches_per_epoch+iteration)
                    
                    summary_strKernel8 = kernel_saver8.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_strBias8 = bias_saver8.eval(feed_dict={X: X_batch, y: y_batch})
                    summary_writer0.add_summary(summary_strKernel8,epoch*n_batches_per_epoch+iteration)
                    summary_writer0.add_summary(summary_strBias8,epoch*n_batches_per_epoch+iteration)
                    
              loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
              print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
              if arret(loss_train0,100,6) == True:
                  break
          summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
          summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
          saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

Pour visualiser l'impact des couches denses seules, on ajoute prograssivement des couches denses

2 couches

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 2
print("Essai avec "+str(i)+" couches")
nom = 'Cdense_'+str(i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    
    dense0 = tf.layers.dense(X,3,name='dense0')
    print('Shape of dense0 : '+str(dense0.get_shape()))
    output = tf.layers.dense(dense0,3,name='dense1')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)

            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
            if arret(loss_train0,100,6) == True:
                break
        summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

3 couches

NB : erreur ds le code à la première exécution (pas pris la bonne couche en paramètre de la suivante)

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 3
print("Essai avec "+str(i)+" couches")
nom = 'Cdense_'+str(i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    
    dense0 = tf.layers.dense(X,3,name='dense0')
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,3,name='dense1')
    print('Shape of dense1 : '+str(dense1.get_shape()))
    output = tf.layers.dense(dense1,3,name='dense2')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/bias')[0]
      kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/kernel')[0]
      bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)
      kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
      bias_saver2 = tf.summary.histogram("Bias2",bias2)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                  
                  summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)

            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
            if arret(loss_train0,100,6) == True:
                break
        summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

4 couches

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 4
print("Essai avec "+str(i)+" couches")
nom = 'Cdense_'+str(i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    
    dense0 = tf.layers.dense(X,3,name='dense0')
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,3,name='dense1')
    print('Shape of dense1 : '+str(dense1.get_shape()))
    dense2 = tf.layers.dense(dense1,3,name='dense2')
    print('Shape of dense2 : '+str(dense2.get_shape()))
    output = tf.layers.dense(dense2,3,name='dense3')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/bias')[0]
      kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/kernel')[0]
      bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/bias')[0]
      kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense3/kernel')[0]
      bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense3/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)
      kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
      bias_saver2 = tf.summary.histogram("Bias2",bias2)
      kernel_saver3 = tf.summary.histogram("Kernel3",kernel3)
      bias_saver3 = tf.summary.histogram("Bias3",bias3)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                  
                  summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                  
                  summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)

            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
            if arret(loss_train0,100,6) == True:
                break
        summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

Comme cela semble ne rien changer : même les couches denses la précision ne reste pas de façon stable en-dessous de 500 cela doit provenir de la base de donnée.</br> 
Le problème doit provenir d'un lot pas assez diversifié et des images 'trop statique' : les images sont déjà réalisées, et en nombre pas forcément assez improtant. De plus, les réglages photoshop ont été peut-être trop poussés. Pour tenter d'en améliorer la qualité, plusieurs solution: </br>
<ol>
    <li>Rotation de l'image de départ puis crop dans l'image résultant</li>
    <li>Réduction dimensions de l'image</li>
</ol>

La solution générale serait d'appliquer cela directement dans le notebook colab. On fera ces modifications aléatoirement et on comptera sur cela pour assurer la différence entre les diférents lots (entrainement, vérification et test)

Récupération des images sources (noms de fichiers numéros)

In [ ]:
images = []
noises = []

i = 1
while os.path.isfile("Images_source/clean/"+str(i)+".jpg"):
    images.append("Images_source/clean/"+str(i)+".jpg")
    i += 1

i = 1
while os.path.isfile("Images_source/noise/"+str(i)+".jpg"):
    noises.append("Images_source/noise/"+str(i)+".jpg")
    i += 1

On crée une fonction qui renvoie l'image créée en :</br>
<ol>
    <li>Tirant une image aléatoirement une image dans le dossier des images source</li>
    <li>Appliquant :</br>
    <ol>
    <li>Rotation libre</li>
    <li>Modification de taille aléatoire mais de façon à tjrs obtenir une image de dimensions attendues en sortie (image carrée tjrs) dépendant de la modification de rotation</li>
    </ol>
</ol>

In [1]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):
    '''
    Return a total of `num` random samples and labels. 
    '''
    print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5), image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée

      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]

      noiseTensor.append(np.array(resultNoise,np.uint8))
      imageTensor.append(np.array(resultImage,np.uint8))
    return np.array(noiseTensor), np.array(imageTensor)

En fait le nombre on peut ajouter des couches en augmentant le nombre de neurones au milieu du réseau. Cela permet de mieux faire converger le modèle : `EssaiLotRemaniePlusNeurones`

In [1]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 5
print("Essai avec "+str(i)+" couches")
nom = 'EssaiLotRemaniePlusNeurones'+str(i)

tf.reset_default_graph()

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    
    dense0 = tf.layers.dense(X,3,name='dense0')
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,3,name='dense1')
    print('Shape of dense1 : '+str(dense1.get_shape()))
    dense2 = tf.layers.dense(dense1,100,name='dense2')
    print('Shape of dense2 : '+str(dense2.get_shape()))
    dense3 = tf.layers.dense(dense2,100,name='dense3')
    print('Shape of dense3 : '+str(dense3.get_shape()))
    output = tf.layers.dense(dense3,3,name='dense4')
    print('Shape of output : '+str(output.get_shape()))
    

    with tf.name_scope('Optimizer'):
      optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
      loss0 = tf.reduce_mean(tf.square(output - y))
      training_op0 = optimizer.minimize(loss0)

    with tf.name_scope("init"):
      init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
      loss_saver0 = tf.summary.scalar("Loss",loss0)
      kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/kernel')[0]
      bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense0/bias')[0]
      kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/kernel')[0]
      bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense1/bias')[0]
      kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/kernel')[0]
      bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense2/bias')[0]
      kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense3/kernel')[0]
      bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense3/bias')[0]
      kernel4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense4/kernel')[0]
      bias4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'dense4/bias')[0]


      kernel_saver0 = tf.summary.histogram("Kernel0",kernel0)
      bias_saver0 = tf.summary.histogram("Bias0",bias0)
      kernel_saver1 = tf.summary.histogram("Kernel1",kernel1)
      bias_saver1 = tf.summary.histogram("Bias1",bias1)
      kernel_saver2 = tf.summary.histogram("Kernel2",kernel2)
      bias_saver2 = tf.summary.histogram("Bias2",bias2)
      kernel_saver3 = tf.summary.histogram("Kernel3",kernel3)
      bias_saver3 = tf.summary.histogram("Bias3",bias3)
      kernel_saver4 = tf.summary.histogram("Kernel4",kernel4)
      bias_saver4 = tf.summary.histogram("Bias4",bias4)

      print(os.path.split(X.name))
      summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
      saver = tf.train.Saver()

      with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
      global Lloss
      if len(Lloss) < duree:
          Lloss.append(lastLoss)
          return False
      else:
          Lloss = Lloss[1:duree] + [lastLoss]
          L = Lloss[:]
          L.sort(reverse=True)
          if L[0] <= seuil:
            if decroissance == True:
              return True if L == Lloss else False
            else:
              return True
          return False

    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, noises,images,height)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel0 = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias0 = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel0,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias0,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)
                  
                  summary_strKernel2 = kernel_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias2 = bias_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel2,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias2,epoch*n_batches_per_epoch+iteration)
                  
                  summary_strKernel3 = kernel_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias3 = bias_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel3,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias3,epoch*n_batches_per_epoch+iteration)

                  summary_strKernel4 = kernel_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_strBias4 = bias_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_strKernel4,epoch*n_batches_per_epoch+iteration)
                  summary_writer0.add_summary(summary_strBias4,epoch*n_batches_per_epoch+iteration)
                  
            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
            if arret(loss_train0,100,6) == True:
                break
        summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer0.close()

Essai avec 5 couches


NameError: name 'tf' is not defined

Pour les <b>couches de convolution</b></br>
Après avoir visionné cette vidéo <a href='https://www.youtube.com/watch?v=YRhxdVk_sIs&index=7&list=PLSbuiZgaaqsKQf2zuihSpALqH5xQd8b4c&t=0s'>Convolutional Neural Networks (CNNs) explained</a> il apparait que le redimensionnement peut poser des problèmes par rapport à l'apprentissage : en effet, le cercle de diamètre différent de chaque étoile variera suivant la taille de la région de l'image considérée, d'autant plus si on la redimensionne aléatoirement (NB : quelques images de départ ont déjà été redimensionnées pour obtenir des zones exploitables). </br>
Cela permettra néanmoins d'avoir un réseau plus polyvalent et moins sensible aux artéfacts de retouche manuelle. Il serait alors nécessaire d'afficher les caractéritiques apprises par chaque couche pour déterminer ce qui est détecté. </br>
De plus, à la suite de cette vidéo, il apparait que les filtres correspondent aux nombre de formes détectables par la couche de convolution.

Avant de lancer la réalisation de ce modèle, d'après les articles <a href='https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f'>GANs from Scratch 1: A deep introduction</a> et <a href='https://towardsdatascience.com/generative-adversarial-networks-using-tensorflow-c8f4518406df'>Generative Adversarial Networks using Tensorflow</a>, il apparait que le type de réseau de neurone que je cherche à réaliser s'appelle un <a href='https://fr.wikipedia.org/wiki/R%C3%A9seaux_antagonistes_g%C3%A9n%C3%A9ratifs'>Réseaux antagonistes génératifs</a>. D'après <a href='https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/gan.py'>cet exemple</a> qui illustre le code générale pour réaliser ce type de réseau --> réalisation du nouveau code. Chercher également à clarifier le code et la création du nécessaire pour l'analyse des couches grâce à un ensemble de fonctions.

Ci-dessous, la version `GAN_essai_1`, la version 0 ayant servie uniquement à mettre en place le système de fonction, à faire correctement la conversion entre différents types d'objets pour permettre l'enregistrement de toutes les variables. 

A noter que dans ce type de model, la fonction de coût utilise un log ce qui pose des problèmes de divergences ce qui a nécessité un plancher minimum pour ne pas atteindre 0.

In [ ]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):
    '''
    Return a total of `num` random samples and labels. 
    '''
#     print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
#     import matplotlib.pyplot as plt
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5), image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée

      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
#       print('CentreRow : ')
#       print((taille//2,rows-taille//2))
#       print('CentreCols : ')
#       print((taille//2,cols-taille//2))
#       tmp = taille//2
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))
#       print('Rotation : '+str(rotatedImage.shape))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
#       print('Crop : '+str(resultImage.shape))
#       plt.imshow(resultImage)

#       noiseTensor.append(np.array(resultNoise,np.uint8))
#       imageTensor.append(np.array(resultImage,np.uint8))
#       print(imageTensor)
#     plt.show()
    return np.array(noiseTensor,np.uint8), np.array(imageTensor,np.uint8)
# next_batch(3,noises,images,200)

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 1
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
    print("Taille de l'entrée : "+str(X.get_shape()))
    def denseLayer(input,nbNeurones,nom,start=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    def generator(input,neurones):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'denseGenerator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'denseGenerator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'denseGenerator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    def discriminator(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="disc_input")
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generator(gen_input,[100,100,100])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminator(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,[100,100,100],'Faux')
    
    gen_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_faux,1e-10,10)))
    disc_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_vrai,1e-10,10)) + tf.log(tf.clip_by_value(1. - disc_faux,1e-10,10)))#Attention peut tendre vers -
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    disc_vars = disc_vars1+disc_vars2
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        for i in range(1,251):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,_,gl,dl = sess.run([train_gen,train_disc,gen_loss,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 5 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Pour analyser au mieux les valeurs de coût affichées, revoir la vidéo <a href='https://youtu.be/JikwRMXJzk0?list=PLSbuiZgaaqsKQf2zuihSpALqH5xQd8b4c'>Les réseaux adversariaux (GAN) | Intelligence artificielle 49</a>. En effet les fonctions de coût pour les `discriminator` et le `generator` sont générées à partir de log mais de façons différentes :

In [ ]:
gen_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_faux,1e-10,10)))
disc_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_vrai,1e-10,10)) + tf.log(tf.clip_by_value(1. - disc_faux,1e-10,10)))

Question à traiter également :</br>
(En sachant que le générateur a à l'origine pour rôle de créer des vrais/fausses images à l'origine (revoir la vidéo pour préciser ce rôle))</br>
<b>Le générateur doit-il sortir une image en pseudo-rgb ?</b>

A regarder aussi : <a href='https://youtu.be/dXB-KQYkzNU'>vidéo sur la normalisation du batch</a> (discutttée en fin de la vidéo <a href='https://youtu.be/JikwRMXJzk0?list=PLSbuiZgaaqsKQf2zuihSpALqH5xQd8b4c'>Les réseaux adversariaux (GAN) | Intelligence artificielle 49</a>)

Code modifié :

In [ ]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):
    '''
    Return a total of `num` random samples and labels. 
    '''
#     print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
#     import matplotlib.pyplot as plt
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5), image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée

      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
#       print('CentreRow : ')
#       print((taille//2,rows-taille//2))
#       print('CentreCols : ')
#       print((taille//2,cols-taille//2))
#       tmp = taille//2
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))
#       print('Rotation : '+str(rotatedImage.shape))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
#       print('Crop : '+str(resultImage.shape))
#       plt.imshow(resultImage)

      noiseTensor.append(np.array(resultNoise,np.uint8))
      imageTensor.append(np.array(resultImage,np.uint8))
#       print(imageTensor)
#     plt.show()
    noiseTensor,imageTensor = np.array(noiseTensor,np.uint8), np.array(imageTensor,np.uint8)
    return noiseTensor, imageTensor
# next_batch(3,noises,images,200)

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 1
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
    def denseLayer(input,nbNeurones,nom,start=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    def generator(input,neurones):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'denseGenerator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'denseGenerator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'denseGenerator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    def discriminator(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="disc_input")
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generator(gen_input,[100,100,100])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminator(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,[100,100,100],'Faux')
    
    gen_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_faux,1e-10,10)))
    disc_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_vrai,1e-10,10)) + tf.log(tf.clip_by_value(1. - disc_faux,1e-10,10)))#Attention peut tendre vers -
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    disc_vars = disc_vars1+disc_vars2
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        for i in range(1,251):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,_,gl,dl = sess.run([train_gen,train_disc,gen_loss,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 5 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Résultat : problèmes d'enregistrement : seule l'entrée sortie du générateur est affichée. On note également qu'entre 2 essais (un où l'enregistrement a échoué (pour une raison inconnue)) si la fonction de coût du générateur arrive à la valeur limitée pour éviter de tendre vers 0, le générateur reste bloqué à ce stade et n'apprend plus.

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 0
nom = 'GAN_essai_test'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
    def denseLayer(input,nbNeurones,nom,start=False,generator=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    def generator(input,neurones):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'denseGenerator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'denseGenerator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'denseGenerator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    def discriminator(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="disc_input")
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generator(gen_input,[100,100,100])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminator(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,[100,100,100],'Faux')
    
    gen_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_faux,1e-10,10)))
    disc_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_vrai,1e-10,10)) + tf.log(tf.clip_by_value(1. - disc_faux,1e-10,10)))#Attention peut tendre vers -
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    disc_vars = disc_vars1+disc_vars2
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        for i in range(1,6):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,_,gl,dl = sess.run([train_gen,train_disc,gen_loss,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 5 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Le code ci-dessous affiche bien les images mais donne des résultats très colorés

D'après la vidéo <a href='https://youtu.be/0VPQHbMvGzg'>Generative Adversarial Networks (LIVE)</a>

Nb moyen de sélectionner les variables à entrainer : <a href='https://youtu.be/0VPQHbMvGzg?t=3147'>ici</a>

Discriminateur : Encodeur</br>
<ul>
    <li>Doit déterminer si la probabilité que l'image en entrée soit une vraie image ou une fausse</li>
    <li>Opération d'encodage de l'information :
        <ul style="list-style-type:&rarr">
            <li>Tester la présence d'un objet</li>
            <li>Pas besoin de le savoir en fonction de la position de l'objet</li>
            <li>Convolution</li>
            <li>Finit par couche intégralement connectée car après qu'on ait détecter les caractéristiques, il faut les mettre en relation</li>
        </ul>
    </li>
</ul>

Générateur : Décodeur
<ul>
    <li>Part normalement d'une gaussienne</li>
    <li>Fait plein de petites images</li>
    <li>Convolution</li>
    <li>Normalisation<br><img src="https://cdn-images-1.medium.com/max/800/1*Hiq-rLFGDpESpr8QNsJ1jg.png" filter='invert(1)' -webkit-filter='invert(1)'>
    </li>
</ul>
    

Objectifs : 
<ol>
    <li>Fonction pour ajouter ajout couche de convolution</li>
    <li>Fonction pour sauvegarde et réentrainement</li>
    <li>Implémenter DiscoGan</li>
    <li>Modifier entrainement pour entrainer plus de fois le générateur que le discriminateur</li>
    <li>Modifier façon dont récupère variables pour entrainement</li>
</ol>

GAN_essai_test3 : Essai à nouveau avec le réseau</br>
GAN_essai_test4 : change fonction de coût : enlève le log ; batch_normalization ; <a href='https://github.com/llSourcell/Pokemon_GAN/blob/master/pokeGAN.py'>source : Generating Pokemon with a Generative Adversarial Network</a>

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 4
nom = 'GAN_essai_test'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
    def denseLayer(input,nbNeurones,nom,start=False,generator=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    def convLayer(input,kernel,filters,start=False,generator=False):
      couche = tf.layers.conv2d(input, filters=filters, kernel_size=kernel,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    def generator(input,neurones):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'denseGenerator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'denseGenerator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
          if i != 0 and i%1 == 0:
            couche = tf.contrib.layers.batch_norm(couche, is_training=True, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='normalization_'+str(i))
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'denseGenerator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    def discriminator(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generator(gen_input,[100,100,100])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminator(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,[100,100,100],'Faux')
    
#     gen_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_faux,1e-10,10)))
#     disc_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(disc_vrai,1e-10,10)) + tf.log(tf.clip_by_value(1. - disc_faux,1e-10,10)))#Attention peut tendre vers -
    
    disc_loss = tf.reduce_mean(disc_faux) - tf.reduce_mean(disc_vrai)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars1+disc_vars2
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        for i in range(1,50):
          for _ in range(6):#Entraine générateur
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          for _ in range(1):#Entraine discriminateur
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 5 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          if i % 5 == 10:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Pas de convergence : cela doit provenir de l'incohérence du système : en effet, le générateur est censé être un encodeur donc réseau neuronal avec des couches de convolution pour réduire la dimensionnalité des données, et les discriminateurs les décodeurs. Il se pose également la question de l'influence de l'influence de la normalisation par batch : est-ce cela qui permet d'avoir une courbe d'avoir une courbe d'apprentissage plus 'lisse' ou bien est-ce la structure même de GAN avec discriminateur et générateur qui induit cela. Et surtout est-ce vraiment la question comme le modèle n'est pas adapté... Le problème au niveau du log pouvait également provenir du fait qu'on travaillait avec des entier non signés sur 8 bits : le réseau n'avait eut-être pas assez d'amplitude pour séparer les éléments importants du bruit...

Mise en place de ce modèle qui plante à cause des couches de convolution de noyaux trop grands

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 6
nom = 'GAN_essai_test_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,generator=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='normalization_'+nom[-1])
      if poolingType == 'mean':
        couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
      else:
        couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
      sortieImage = tf.summary.image("output/"+nom if start==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if end == True else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def generator(inpt,kernels,num_filters,pooling,poolingTypes):
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'denseGenerator0',True)
      saver = [kernel_saver,bias_saver]
      kernelBiasList = [kernel,bias]
      sortiesImages = [sortieImage]
#       for i in range(1,len(kernels)-1):
#         couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'denseGenerator'+str(i),end=True)
#         kernelBiasList.append(kernel)
#         kernelBiasList.append(bias)
#         saver.append(kernel_saver)
#         saver.append(bias_saver)
#         sortiesImages.append(sortieImage)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'denseGenerator'+str(len(pooling)+1))
      kernelBiasList.append(kernel)
      kernelBiasList.append(bias)
      saver.append(kernel_saver)
      saver.append(bias_saver)
      sortiesImages.append(sortieImage)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,neurones[0],'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages

    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generator(gen_input,[55,27,13],[96,256,384],[3,3,1],['mean','mean','mean'])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminator(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,[384,100,100],'Faux')
    
    disc_loss = tf.reduce_mean(disc_faux) - tf.reduce_mean(disc_vrai)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars1+disc_vars2
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        print('Entrainement....')
        for i in range(1,50):
          for _ in range(6):#Entraine générateur
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          for _ in range(1):#Entraine discriminateur
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 2 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          if i % 5 == 10:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

D'où démultiplication suivant la note sur les couches de convolution du livre sur tensorflow : 2 couches de 3*3 équivalent à 1 couche de 9*9. Ici comme la source de ce principe n'a pas été éclaircie mais que globalement on perçoit la raison de ce principe, on spécifiera manuellement la démultiplication quitte à faire après une fonction renvoit la liste de démultiplication automatiquement...

In [1]:
a = [[2]+3*[3],3*[3],2*[2]+[3]]
print(a)

[[2, 3, 3, 3], [3, 3, 3], [2, 2, 3]]


In [6]:
import numpy as np
np.linspace(60,399,34,dtype=np.int)

array([ 60,  70,  80,  90, 101, 111, 121, 131, 142, 152, 162, 173, 183,
       193, 203, 214, 224, 234, 244, 255, 265, 275, 286, 296, 306, 316,
       327, 337, 347, 357, 368, 378, 388, 399])

A noter une tentative avec reshape de tensorflow mais comme pour numpy il garde le même nombre d'éléments.

Essai sans TPU ou GPU. Parfosi le problème vient de là... A noter que dès le début 4Gb de RAM sont occupés par le modèle. La session plante toujours

Essai en travaillant avec des images de 199 et non 399 --> Fonctionne ; on relance en essai 8

In [ ]:
height = 199
width = 199
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 7
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    #generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[55,27,13],[96,256,384],[3,3,1],['mean','mean','mean'])
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminatorDense(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,np.linspace(100,199,7,dtype=np.int),np.linspace(150,3,7,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars2 + disc_vars1
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        print('Entrainement....')
        for i in range(1,5):
          print('Generateur')
          for a in range(6):#Entraine générateur
            print('Iteration' +str(a),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print('Discriminateurs')
          for _ in range(1):#Entraine discriminateur
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          if i % 1 == 0:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          if i % 5 == 10:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Taille de gen_input : (7, 199, 199, 3)</br>
Taille de convGenerator0_split_0 : (7, 199, 199, 50)</br>
Taille de convGenerator0_split_1 : (7, 199, 199, 50)</br>
Taille de convGenerator1_split_0 : (7, 100, 100, 100)</br>
Taille de convGenerator2_split_0 : (7, 100, 100, 150)</br>
Taille de convGenerator2_split_1 : (7, 100, 100, 150)</br>
Taille de denseVraiDiscriminator0 : (7, 199, 199, 3)</br>
Taille de denseVraiDiscriminator1 : (7, 199, 199, 100)</br>
Taille de denseVraiDiscriminator2 : (7, 199, 199, 100)</br>
Taille de denseVraiDiscriminator3 : (7, 199, 199, 100)</br>
Taille de denseVraiDiscriminator4 : (7, 199, 199, 3)</br>
Taille de l'entrée du discriminateur : (7, 50, 50, 150)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux0 : 51</br>
Taille de deconvDiscriminatorFaux0 : (7, 100, 100, 150)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux1 : 17</br>
Taille de deconvDiscriminatorFaux1 : (7, 116, 116, 125)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux2 : 18</br>
Taille de deconvDiscriminatorFaux2 : (7, 133, 133, 101)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux3 : 17</br>
Taille de deconvDiscriminatorFaux3 : (7, 149, 149, 76)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux4 : 18</br>
Taille de deconvDiscriminatorFaux4 : (7, 166, 166, 52)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux5 : 17</br>
Taille de deconvDiscriminatorFaux5 : (7, 182, 182, 27)</br>
Taille du noyau de deconvolution de deconvDiscriminatorFaux6 : 18</br>
Taille de deconvDiscriminatorFaux6 : (7, 199, 199, 3)</br>
Taille de sortie disc_faux : (7, 199, 199, 3)</br>
Taille de sortie disc_input : (7, 199, 199, 3)</br>
Generator variables : ...</br>
Trainable variables : ...</br>
Entrainement....</br>
Generateur</br>
Iteration0, Iteration1, Iteration2, Iteration3, Iteration4, Iteration5, Discriminateurs</br>
Step 1: Generator Loss: -0.000001, Discriminator Loss: -11.519882</br>
Generateur</br>
Iteration0, Iteration1, Iteration2, Iteration3, Iteration4, Iteration5, Discriminateurs</br>
Step 2: Generator Loss: -0.000005, Discriminator Loss: -13.513853</br>
Generateur</br>
Iteration0, Iteration1, Iteration2, Iteration3, Iteration4, Iteration5, Discriminateurs</br>
Step 3: Generator Loss: -0.000002, Discriminator Loss: -21.686550</br>
Generateur</br>
Iteration0, Iteration1, Iteration2, Iteration3, Iteration4, Iteration5, Discriminateurs</br>
Step 4: Generator Loss: -0.000005, Discriminator Loss: -18.946047

Visiblement, je me suis trompé dans les priorité d'entrainement : le générateur est trop précis et le discriminateur pas assez.</br>
En conséquence, on ajuste au fur et à mesure de la façon suivante : </br>
Avec générateur et discriminateur on doit toujours avoir 10 entrainement par etape d'entrainement</br>
On pondère en fonction le nombre d'entrainement

D'où l'essai 9 :

In [ ]:
height = 199
width = 199
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 9
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    #generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[55,27,13],[96,256,384],[3,3,1],['mean','mean','mean'])
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminatorDense(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,np.linspace(100,199,7,dtype=np.int),np.linspace(150,3,7,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars2 + disc_vars1
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        print('Entrainement....')
        lastDisc, lastGen = 1,1
        for i in range(1,5):
          nbEntrainementGenerateur,nbEntrainementDiscriminateur = 5,5
          if lastDisc > lastGen:
            nbEntrainementGenerateur = 5 + 4-int(4*lastGen/lastDisc)
            nbEntrainementDiscriminateur = 10 - nbEntrainementGenerateur
          elif lastGen > lastDisc:
            nbEntrainementDiscriminateur = 5 + 4-int(4*lastDisc/lastGen)
            nbEntrainementGenerateur = 10 - nbEntrainementDiscriminateur
          print('Generateur : ',end='')
          for a in range(nbEntrainementGenerateur):#Entraine générateur
            print('Iteration ' +str(a),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          print('Discriminateur : ',end='')
          for _ in range(nbEntrainementDiscriminateur):#Entraine discriminateur
            print('Iteration ' +str(a),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          lastDisc, lastGen = dl,gl
          if i % 5 == 10:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Mais à cause du manque d'iteration le modèle n'a pas renvoyé d'image.

On augmente le nombre d'iteration et pour tenter d'améliorer la précision on augmente la zone d'image analysée. Faisons un essai pour voir ce que donne l'image même si la fonction de coût qui minimisera par la suite la moyenne de l'image de sortie a un comportement obscur... Elle permet peut-être de ne pas avoir de valeurs trop extrèmes. D'où l'essai 10

In [ ]:
import cv2
import numpy as np
def next_batch(batch_size, noises, images,tailleAttendue):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    '''
    Return a total of `num` random samples and labels. 
    '''
#     print("Size of {}".format(len(noises)))
    noiseTensor = []
    imageTensor = []
#     import matplotlib.pyplot as plt
    for i in range(batch_size):
      choix = np.random.randint(0,len(images))#choix aléatoire de l'image
      image = cv2.imread(images[choix],1)#Ouvre en rgb l'image nettoyée
      noise = cv2.imread(noises[choix],1)#Ouvre en rgb l'image bruitée
      angle = np.random.randint(0,90)
      taille = np.random.randint(int(tailleAttendue*2**0.5)*4, image.shape[0]+1)# Racine de deux pour pouvoir toujours récupérer la taille désirée
      ##ATENTION : ci-dessus peut porter à erreur si dépasse image.shape[0]+1
      resizedImage = cv2.resize(image,(taille,taille))
      resizedNoise = cv2.resize(noise,(taille,taille))
      
      rows,cols = resizedImage.shape[:2]
#       print('CentreRow : ')
#       print((taille//2,rows-taille//2))
#       print('CentreCols : ')
#       print((taille//2,cols-taille//2))
#       tmp = taille//2
      
      centre = (np.random.randint(taille//2,rows-taille//2) if taille//2<rows-taille//2 else taille//2,np.random.randint(taille//2,cols-taille//2) if taille//2<cols-taille//2 else taille//2)
      M = cv2.getRotationMatrix2D(centre,angle,1)

      rotatedImage = cv2.warpAffine(resizedImage,M,(cols,rows))
      rotatedNoise = cv2.warpAffine(resizedNoise,M,(cols,rows))
#       print('Rotation : '+str(rotatedImage.shape))

      resultImage = rotatedImage[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
      resultNoise = rotatedNoise[centre[0]-tailleAttendue//2:centre[0]+tailleAttendue//2+1,centre[1]-tailleAttendue//2:centre[1]+tailleAttendue//2+1]
#       print('Crop : '+str(resultImage.shape))
#       plt.imshow(resultImage)

      noiseTensor.append(np.array(resultNoise,np.uint8))
      imageTensor.append(np.array(resultImage,np.uint8))
#       print(imageTensor)
#     plt.show()
    noiseTensor,imageTensor = np.array(noiseTensor,np.float32), np.array(imageTensor,np.float32)
    return noiseTensor, imageTensor
# next_batch(3,noises,images,200)

In [ ]:
height = 199
width = 199
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 10
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(input,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(input,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    #generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[55,27,13],[96,256,384],[3,3,1],['mean','mean','mean'])
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminatorDense(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,np.linspace(100,199,7,dtype=np.int),np.linspace(150,3,7,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars2 + disc_vars1
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        init.run()
        print('Entrainement....')
        lastDisc, lastGen = 1,1
        for i in range(1,5):
          nbEntrainementGenerateur,nbEntrainementDiscriminateur = 1,9# Pour favoriser le discriminateur d'emblée
          if lastDisc > lastGen:
            nbEntrainementDiscriminateur = 5 + 4-int(4*lastGen/lastDisc)
            nbEntrainementGenerateur = 10 - nbEntrainementDiscriminateur
          elif lastGen > lastDisc:
            nbEntrainementGenerateur = 5 + 4-int(4*lastDisc/lastGen)
            nbEntrainementDiscriminateur = 10 - nbEntrainementGenerateur
          print('Generateur : ',end='')
          for a in range(nbEntrainementGenerateur):#Entraine générateur
            print('Iteration ' +str(a),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          print('Discriminateur : ',end='')
          for b in range(nbEntrainementDiscriminateur):#Entraine discriminateur
            print('Iteration ' +str(b),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          lastDisc, lastGen = abs(dl),abs(gl)
          if i % 5 == 10:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

In [ ]:
height = 199
width = 199
channels = 3

batch_size = 7
learning_rate = 0.0002
i = 11
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(inpt,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    #generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[55,27,13],[96,256,384],[3,3,1],['mean','mean','mean'])
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    disc_vrai,disc_vars1,disc_saver1,disc_vrai_sortiesImages = discriminatorDense(disc_input,[100,100,100],'Vrai')
    disc_faux,disc_vars2,disc_saver2,disc_faux_sortiesImages = discriminator(generator,np.linspace(100,199,7,dtype=np.int),np.linspace(150,3,7,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
    gen_loss = -tf.reduce_mean(disc_faux)  # This optimizes the generator
  
    loss_saver_gen = tf.summary.scalar("Generator_loss",gen_loss)
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",gen_loss)
    
    optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    disc_vars = disc_vars2 + disc_vars1
    saver = gen_saver + disc_saver2 + disc_saver1
    print("Generator variables : "+str(gen_vars))
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
    train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
#         init.run()
        tf_saver.restore(sess, 'model/model'+nom+'.ckpt')
        print('Entrainement....')
        lastDisc, lastGen = 1,1
        for i in range(6,50):
          nbEntrainementGenerateur,nbEntrainementDiscriminateur = 1,9# Pour favoriser le discriminateur d'emblée
          if lastDisc > lastGen:
            nbEntrainementDiscriminateur = 5 + 4-int(4*lastGen/lastDisc)
            nbEntrainementGenerateur = 10 - nbEntrainementDiscriminateur
          elif lastGen > lastDisc:
            nbEntrainementGenerateur = 5 + 4-int(4*lastDisc/lastGen)
            nbEntrainementDiscriminateur = 10 - nbEntrainementGenerateur
          print('Generateur : ',end='')
          for a in range(nbEntrainementGenerateur):#Entraine générateur
            print('Iteration ' +str(a),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,gl = sess.run([train_gen,gen_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          print('Discriminateur : ',end='')
          for b in range(nbEntrainementDiscriminateur):#Entraine discriminateur
            print('Iteration ' +str(b),end=', ')
            gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
            _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          summary_str = loss_saver_gen.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(gen_saver,gen_inpt,disc_inpt)
          save(disc_saver1,gen_inpt,disc_inpt)
          save(disc_saver2,gen_inpt,disc_inpt)
          print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
          lastDisc, lastGen = abs(dl),abs(gl)
          if i % 5 == 0:
            tf_saver.save(sess,'/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_vrai_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Comme l'apprentissage du générateur dégrade plutôt l'image en lui imposant une image aussi lisse que possible, on revient sur un fonctionnement d'autoencodeur mais qui sera cette fois plus adapté car comportant des couches de déconvolution

Après plus de 50 iterations en fait problème d'enregistrement. Après modifications on tente à nouveau : essai 13

In [ ]:
height = 199
width = 199
channels = 3
forceInit = True

batch_size = 7
learning_rate = 0.0002
i = 13
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print('precDimension : '+str(precDimension))
#           print('outputSize : '+str(outputSize))
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(inpt,3,'dense'+str(ID)+'Discriminator0',True)
        saver = [kernel_saver,bias_saver]
        kernelBiasList = [kernel,bias]
        sortiesImages = [sortieImage]
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'Discriminator'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,3,'dense'+str(ID)+'Discriminator'+str(len(neurones)+1))
        kernelBiasList.append(kernel)
        kernelBiasList.append(bias)
        saver.append(kernel_saver)
        saver.append(bias_saver)
        sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    print(generator.get_shape())
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,51):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Après cet essai on ajoute des couches intégralement connectées au milieu pour améliorer le traitement 14

In [ ]:
height = 199
width = 199
channels = 3
forceInit = True

batch_size = 7
learning_rate = 0.0002
i = 14
nom = 'GAN_essai_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver,sortiesImages = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,101):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          print()
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Rien ne converge. Pour l'essai 15_C et 15_N on essai de reproduire avec la structure du 14 l'image clean ou l'image noise

Le but serait de mettre en place l'architecture suivante :

<img src="D:/Github/StructureIA.jpg">

15_N

In [ ]:
height = 199
width = 199
channels = 3
forceInit = False

batch_size = 7
learning_rate = 0.0002
i = 15
nom = 'GAN_essai_'+str(i)+'_N'

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver,sortiesImages = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-gen_input)  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,101):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

15_C

In [ ]:
height = 199
width = 199
channels = 3
forceInit = False

batch_size = 7
learning_rate = 0.0002
i = 15
nom = 'GAN_essai_'+str(i)+'_C'

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(disc_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver,sortiesImages = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    disc_loss = tf.reduce_mean(disc_faux-disc_input)  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,101):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
          save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')
summary_writer.close()

Comme visiblement (1/5 entrainement fini pour les 2 modèles) le modèle ne converge pas, il manquerait des images. En effet, le lot de données doit être trop restreint pour dégager la cohérence du lot.  Pour cela on prendra des images sur astrobin on détériorera l'image artificiellement pour observer si cela permet d'améliorer l'apprentissage. En effet, cela permettrait de montrer plus d'exemples pertinents au modèle (avec plus d'objets). A ce moment là on ne redimensionnerait que peu la taille de l'image pour permettre au modèle de se concentrer sur l'apprentissage des caractéristiques propres aux 'objets du ciel profond'

On envisage l'algorithme suivant pour détériorer les images artificiellement avec tout d'abord un exemple de fonctionnement en local :

In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
image = cv2.imread('D:/Google drive/TIPE/Images_source/clean/3.jpg',1)
noiseImage = cv2.imread('D:/Google drive/TIPE/Images_source/noise/3.jpg',1)
row,col,ch = image.shape
image = cv2.resize(image,(col//5,row//5))
row,col,ch = image.shape
gauss = np.array(np.random.normal(np.mean(noiseImage),1,(row,col,ch)),dtype=np.int)
print(gauss.shape)
noisy = image + gauss
imageContrast = (np.array(image,dtype=np.float64)-np.mean(image))//2+np.mean(image)
imageContrast = np.array(imageContrast,dtype=np.float64)
print(noisy.shape)
print(type(image)==type(noisy))
# print(noisy[20:50,20:50,0])
# cv2.destroyAllWindows()
plt.figure(3)
plt.imshow(image)
plt.figure(2)
plt.imshow(image//np.random.randint(3,7)+gauss)
plt.figure(1)
plt.imshow(noiseImage)
plt.show()

(732, 1100, 3)
(732, 1100, 3)
True


FigureCanvasNbAgg()

Néanmoins ce programme nécessite la valeur moyenne de l'image bruitée donc l'image bruitée ce qui n'est pas le but.</br>
Sur mes images affichons les valeurs moyennes de chaque image non traitée : 

In [50]:
temps = ['27/08/2018,22:04','05/08/2018,01:29','02/08/2018,01:10','02/08/2018,00:57','02/08/2018,01:10','06/08/2018,00:22','28/08/2018,00:31']
print("[",end='')
for elem in temps[:-1]:
    sselemts = elem.split(',')
    date = sselemts[0].split('/')
    print("'"+date[-1]+"-"+date[1]+"-"+date[0]+" "+sselemts[-1]+":00',",end='')
sselemts = elem.split(',')
date = sselemts[0].split('/')
print("'"+date[-1]+"-"+date[1]+"-"+date[0]+" "+sselemts[-1]+":00'",end='')
print(']')

['2018-08-27 22:04:00','2018-08-05 01:29:00','2018-08-02 01:10:00','2018-08-02 00:57:00','2018-08-02 01:10:00','2018-08-06 00:22:00','2018-08-06 00:22:00']


http://docs.astropy.org/en/stable/generated/examples/coordinates/plot_obs-planning.html

https://astroplan.readthedocs.io/en/latest/_modules/astroplan/moon.html ne fonctionnent pas

Essai avec https://astroplan.readthedocs.io/en/v0.1/_modules/astroplan/moon.html

In [51]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

# Third-party
import numpy as np
from astropy.coordinates import get_moon, get_sun

__all__ = ["moon_phase_angle", "moon_illumination"]
def moon_illumination(time, ephemeris=None):
    """
    Calculate fraction of the moon illuminated.

    Parameters
    ----------
    time : `~astropy.time.Time`
        Time of observation

    ephemeris : str, optional
        Ephemeris to use.  If not given, use the one set with
        `~astropy.coordinates.solar_system_ephemeris` (which is
        set to 'builtin' by default).

    Returns
    -------
    k : float
        Fraction of moon illuminated
    """
    i = moon_phase_angle(time, ephemeris=ephemeris)
    k = (1 + np.cos(i))/2.0
    return k.value

def moon_phase_angle(time, ephemeris=None):
    """
    Calculate lunar orbital phase in radians.

    Parameters
    ----------
    time : `~astropy.time.Time`
        Time of observation

    ephemeris : str, optional
        Ephemeris to use.  If not given, use the one set with
        `~astropy.coordinates.solar_system_ephemeris` (which is
        set to 'builtin' by default).

    Returns
    -------
    i : float
        Phase angle of the moon [radians]
    """
    # TODO: cache these sun/moon SkyCoord objects

    sun = get_sun(time)
    moon = get_moon(time, ephemeris=ephemeris)
    elongation = sun.separation(moon)
    return np.arctan2(sun.distance*np.sin(elongation),
                      moon.distance - sun.distance*np.cos(elongation))

In [52]:
def sortOnList(listList,indexListSort):
    arraySort = [(list(range(len(listList[0])))[i],listList[indexListSort][i]) for i in range(len(listList[indexListSort]))]
#     print(arraySort)
    arraySort.sort(key=lambda x: x[1])
    sortedTupple = []
    for i in range(len(listList)):
        listIndices = [arraySort[a][0] for a in range(len(arraySort))]
        array = [listList[i][indice] for indice in listIndices]
        sortedTupple.append(array)
    return sortedTupple

In [53]:
import cv2
temps = ['2018-08-27 22:04:00','2018-08-05 01:29:00','2018-08-02 01:10:00','2018-08-02 00:57:00','2018-08-02 01:10:00','2018-08-06 00:22:00','2018-08-06 00:22:00']
lune = [98.9,42.4,73.0,73.0,73.0,31.6,31.6]
distanceObjetLune = [43+23/60,92+52/60,107+22/60,62+33/60,74+12/60,99+38/60,69+28/60]
indices = list(range(1,8))
moyennes = []
import numpy as np
for i in range(1,8):
    imageNoise = cv2.imread('D:/Google drive/TIPE/Images_source/noise/'+str(i)+'.jpg',1)
    moyennes.append(np.mean(imageNoise))
    print("Moyenne de %f, phase de %f, distanceLune %f pour l'image %i" % (moyennes[-1],lune[i-1],distanceObjetLune[i-1],i))

Moyenne de 51.570884, phase de 98.900000, distanceLune 43.383333 pour l'image 1
Moyenne de 115.856622, phase de 42.400000, distanceLune 92.866667 pour l'image 2
Moyenne de 42.923483, phase de 73.000000, distanceLune 107.366667 pour l'image 3
Moyenne de 19.210552, phase de 73.000000, distanceLune 62.550000 pour l'image 4
Moyenne de 1.992748, phase de 73.000000, distanceLune 74.200000 pour l'image 5
Moyenne de 155.987816, phase de 31.600000, distanceLune 99.633333 pour l'image 6
Moyenne de 68.023631, phase de 31.600000, distanceLune 69.466667 pour l'image 7


In [54]:
indicesSortLn,lnSortLn,dSortLn,mSortLn = sortOnList([indices,lune,distanceObjetLune,moyennes],1)
indicesSortD,lnSortD,dSortD,mSortD = sortOnList([indices,lune,distanceObjetLune,moyennes],2)
import matplotlib.pyplot as plt
%matplotlib ipympl

plt.figure(1)
plt.xlabel('Illumination lune')
plt.ylabel("g = distance de l'objet à la lune ; b = moyenne de l'image")
plt.plot(lnSortLn,dSortLn,linestyle='None',marker='o',color='g')
for i in range(len(lnSortLn)):
    plt.annotate(str(indicesSortLn[i])+'.jpg',(lnSortLn[i],dSortLn[i]))
plt.plot(lnSortLn,mSortLn,linestyle='None',marker='o',color='b')
for i in range(len(lnSortLn)):
    plt.annotate(str(indicesSortLn[i])+'.jpg',(lnSortD[i],mSortLn[i]))
plt.figure(2)
plt.xlabel("distance de l'objet à la lune")
plt.ylabel("r = moyenne de l'image")
plt.plot(dSortD,(np.array(mSortD)),linestyle='None',marker='o',color='r')
for i in range(len(lnSortD)):
    plt.annotate(str(indicesSortD[i])+'.jpg',(dSortD[i],mSortD[i]))
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Pas de relation particulière... même si la courbe avec moyenne et distance à la lune parait plus intéressante (points moins resserés)

On pourrait utiliser à nouveau une ia pour trouver une potentielle relation entre distance de l'objet observé et moyenne de l'image mais il faudrait plus d'images obligatoirement ici (faisant varier la distance à la lune sans post traitement)

On refait ça sur plus d'images trouvées dans le disque dur.

On renomme déjà les images présentes dans le dossier pour aller plus vite après.

src http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.from_name ; http://docs.astropy.org/en/stable/coordinates/matchsep.html

In [55]:
import os
fichiers = os.listdir("D:/Github/ImagesAnalyseVoile")[:-1]
fichiersComplets = ["D:/Github/ImagesAnalyseVoile/"+fichiers[i] for i in range(len(fichiers))]
# print(fichiers)
dates = []
illuminationLn = []
from PIL import Image
def get_date_taken(path):
    return Image.open(path)._getexif()[36867]
from astroplan import Observer
from astropy import units as u
from astropy.coordinates import EarthLocation,SkyCoord
from astropy.time import Time,TimezoneInfo
location = EarthLocation.from_geodetic((45+56/60)*u.deg, (6+42/60)*u.deg,1000*u.m)
passy = Observer(location=location, name="Passy", timezone=TimezoneInfo(utc_offset=2*u.hour))
nomObjets = []
separationLn = []
ISO = []
tpose = []
moyennes = []
dbObjects = [('M51','13h29m53s +47d11m43s'),('NGC7023','21h01m37s +68d09m48s'),('NGC281','0h52m25s +56d33m54s'),('M27','19h59m36s +22d43m16s'),
             ('M33','1h33m51s +30d39m36s'),('M20','18h02m42s -22d58m19s'),('M13','16h41m42s +36d27m41s')]# Dictionnaire peut-être plus adapté mais bon
def convertCoordFormat(chaine):
    import re
    L = re.split('\s|h|m|s|d',chaine)[:-1]
    del L[3]
#     print(L)
    L = list(map(int,L))
    RA,DEC = L[:3],L[3:]
    RA,DEC = RA[0]+RA[1]/60+RA[2]/3600,DEC[0]+DEC[1]/60+DEC[2]/3600
    return RA,DEC

def searchCoord(name):
    global dbObjects
    i = 0
    elem = dbObjects[0][0]
    while elem != name and i < len(dbObjects):
        i += 1
        elem = dbObjects[i][0]
    return convertCoordFormat(dbObjects[i][1])

for fichier in fichiersComplets:
#     print(fichier,end=', ')
    val = get_date_taken(fichier).split(' ')
    dates.append('-'.join(val[0].split(':'))+' '+val[1])
    time = Time(dates[-1])
    
    imageNoise = cv2.imread(fichier,1)
    moyennes.append(np.mean(imageNoise))
    
    illuminationLn.append(int(passy.moon_illumination(time)*100))
    
    nomObjets.append(str(fichier[32:-4]))#prémâché par lightroom
    
    objet = searchCoord(nomObjets[-1])
    coordObj = SkyCoord(ra=objet[0],dec=objet[1], frame="icrs",unit=(u.hour,u.deg))
    coordLn = get_moon(time=time,location=location)
    sep = coordObj.separation(coordLn)
    separationLn.append(sep.deg)
    
    ISO.append(Image.open(fichiersComplets[i])._getexif()[34855])
    
    tpose.append(Image.open(fichiersComplets[i])._getexif()[33434][0])
    
#     print(objet)
#     print(nomObjets[-1],end=', ')
#     print(sep.deg)
#     print(get_date_taken(fichier))
#     print(val)
#     print(dates[-1])
#     print(str(illuminationLn[-1])+'%')

In [115]:
print(illuminationLn[0], separationLn[0],ISO[0],tpose[0], moyennes[0])

99 113.79591981420148 12800 64 100.55385098793752


<a id="lienAstrobin"></a> 
Plus qu'à faire l'IA qui donnera en fonction de `separationLn`, `ISO`, `tpose`, `illuminationLn` la moyenne de l'image puis une fois entrainée faire le protocle de récupération des images sur astrobin : exemple d'adresse : https://www.astrobin.com/full/392940/0/?nc=all (mis en place [ici](#algorithmeAstrobin))

IA : 

In [56]:
def next_batch(batch_size, illuminationLn, separationLn,ISO,tpose, moyennes):
    indices = np.arange(0,len(illuminationLn),1)
    np.random.shuffle(indices)
    indices = indices[0:batch_size]
    illuminationLnArray = np.reshape(np.array([illuminationLn[i] for i in indices]),(batch_size,1))
    separationLnArray = np.reshape(np.array([separationLn[i] for i in indices]),(batch_size,1))
    ISOArray = np.reshape(np.array([ISO[i] for i in indices]),(batch_size,1))
    tposeArray = np.reshape(np.array([tpose[i] for i in indices]),(batch_size,1))
    moyennesArray = np.reshape(np.array([moyennes[i] for i in indices]),(batch_size,1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [57]:
next_batch(7,illuminationLn, separationLn,ISO,tpose, moyennes)[0].shape

(7, 4)

In [60]:
def data(illuminationLn, separationLn,ISO,tpose, moyennes):
    indices = np.arange(0,len(illuminationLn),1)
    np.random.shuffle(indices)
    illuminationLnArray = np.reshape(np.array([illuminationLn[i] for i in indices]),(len(separationLn),1))
    separationLnArray = np.reshape(np.array([separationLn[i] for i in indices]),(len(separationLn),1))
    ISOArray = np.reshape(np.array([ISO[i] for i in indices]),(len(separationLn),1))
    tposeArray = np.reshape(np.array([tpose[i] for i in indices]),(len(separationLn),1))
    moyennesArray = np.reshape(np.array([moyennes[i] for i in indices]),(len(separationLn),1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [22]:
import tensorflow as tf
forceInit = False

batch_size = 7
learning_rate = 0.0002
i = 0
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,100,150,100,50,1],'Lien')
    
    loss = tf.reduce_mean(tf.square(couche-inpt))  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    train_disc = optimizer_disc.minimize(loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14:16]) > lastTrained:
              lastTrained = int(elem[14:16])
          tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,51):
#           print(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)
          inptLayer, moyenneLayer = next_batch(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)#Problèmes si gen_input
#           print(type(inpt),type(moyenne))
          _,dl = sess.run([train_disc,loss], feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_str = loss_saver_disc.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_writer.add_summary(summary_str,i)
#           save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        tf_saver.save(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
summary_writer.close()

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 100)
Taille de denseLien_3 : (7, 150)
Taille de denseLien_4 : (7, 100)
Taille de denseLien_5 : (7, 50)
Taille de denseLien_6 : (7, 1)
['checkpoint', 'checkpoint_at_35.ckpt.data-00000-of-00001', 'checkpoint_at_35.ckpt.index', 'checkpoint_at_35.ckpt.meta', 'checkpoint_at_40.ckpt.data-00000-of-00001', 'checkpoint_at_40.ckpt.index', 'checkpoint_at_40.ckpt.meta', 'checkpoint_at_45.ckpt.data-00000-of-00001', 'checkpoint_at_45.ckpt.index', 'checkpoint_at_45.ckpt.meta', 'checkpoint_at_50.ckpt.data-00000-of-00001', 'checkpoint_at_50.ckpt.index', 'checkpoint_at_50.ckpt.meta']
INFO:tensorflow:Restoring parameters from D:/Github/ImagesAnalyseVoile/IA/checkpoints_Moyenne_image_0/checkpoint_at_50.ckpt
Entrainement....
Step 0: Discriminator Loss: 61163970560.000000
Step 1: Discriminator Loss: 49650774016.000000
Step 2: Discriminator Loss: 39767715840.000000
Step 3: Discriminator Loss: 32323192832.000000
Step 4: Discriminator Loss: 26890205184

In [99]:
import tensorflow as tf
forceInit = True

batch_size = 7
learning_rate = 5*10**-3
i = 28
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,5,6,5,1],'Lien')
    
    loss = tf.reduce_mean(couche-inpt)  # This optimizes the discriminator.
    loss1 = tf.sign(loss)*loss
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss1)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    train_disc = optimizer_disc.minimize(loss1)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14:16]) > lastTrained:
              lastTrained = int(elem[14:16])
          tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        lastPrec = 1
        pente = []
        for i in range(0,301):
#           print(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)
          inptLayer, moyenneLayer = next_batch(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)#Problèmes si gen_input
#           print(type(inpt),type(moyenne))
          _,dl,l1 = sess.run([train_disc,loss,loss1], feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_str = loss_saver_disc.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_writer.add_summary(summary_str,i)
#           save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f loss1 : %f' % (i, dl,l1))
          pente.append((dl-lastPrec)/lastPrec)
#           print('Pente : %f'%(pente[-1]))
          lastPrec = dl
          if i % 5 == 0:
            tf_saver.save(sess,'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        print(np.mean(pente[1:]))
        tf_saver.save(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
summary_writer.close()

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 5)
Taille de denseLien_3 : (7, 6)
Taille de denseLien_4 : (7, 5)
Taille de denseLien_5 : (7, 1)
No previous training found...
Entrainement....
Step 0: Discriminator Loss: 15938.234375 loss1 : 15938.234375
Step 1: Discriminator Loss: 14448.819336 loss1 : 14448.819336
Step 2: Discriminator Loss: 13018.708008 loss1 : 13018.708008
Step 3: Discriminator Loss: 11637.017578 loss1 : 11637.017578
Step 4: Discriminator Loss: 10341.463867 loss1 : 10341.463867
Step 5: Discriminator Loss: 9107.377930 loss1 : 9107.377930
Step 6: Discriminator Loss: 7953.870117 loss1 : 7953.870117
Step 7: Discriminator Loss: 6879.565430 loss1 : 6879.565430
Step 8: Discriminator Loss: 5814.137207 loss1 : 5814.137207
Step 9: Discriminator Loss: 4825.145020 loss1 : 4825.145020
Step 10: Discriminator Loss: 3889.126465 loss1 : 3889.126465
Step 11: Discriminator Loss: 2992.819092 loss1 : 2992.819092
Step 12: Discriminator Loss: 2166.636963 loss1 : 2166.636963
Step 

In [106]:
def submitImageData(batch_size, illuminationLn, separationLn,ISO,tpose, moyennes):
    illuminationLnArray = np.reshape(np.array([illuminationLn for _ in range(batch_size)]),(batch_size,1))
    separationLnArray = np.reshape(np.array([separationLn for _ in range(batch_size)]),(batch_size,1))
    ISOArray = np.reshape(np.array([ISO for _ in range(batch_size)]),(batch_size,1))
    tposeArray = np.reshape(np.array([tpose for _ in range(batch_size)]),(batch_size,1))
    moyennesArray = np.reshape(np.array([moyennes for _ in range(batch_size)]),(batch_size,1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [111]:
import tensorflow as tf
forceInit = True

batch_size = 7
learning_rate = 5*10**-3
i = 28
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,5,6,5,1],'Lien')
    
    loss = tf.reduce_mean(couche-inpt)  # This optimizes the discriminator.
    loss1 = tf.sign(loss)*loss
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss1)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    train_disc = optimizer_disc.minimize(loss1)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
        inptLayer, moyenneLayer = submitImageData(batch_size,illuminationLn[0], separationLn[0],ISO[0],tpose[0],moyennes[0])
        moyenneResult = couche.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
        print(moyenneResult)
        print(moyenneLayer)

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 5)
Taille de denseLien_3 : (7, 6)
Taille de denseLien_4 : (7, 5)
Taille de denseLien_5 : (7, 1)
INFO:tensorflow:Restoring parameters from D:/Github/ImagesAnalyseVoile/IA/model/modelMoyenne_image_28.ckpt
[[3278.1338]
 [3278.1338]
 [3278.1338]
 [3278.1338]
 [3278.134 ]
 [3278.134 ]
 [3278.134 ]]
[[100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]]


Problème : l'image est sur 8 bit donc une valeur moyenne à 3278 est absurde et va tout saturer... -> On change la fonction d'activation si on dépasse 255

In [169]:
def next_batch(batch_size, illuminationLn, separationLn,ISO,tpose, moyennes):
    indices = np.arange(0,len(illuminationLn),1)
    np.random.shuffle(indices)
    indices = indices[0:batch_size]
    illuminationLnArray = np.reshape(np.array([illuminationLn[i] for i in indices]),(batch_size,1))
    separationLnArray = np.reshape(np.array([separationLn[i] for i in indices]),(batch_size,1))
    ISOArray = np.reshape(np.array([ISO[i] for i in indices]),(batch_size,1))
    tposeArray = np.reshape(np.array([tpose[i] for i in indices]),(batch_size,1))
    moyennesArray = np.reshape(np.array([moyennes[i] for i in indices]),(batch_size,1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [140]:
next_batch(7,illuminationLn, separationLn,ISO,tpose, moyennes)[0].shape

(7, 4)

In [138]:
def data(illuminationLn, separationLn,ISO,tpose, moyennes):
    indices = np.arange(0,len(illuminationLn),1)
    np.random.shuffle(indices)
    illuminationLnArray = np.reshape(np.array([illuminationLn[i] for i in indices]),(len(separationLn),1))
    separationLnArray = np.reshape(np.array([separationLn[i] for i in indices]),(len(separationLn),1))
    ISOArray = np.reshape(np.array([ISO[i] for i in indices]),(len(separationLn),1))
    tposeArray = np.reshape(np.array([tpose[i] for i in indices]),(len(separationLn),1))
    moyennesArray = np.reshape(np.array([moyennes[i] for i in indices]),(len(separationLn),1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [125]:
import tensorflow as tf
forceInit = False

batch_size = 7
learning_rate = 0.0002
i = 0
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,100,150,100,50,1],'Lien')
    
    loss = tf.reduce_mean(tf.square(couche-inpt))  # This optimizes the discriminator.
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    train_disc = optimizer_disc.minimize(loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14:16]) > lastTrained:
              lastTrained = int(elem[14:16])
          tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,51):
#           print(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)
          inptLayer, moyenneLayer = next_batch(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)#Problèmes si gen_input
#           print(type(inpt),type(moyenne))
          _,dl = sess.run([train_disc,loss], feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_str = loss_saver_disc.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_writer.add_summary(summary_str,i)
#           save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          if i % 5 == 0:
            tf_saver.save(sess,'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        tf_saver.save(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
summary_writer.close()

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 100)
Taille de denseLien_3 : (7, 150)
Taille de denseLien_4 : (7, 100)
Taille de denseLien_5 : (7, 50)
Taille de denseLien_6 : (7, 1)
['checkpoint', 'checkpoint_at_35.ckpt.data-00000-of-00001', 'checkpoint_at_35.ckpt.index', 'checkpoint_at_35.ckpt.meta', 'checkpoint_at_40.ckpt.data-00000-of-00001', 'checkpoint_at_40.ckpt.index', 'checkpoint_at_40.ckpt.meta', 'checkpoint_at_45.ckpt.data-00000-of-00001', 'checkpoint_at_45.ckpt.index', 'checkpoint_at_45.ckpt.meta', 'checkpoint_at_50.ckpt.data-00000-of-00001', 'checkpoint_at_50.ckpt.index', 'checkpoint_at_50.ckpt.meta']
INFO:tensorflow:Restoring parameters from D:/Github/ImagesAnalyseVoile/IA/checkpoints_Moyenne_image_0/checkpoint_at_50.ckpt
Entrainement....
Step 0: Discriminator Loss: 38452.437500
Step 1: Discriminator Loss: 38248.878906
Step 2: Discriminator Loss: 36917.433594
Step 3: Discriminator Loss: 38842.480469
Step 4: Discriminator Loss: 27158.371094
Step 5: Discriminator 

In [172]:
import tensorflow as tf
forceInit = False

batch_size = 7
learning_rate = 5*10**-4
i = 33
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,1],'Lien')
    
    loss = tf.reduce_max(couche-inpt)  # This optimizes the discriminator.
    loss1 = tf.sign(loss)*loss
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss1)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    train_disc = optimizer_disc.minimize(loss1)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14:17]) > lastTrained:
              lastTrained = int(elem[14:17])
          tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        lastPrec = 1
        pente = []
        for i in range(0,301):
#           print(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)
          inptLayer, moyenneLayer = next_batch(batch_size,illuminationLn, separationLn,ISO,tpose, moyennes)#Problèmes si gen_input
#           print(type(inpt),type(moyenne))
          _,dl,l1 = sess.run([train_disc,loss,loss1], feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_str = loss_saver_disc.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
          summary_writer.add_summary(summary_str,i)
#           save(disc_saver,gen_inpt,disc_inpt)
          print('Step %i: Discriminator Loss: %f loss1 : %f' % (i, dl,l1))
          pente.append((dl-lastPrec)/lastPrec)
#           print('Pente : %f'%(pente[-1]))
          lastPrec = dl
          if i % 5 == 0:
            tf_saver.save(sess,'D:/Github/ImagesAnalyseVoile/IA/checkpoints_'+nom+'/checkpoint_at_'+str(i)+'.ckpt')
        print(np.mean(pente[1:]))
        tf_saver.save(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
summary_writer.close()

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 1)
No previous training found...
Entrainement....
Step 0: Discriminator Loss: 377.842285 loss1 : 377.842285
Step 1: Discriminator Loss: 354.464355 loss1 : 354.464355
Step 2: Discriminator Loss: 332.076630 loss1 : 332.076630
Step 3: Discriminator Loss: 307.846405 loss1 : 307.846405
Step 4: Discriminator Loss: 285.816345 loss1 : 285.816345
Step 5: Discriminator Loss: 261.933533 loss1 : 261.933533
Step 6: Discriminator Loss: 240.051208 loss1 : 240.051208
Step 7: Discriminator Loss: 214.381012 loss1 : 214.381012
Step 8: Discriminator Loss: 194.285553 loss1 : 194.285553
Step 9: Discriminator Loss: 170.101456 loss1 : 170.101456
Step 10: Discriminator Loss: 148.519165 loss1 : 148.519165
Step 11: Discriminator Loss: 122.593872 loss1 : 122.593872
Step 12: Discriminator Loss: 103.256699 loss1 : 103.256699
Step 13: Discriminator Loss: 80.375580 loss1 : 80.375580
Step 14: Discriminator Loss: 57.495438 loss1 : 57.495438
Step 15: Discriminat

In [106]:
def submitImageData(batch_size, illuminationLn, separationLn,ISO,tpose, moyennes):
    illuminationLnArray = np.reshape(np.array([illuminationLn for _ in range(batch_size)]),(batch_size,1))
    separationLnArray = np.reshape(np.array([separationLn for _ in range(batch_size)]),(batch_size,1))
    ISOArray = np.reshape(np.array([ISO for _ in range(batch_size)]),(batch_size,1))
    tposeArray = np.reshape(np.array([tpose for _ in range(batch_size)]),(batch_size,1))
    moyennesArray = np.reshape(np.array([moyennes for _ in range(batch_size)]),(batch_size,1))
    inpt = np.concatenate((illuminationLnArray,separationLnArray,ISOArray,tposeArray),axis=1)
    return np.array(inpt,dtype=np.float32),np.array(moyennesArray,dtype=np.float32)

In [173]:
import tensorflow as tf
forceInit = True

batch_size = 7
learning_rate = 5*10**-2
i = 33
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      return couche,kernel,bias,kernel_saver,bias_saver

    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
        return couche,kernelBiasList,saver
    
    inpt = tf.placeholder(tf.float32,shape=[batch_size,4],name='inpt')
    moyenne = tf.placeholder(tf.float32,shape=[batch_size,1],name='moyenne')
    
    import numpy as np
    couche,kernelBiasList,saver = discriminatorDense(inpt,[4,1],'Lien')
    
    loss = tf.reduce_max(couche-inpt)  # This optimizes the discriminator.
    diff = couche-inpt
    loss1 = tf.sign(loss)*loss
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",loss1)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    train_disc = optimizer_disc.minimize(loss1)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        tf_saver.restore(sess, 'D:/Github/ImagesAnalyseVoile/IA/model/model'+nom+'.ckpt')
        inptLayer, moyenneLayer = submitImageData(batch_size,illuminationLn[0], separationLn[0],ISO[0],tpose[0],moyennes[0])
        moyenneResult = couche.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
        difference = diff.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
        lossEval = loss.eval(feed_dict={inpt: inptLayer,moyenne: moyenneLayer})
        print(moyenneResult)
        print(moyenneLayer)
        print(difference)
        print(lossEval)

Taille de denseLien_1 : (7, 4)
Taille de denseLien_2 : (7, 1)
INFO:tensorflow:Restoring parameters from D:/Github/ImagesAnalyseVoile/IA/model/modelMoyenne_image_33.ckpt
[[62.994953]
 [62.994953]
 [62.994953]
 [62.994953]
 [62.994953]
 [62.994953]
 [62.994953]]
[[100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]
 [100.55385]]
[[-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]
 [-3.6005047e+01 -5.0800968e+01 -1.2737005e+04 -1.0050468e+00]]
-1.0050468


<b>Conclusion : il faut faire attention à quelle fonction de coût on utilise et particulièrement au signe du gradient qu'il est parfois nécessaire de modifier ainsi que la fonction reduce_sum pas toujours adaptée. De plus le pas d'apprentissage est un facteur très important pour modifier le comportement du modèle</b>

In [175]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl
image = cv2.imread('D:/Google drive/TIPE/Images_source/clean/1.jpg',1)
print(fichiersComplets[0]+' a une moyenne réelle de : %f'% moyennes[0])
noiseImage = cv2.imread('D:/Google drive/TIPE/Images_source/noise/1.jpg',1)
row,col,ch = image.shape
image = cv2.resize(image,(col//5,row//5))
row,col,ch = image.shape
gauss = np.array(np.random.normal(63,1,(row,col,ch)),dtype=np.int)
gauss1 = np.array(np.random.normal(np.mean(noiseImage),1,(row,col,ch)),dtype=np.int)
print(gauss.shape)
print(noisy.shape)
print(type(image)==type(noisy))
# print(noisy[20:50,20:50,0])
# cv2.destroyAllWindows()
plt.figure(4)
plt.title('Clean')
plt.imshow(image)
plt.figure(3)
plt.title('Créée avec vraie moyenne')
custom1 = image//np.random.randint(3,7)+gauss1
plt.imshow(custom1)
print(np.max(custom1))
plt.figure(2)
plt.title('Créée avec moyenne estimée')
custom2 = image//np.random.randint(3,7)+gauss
print(np.max(custom2))
plt.imshow(custom2)
plt.figure(1)
plt.title('Noise')
plt.imshow(noiseImage)
plt.show()

D:/Github/ImagesAnalyseVoile/01_M51.jpg a une moyenne réelle de : 100.553851
(433, 770, 3)
(732, 1100, 3)
True


FigureCanvasNbAgg()

FigureCanvasNbAgg()

127


FigureCanvasNbAgg()

139


FigureCanvasNbAgg()

Essai non abouti avec Keras : à terme très utile mais nécessite un apprentissage complet -> pas le temps

In [71]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers
batch_size = 7
learning_rate = 10**-9
i = 13
nom = 'Moyenne_image_'+str(i)

tf.reset_default_graph()
import traceback

root_logdir = "D:/Github/ImagesAnalyseVoile/IA/tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)

import numpy as np
with tf.Graph().as_default():
    model = Sequential([
    layers.Dense(4, activation=tf.nn.relu),
    layers.Dense(10, activation=tf.nn.relu),
    layers.Dense(50, activation=tf.nn.relu),
    layers.Dense(10, activation=tf.nn.relu),
    layers.Dense(1, activation=tf.nn.relu),
    ])
    model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy'])
    inpt,moyenne = data(illuminationLn, separationLn,ISO,tpose, moyennes)
    model.fit(inpt, moyenne, epochs=500)
    model.summary()

Epoch 1/500


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("sequential_1_input:0", shape=(?, 4), dtype=float32) is not an element of this graph.

<a id="algorithmeAstrobin"></a> Maintenant il faut récupérer les images et les métadonnées associées comme dit [ici](#lienAstrobin)

Essai de récupération d'image

In [20]:
import urllib.request
import re
import time
from selenium import webdriver

url = 'https://www.astrobin.com/full/392940/0/?nc=all'
def retrieveImg(url):
    req = urllib.request.Request(url)
    resp = urllib.request.urlopen(req)
    respData = resp.read().decode("utf-8")
    Limg = []
 
    driver = webdriver.Chrome("D:/Github/chromedriver_win32/chromedriver.exe")

    driver.get(url)

    images = driver.find_elements_by_tag_name('img')
    for image in images:
        print(image.get_attribute('src'))
        Limg.append(image.get_attribute('src'))

    driver.close()
    print(Limg)
    if len(Limg) == 0:
        print('No images found')
        file = urllib.request.urlopen(url)
        page = file.read().decode("utf-8")
        file.close()
        f = open('C:/Users/Robin/Desktop/PB'+url[-6:-5]+'.html', 'w')
        f.write(page)
        f.close
#         raise ValueError('Error')
        return None
    LimgSelect = []
    for img in Limg:
        if img[-3:] == 'jpg':
            LimgSelect.append(img)
    imgPath = 'D:/Github/ImagesAnalyseVoile/'
    if len(LimgSelect) == 0:
        return None
    choixIndex = 1 if len(LimgSelect)>1 else 0
    imgPath += LimgSelect[choixIndex].split('/')[-1]
    class AppURLopener(urllib.request.FancyURLopener):
        version = "Mozilla/5.0"

    opener = AppURLopener()
    f = open(imgPath,'wb')
    f.write(opener.open(LimgSelect[choixIndex]).read())
    f.close()
    # print(type(opener.open(Limg[0]).read()))
    return imgPath
retrieveImg(url)

https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png
https://cdn.astrobin.com/thumbs/Xo22AB02MYyF_1824x0_fpu6-biL.jpg
['https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png', 'https://cdn.astrobin.com/thumbs/Xo22AB02MYyF_1824x0_fpu6-biL.jpg']


C:\Users\Robin\AppData\Roaming\Python\Python36\site-packages\ipykernel\__main__.py:46: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


'D:/Github/ImagesAnalyseVoile/Xo22AB02MYyF_1824x0_fpu6-biL.jpg'

Déjà on reconstruit l'url de l'image avec les infos

In [12]:
url = "https://www.astrobin.com/full/392940/0/?nc=all"
L = url.split('/')
del L[3]
del L[-1]
urlInfos = '/'.join(L) + "/"
print(urlInfos == "https://www.astrobin.com/392940/0/")
# https://www.astrobin.com/392940/0/
def reconstructURLInfo(url):
    L = url.split('/')
    del L[3]
    del L[-1]
    urlInfos = '/'.join(L) + "/"
    return urlInfos

True


Ensuite on doit sélectionner les images comportant des information sur les iso et le temps de pose unitaire

In [22]:
def getRaDecDateISOtPose(url):
    req = urllib.request.Request(url)
    resp = urllib.request.urlopen(req)
    respData = resp.read().decode("utf-8")
#     print(respData)
    def searchAfter(webPageStr,listeChaine_LimiteFin):
        i = 0
        L = [[] for i in range(len(listeChaine_LimiteFin))]
        while i<len(webPageStr)-max([len(listeChaine_LimiteFin[i][0]) for i in range(len(listeChaine_LimiteFin))]):
            for index,chaine_limiteFin in enumerate(listeChaine_LimiteFin):
                chaine,limiteFin = chaine_limiteFin[0],chaine_limiteFin[1]
                if webPageStr[i:i+len(chaine)] == chaine:
                    i = i+len(chaine)
                    txt = ''
                    while webPageStr[i:i+len(limiteFin)] != limiteFin and i<len(webPageStr)-len(limiteFin):
                        txt += str(webPageStr[i])
                        i += 1
                    L[index].append(txt)
            i += 1
        return L
    Lfiltres = [["RA center:</strong>","degrees"],["DEC center:</strong>","degrees"],["start_date=","&amp"],["Frames:</strong>","</p>"]]
    Linfos = searchAfter(respData,Lfiltres)
#     print(Linfos)
    for i in range(len(Linfos)):
        if len(Linfos[i]) == 0:
            print('Manque trop de données')
            return None
    part1 = [float(Linfos[0][i]) for i in range(len(Linfos[0]))]
    part2 = [float(Linfos[1][i]) for i in range(len(Linfos[1]))]
    Linfos = part1+part2+Linfos[2:]
    i = 0
    iso,tpose = [],[]
    while i<len(Linfos[-1][0]):
        if Linfos[-1][0][i:i+3] == 'ISO':
            a = i+3
            isoChaine = ''
            while (Linfos[-1][0][a:a+3] != '<br' and Linfos[-1][0][a:a+2] != '\n') and a<len(Linfos[-1][0]):
                isoChaine += Linfos[-1][0][a]
                a += 1
            iso.append(isoChaine)
            a = i-1
            tposeChaine = ''
            while Linfos[-1][0][a] != ':' and a>0:
                tposeChaine += Linfos[-1][0][a]
                a -= 1
            tpose.append(tposeChaine[::-1])
            i += 1
        else: 
            i += 1
    tpose = [tpose[index].strip().split('x') for index in range(len(tpose))]
    tpose = [[tpose[index][0]]+[tpose[index][1][:-1]] for index in range(len(tpose))]
    try:
        Linfos = [part1]+[part2]+[Linfos[2]]+[[float(iso[i]) for i in range(len(iso))]]+[tpose]
    except:
        return None
    for i in range(2,len(Linfos)):
        if len(Linfos[i]) == 0:
            print('Manque données annexes (ISO ~)')
            return None
    return Linfos
# var = getRaDecDateISOtPose("https://www.astrobin.com/393134/?nc=all")
# print(var)

Attention à la mutabilité !

In [400]:
a = [[] for i in range(4)]
print(a)
a[0].append(2)
a[0].append(2)
a[0][0] = 3
print(a)

[[], [], [], []]
[[3, 2], [], [], []]


D'où au total :

In [21]:
derniereAdresse = "https://www.astrobin.com/full/393040/0/?nc=all"
derniereAdresseSplit = derniereAdresse.split('/')
i = int(derniereAdresseSplit[-3])
nb = 100
nbParpaquet = 10
Info = []
while i > i-nb:
#     print(derniereAdresseSplit[-3])
#     print(i)
    derniereAdresseSplit[-3] = str(i)
    adresse = '/'.join(derniereAdresseSplit)
    print(reconstructURLInfo(adresse))
    try:
        Linfos = getRaDecDateISOtPose(reconstructURLInfo(adresse))
        if Linfos != None:
            print('ok assez de données')
            for essai in range(1):
                img = retrieveImg(adresse)
                if img != None:
                    print('Ok : téléchargée')
                    Info.append(Linfos+[img])
                    break
                else:
                    print("Erreur peut-être de format ou autre : voir balise de l'image à cette adresse : "+adresse)
    except urllib.error.HTTPError:
        print("Le site n'a pas donné suite à la demande.....")
    i -= 1
print(Info)

https://www.astrobin.com/393040/0/
ok assez de données
https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png
https://cdn.astrobin.com/thumbs/Ei7W0Dem_YUO_1824x0_wmhqkGbg.jpg
['https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png', 'https://cdn.astrobin.com/thumbs/Ei7W0Dem_YUO_1824x0_wmhqkGbg.jpg']


C:\Users\Robin\AppData\Roaming\Python\Python36\site-packages\ipykernel\__main__.py:46: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


Ok : téléchargée
https://www.astrobin.com/393039/0/
Manque données annexes (ISO ~)
https://www.astrobin.com/393038/0/
Manque données annexes (ISO ~)
https://www.astrobin.com/393037/0/
ok assez de données
https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png
https://cdn.astrobin.com/thumbs/bqQq1gKgBYeG_1824x0_wmhqkGbg.jpg
['https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png', 'https://cdn.astrobin.com/thumbs/bqQq1gKgBYeG_1824x0_wmhqkGbg.jpg']
Ok : téléchargée
https://www.astrobin.com/393036/0/
ok assez de données
https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png
https://cdn.astrobin.com/thumbs/O2j4H1KClsIz_1824x0_wmhqkGbg.jpg
['https://cdn.astrobin.com/static/astrobin/images/astrobin-logo-small.81f9c70ff8b4.png', 'https://cdn.astrobin.com/thumbs/O2j4H1KClsIz_1824x0_wmhqkGbg.jpg']
Ok : téléchargée
https://www.astrobin.com/393035/0/
Manque données annexes (ISO ~)
https://www.astrobin.com/39

ValueError: could not convert string to float: '133 (gain: 19.00) -25C bin 1x1\n                                    \n                                '

# Résumé démarche

Evolution démarche
Autoencodeur sur image : essayer de comprendre l'influence des couches de convolutions. Une IA ayant normalement besoin de beaucoup d'exemples on découpe chacune des 6 images en sous-images de 400 par 400 pixel (toujours en RGB)
Comme cela ne fonctionnait pas, utilisation de couches intégralement connectée : normalement censées être plus précises. Influence du nombre de couches
Retour sur les couches de convolution : tests plus formels en faisant varier la taille des noyaux pour voir si cela change la précision finale du modèle.
Pas de résultats.
Après recherches, aperçu que ce sont les couches de pooling qui semblent pouvoir synthétiser l'information.
Le système reste toujours imprécis. En effet, la seule solution trouvée pour pouvoir comparée l'image de dimension réduite à l'image originale était d'augmenter à nouveau l'image avec une fonction d'interpolation préconçue en ajoutant au  milieu des couches de convolution pour essayer de reprendre la même démarche qu'à la réduction de dimension : ajouter une étape avant de modifier la taille de l'image qui l'analyse.
Comme cela ne fonctionne pas, ajout de couches denses pour améliorer la précision. Cela ne change rien.
Comme j'ai senti que les couches de convolutions restaient un point obscur dans l'ia : essais sur le nombre de couches qui donnent pas de résultat réellement marqué : les courbes de précision comme précédemment sont très instables sans changement. En observant mon modèle : choix de fixer le nombre de filtre à 3 pour les 3 couches RGB. N'en faudrait-il pas plus ? Essai avec plus de filtres. Pas de changement probant. De même en essayant avec des couches denses. Peut-être cela provient du lot d'images pas assez diversifié pour trouver une tendance générale dans les photos. De plus les réglages ont imposer un recadrage pour certaines images et pas d'autres ce qui amène sur quelques images où une étoile fera une taille moyenne de par exemple 40 par 40 pixel et sur d'autres 20 pixels. Pour avoir une IA pouvant traiter une image en s'affranchissant de recadrages / type d'appareil télescope utilisé, on choisit alors de modifier aléatoirement l'image d'entrée pour entrainer l'ia en effectant : rotation, redimensionnement et recadrage aléatoire de l'image source de l'APN. "Après avoir visionné cette vidéo Convolutional Neural Networks (CNNs) explained il apparait que le redimensionnement peut poser des problèmes par rapport à l'apprentissage : en effet, le cercle de diamètre différent de chaque étoile variera suivant la taille de la région de l'image considérée, d'autant plus si on la redimensionne aléatoirement (NB : quelques images de départ ont déjà été redimensionnées pour obtenir des zones exploitables). 
Cela permettra néanmoins d'avoir un réseau plus polyvalent et moins sensible aux artéfacts de retouche manuelle. Il serait alors nécessaire d'afficher les caractéritiques apprises par chaque couche pour déterminer ce qui est détecté. 
De plus, à la suite de cette vidéo, il apparait que les filtres correspondent aux nombre de formes détectables par la couche de convolution." De plus, "Avant de lancer la réalisation de ce modèle, d'après les articles GANs from Scratch 1: A deep introduction et Generative Adversarial Networks using Tensorflow, il apparait que le type de réseau de neurone que je cherche à réaliser s'appelle un Réseaux antagonistes génératifs."
De cette piste : réalisation de plusieurs modèles mettant en place le principe des GAN qui lie plusieurs encodeurs-decodeurs pour reproduire une image en entrée, en créer une autre. Cependant, ce réseau utilisant une fonction log comme fonction de coût, il sature très vite et donne des résultats très extrèmes. Néanmoins, après visionnage vidéo : meilleure compréhension du fonctionnement : ce réseau est constitué de paires encodeur-decodeur le premier résumant ce qu'il perçoit des tendances des images et le deuxième recréant une image à partir de l'encodage de celle-ci tout cela en tentant de suivre la règle qui impose de minimiser une fonction de coût (image de la précision).
A cette occasion, et par une autre vidéo mise en place de la normalisation par lot qui permet normalement de réduire la 'saturation' de l'image : autrement dit cette couche du modèle permet de garder les données travaillée dans la plage moyenne de travail. 
Cela ne donne toujours rien. Les couches de convolution de convolution sont un peu mieux comprises : elles possèdent deux caractéristiques importantes : une taille de noyau et un nombre de filtres. La taille du noyau détermine quelle taille de zone doit être résumé en une seule valeur. Puis, le nombre de filtre détermine grossièrement le nombre de motifs, groupe de couleurs, ... que la couche peut détecter. Après entrevue avec Mr Quiblier, les couches de convolution utilise bien un produit de convolution pour synthétiser l'information. A ce moment on comprend mieux comment plusieurs couches avec des petits noyaux peuvent petit à petit regrouper des zones similaires et avoir le même rôle qu'une seule couche avec un gros noyau de convolution. D'où une démultiplication des couches de convolution en sous-couches avant pooling (qui réduit la taille et finit l'étape de synthèse). La fonction de coût reste un point obscur. Comme le GAN est un ensemble d'autoencodeurs liés à différents points, on revient sur le fonctionnement en autoencodeur. Le but serai de faire tout d'abord 2 autoencodeurs reproduisant respectivement l'image bruitée avec l'image bruitée en entrée et l'image nettoyée avec l'image nettoyée. A cette étape rien ne converge. Se pose toujours la question de la fonction de coût (moyenne de la différence) et du nombre d'image insuffisant. Mise en place d'un algorithme de bruitage d'image téléchargées automatiquement depuis internet : on a trouvé qu'on peut approximer l'image bruitée à partir de l'image nettoyée en effectant image//np.random.randint(3,7)+gauss avec une gaussienne centrée sur la moyenne de l'image bruitée qu'il est donc nécessaire d'estimée. Pour cela on entraine une ia pour trouver ce paramètre en fonction de la sensibilité, du temps de pose, de l'éloignement de l'objet photographié à la Lune, de la phase de la Lune. Puis on met en place l'algorithme allant sur Astrobin, sélectionnant les images ayant toutes les informations pour prédire la valeur moyenne de l'image avant traitement et on télécharge.

Parallèlement à cela, le dernier point qu'il me reste à éclaircir est la fonction de coût. On repart sur les exemples d'autoencodeurs purs (GAN 15_N et 15_C pour avoir une idée pour l'image bruitée et débruitée : en effet, sur la photo une fonction de coût serait peut-être plus adaptée pour influencer les réglages dans le bon sens.)

En terme de fonction de coût plusieurs choses peuvent être envisagées : 
<ul>
    <li>En gardant $x=sortie-entree$</li>
    <li>Choisir entre <a href = 'https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean'>tf.reduce_mean</a>, <a href = 'https://www.tensorflow.org/api_docs/python/tf/math/reduce_max'>tf.reduce_max</a>,  <a href = 'https://www.tensorflow.org/api_docs/python/tf/math/reduce_logsumexp'>tf.reduce_logsumexp</a>,  <a href = 'https://www.tensorflow.org/api_docs/python/tf/math/reduce_std'>tf.reduce_std</a>,  <a href = 'https://www.tensorflow.org/api_docs/python/tf/math/reduce_variance'>tf.reduce_variance</a> <a href='https://www.tensorflow.org/api_docs/python/tf/math'>(index des fonctions utilisables)</a></li>
    <li>Mettre x au carré ou multiplier le résultat de la fonction par un coefficient supérieur à 1 pour augmenter l'agressivité des correction </li>
    <li>Prendre la valeur absolue ou multiplier par le signe du résultat pour observer l'influence sur la convergens</li>
    <li>Modifier le taux d'apprentissage : 10**-6,10**-4,10**-2,10**-1</li>
</ul>

D'où les essais suivants : </br>
On  fait un `enregistrement automatique` toutes les `4 étapes` écrasant les précédentes sauvegarde et mettant `tout au fur et à mesure dans le dossier` des log et modèles</br>
On arrête si diverge complètement ou est un instable sans amélioration avec l'entrainement</br>
On `modifie le pas d'apprentissage`</br>
De base l'entrainement se fait sur `50 époques` ce qui est suffisant pour se rendre compte (d'après les expériences précédentes) si le modèle va bien converger ou pas.</br>
On `affiche régulièrement la valeur minimale, maximale et moyenne du tenseur`

| Réalisé | Fonction de coût | Photo bruitée | Résultat | Photo traitée | Résultat |
| --- | --- | --- | --- | --- | --- |
| -- | tf.reduce_mean(sortie-entree) | Autoencodeur_N_mean_1_10e-6 | - |Autoencodeur_C_mean_1_10e-6| - |
| -- | tf.reduce_mean(sortie-entree) | Autoencodeur_N_mean_1_10e-4 | - |Autoencodeur_C_mean_1_10e-4| - |
| -- | tf.reduce_mean(sortie-entree) | Autoencodeur_N_mean_1_10e-2 | - |Autoencodeur_C_mean_1_10e-2| - |
| -- | tf.reduce_mean(sortie-entree) | Autoencodeur_N_mean_1_10e-1 | - |Autoencodeur_C_mean_1_10e-1| - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.sign() | Autoencodeur_N_mean_2_10e-6 | - | Autoencodeur_C_mean_2_10e-6 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.sign() | Autoencodeur_N_mean_2_10e-4 | - | Autoencodeur_C_mean_2_10e-4 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.sign() | Autoencodeur_N_mean_2_10e-2 | - | Autoencodeur_C_mean_2_10e-2 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.sign() | Autoencodeur_N_mean_2_10e-1 | - | Autoencodeur_C_mean_2_10e-1 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.abs() | Autoencodeur_N_mean_3_10e-6 | - | Autoencodeur_C_mean_3_10e-6 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.abs() | Autoencodeur_N_mean_3_10e-4 | - | Autoencodeur_C_mean_3_10e-4 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.abs() | Autoencodeur_N_mean_3_10e-2 | - | Autoencodeur_C_mean_3_10e-2 | - |
| -- | tf.reduce_mean(sortie-entree) et </br> tf.abs() | Autoencodeur_N_mean_3_10e-1 | - | Autoencodeur_C_mean_3_10e-1 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_mean() | Autoencodeur_N_mean_4_10e-6 | - | Autoencodeur_C_mean_4_10e-6 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_mean() | Autoencodeur_N_mean_4_10e-4 | - | Autoencodeur_C_mean_4_10e-4 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_mean() | Autoencodeur_N_mean_4_10e-2 | - | Autoencodeur_C_mean_4_10e-2 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_mean() | Autoencodeur_N_mean_4_10e-1 | - | Autoencodeur_C_mean_4_10e-1 | - |
| -- | tf.reduce_max(sortie-entree) | Autoencodeur_N_max_1_10e-6 | - |Autoencodeur_C_max_1_10e-6 | - |
| -- | tf.reduce_max(sortie-entree) | Autoencodeur_N_max_1_10e-4 | - |Autoencodeur_C_max_1_10e-4 | - |
| -- | tf.reduce_max(sortie-entree) | Autoencodeur_N_max_1_10e-2 | - |Autoencodeur_C_max_1_10e-2 | - |
| -- | tf.reduce_max(sortie-entree) | Autoencodeur_N_max_1_10e-1 | - |Autoencodeur_C_max_1_10e-1 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_max() | Autoencodeur_N_max_2_10e-6 | - |Autoencodeur_C_max_2_10e-6 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_max() | Autoencodeur_N_max_2_10e-4 | - |Autoencodeur_C_max_2_10e-4 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_max() | Autoencodeur_N_max_2_10e-2 | - |Autoencodeur_C_max_2_10e-2 | - |
| -- | tf.pow(sortie-entree,2) et </br> tf.reduce_max() | Autoencodeur_N_max_2_10e-1 | - |Autoencodeur_C_max_2_10e-1 | - |

Code de base :

Noise

In [ ]:
height = 199
width = 199
channels = 3
forceInit = False

batch_size = 7
learning_rate_power = 6
learning_rate = 10**-learning_rate_power
i = 1
nom = 'Autodencodeur_N_'+'mean'+'_'+str(i)+'_10e-'+str(learning_rate_power)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(gen_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver,sortiesImages = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    difference = disc_faux-gen_input
    disc_loss = tf.reduce_mean(difference)
    
    maximumGradient = tf.reduce_max(difference)
    minimumGradient = tf.reduce_min(difference)
    meanGradient = tf.reduce_mean(difference)
    maximumSortie = tf.reduce_max(disc_faux)
    minimumSortie = tf.reduce_min(disc_faux)
    meanSortie = tf.reduce_mean(disc_faux)
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,51):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
            
          save(disc_saver,gen_inpt,disc_inpt)
        
          maximumGradientValue = maximumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumGradientValue = minimumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanGradientValue = meanGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          maximumSortieValue = maximumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumSortieValue = minimumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanSortieValue = meanSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
        
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          print('Différence : max %f, min %f, mean %f' % (maximumGradientValue, minimumGradientValue,meanGradientValue))
          print('Sortie : max %f, min %f, mean %f' % (maximumSortieValue, minimumSortieValue,meanSortieValue))
          if i % 4 == 0:
            os.remove('model/model'+nom+'.ckpt')
            tf_saver.save(sess,'model/model'+nom+'.ckpt')
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        os.remove('model/model'+nom+'.ckpt')
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')

Clean

In [ ]:
height = 199
width = 199
channels = 3
forceInit = False

batch_size = 7
learning_rate_power = 6
learning_rate = 10**-learning_rate_power
i = 1
nom = 'Autodencodeur_N_'+'mean'+'_'+str(i)+'_10e-'+str(learning_rate_power)

tf.reset_default_graph()
import traceback

root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,nom)
with tf.Graph().as_default():
  
    def denseLayer(input,nbNeurones,nom,start=False,end=False):
      couche = tf.layers.dense(input,nbNeurones,name=nom)
      print('Taille de '+nom+' : '+str(couche.get_shape()))
      kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
      bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
      kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
      bias_saver = tf.summary.histogram("Bias/"+nom,bias)
      sortieImage = tf.summary.image("output/"+nom if end==True else "input/"+nom,tf.cast(couche,dtype=tf.uint8)) if nbNeurones == 3 else None
      return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
    
    def lrelu(x, leak=0.2): 
      return tf.maximum(x, leak * x)
    
    def convLayer(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start=False,end=False,generator=False):
      def conv(inpt,kernel,num_filters,poolingSize,poolingType,nom,start=False,end=False,pooling=True):
        couche = tf.layers.conv2d(inpt, filters=num_filters, kernel_size=kernel,
             strides=1, padding='SAME',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        if pooling == True:
          if poolingType == 'mean':
            couche = tf.layers.average_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
          else:
            couche = tf.layers.max_pooling2d(lrelu(couche), pool_size=(poolingSize,poolingSize), strides=(2,2), padding='same')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(kernelListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(inpt,kernelListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',start,end,pooling=False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(kernelListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = conv(couche,kernelListOrNot[i],num_filters,poolingSize,poolingType,nom+'_split_'+str(i),False,False if i != len(kernelListOrNot)-1 else end,pooling=False if i != len(kernelListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return conv(inpt,kernelListOrNot,num_filters,poolingSize,poolingType,nom,start,end)
      
    def deconvLayer(inpt,sizeListOrNot,num_filters,nom,start=False,end=False):
      """Ceci est une documentation"""
      def deconv(inpt,size,num_filters,nom,start=False,end=False):
        def calculateParameters(precDimension,outputSize, strides): # Ref : https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#           print(precDimension)
          return outputSize-(precDimension-1)*strides
        kernelSize = calculateParameters(inpt.get_shape()[1],size,1)
        print('Taille du noyau de deconvolution de '+nom+' : ' +str(kernelSize))
        couche = tf.layers.conv2d_transpose(inpt, filters=num_filters, kernel_size=(kernelSize,kernelSize),
             strides=1, padding='VALID',
             activation=tf.nn.relu, name=nom)
        print('Taille de '+nom+' : '+str(couche.get_shape()))
        kernel = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/kernel')[0]
        bias = tf.get_collection(tf.GraphKeys.VARIABLES,  nom+'/bias')[0]
        kernel_saver = tf.summary.histogram("Kernel/"+nom,kernel)
        bias_saver = tf.summary.histogram("Bias/"+nom,bias)
        couche = tf.contrib.layers.batch_norm(couche, is_training=training, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope=nom+'_normalization')
        sortieImage = None
        if start==True:
          sortieImage = tf.summary.image("input/"+nom,tf.cast(couche,dtype=tf.uint8))
        else:
          sortieImage = tf.summary.image("output/"+nom,tf.cast(couche,dtype=tf.uint8))
        return couche,kernel,bias,kernel_saver,bias_saver,sortieImage
      if type(sizeListOrNot) == list:
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(inpt,sizeListOrNot[0],num_filters,poolingSize,poolingType,nom+'_split_0',True,False)
        kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList = [kernel],[bias],[kernel_saver],[bias_saver],[sortieImage]
        for i in range(1,len(sizeListOrNot)):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconv(couche,sizeListOrNot[i],num_filters,nom+'_split_'+str(i),False,False if i != len(sizeListOrNot)-1 else True)
          kernelList.append(kernel)
          biasList.append(bias)
          kernel_saverList.append(kernel_saver)
          bias_saverList.append(bias_saver)
          sortieImageList.append(sortieImage)
        return couche,kernelList,biasList,kernel_saverList,bias_saverList,sortieImageList
      else: 
        return deconv(inpt,sizeListOrNot,num_filters,nom,start,end)
    def generateur(inpt,kernels,num_filters,pooling,poolingTypes):
      """
      Description générale : Fonction créant le générateur, encodeur d'informations de l'image
      Entree : 
        inpt, couche d'entrée, ici ce sera l'image bruitée
        kernels, les différents noyaux de convolution, se présente soit sous forme de 
                liste simple soit sous forme d'une double liste. 
                Chaque sous liste représente un noyau de convolution 
                trop gros pour être réalisé en une couche et qui est donc séparé en sous-couches
        num_filters, la taille de filtre pour la couche (en gardant le même nombre de filtre
                pour chaque sous-couche si on a séparé la couche en sous-couches)
        poolingSize, les tailles des noyaux des couche de pooling
        poolingType, le type de pooling utilisé : SAME en générale
      Sortie : 
        L'image encodée (même si ce n'est plus réellement une image maintenant)
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(inpt,kernels[0],num_filters[0],pooling[0],poolingTypes[0],'convGenerator0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(kernels)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[i],num_filters[i],pooling[i],poolingTypes[i],'convGenerator'+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = convLayer(couche,kernels[-1],num_filters[-1],pooling[-1],poolingTypes[-1],'convGenerator'+str(len(kernels)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages

    def discriminator(inpt,size,num_filters,ID):
      """
      Entrees : 
        inpt, couche d'entrée du discriminateur
        size, liste, ou liste de liste si décomposition les tailles de couche successives pour revenir à la taille initiale
        num_filters, le nombre de filtre par couches
        ID, id unique pour séparer deux potentiels discriminateurs
      Sortie :
        Couche traitée par le discriminateur
      """
      kernelBiasList,saver,sortiesImages= [],[],[]
      print("Taille de l'entrée du discriminateur : ",end='')
      print(inpt.get_shape())
      def recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages):
        if type(kernel) == list:
          for k in kernel:
            kernelBiasList.append(k)
          for b in bias:
            kernelBiasList.append(b)
          for ks in kernel_saver:
            saver.append(ks)
          for bs in bias_saver:
            saver.append(bs)
          for i in sortieImage:
            sortiesImages.append(i)
        else:
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return None #Utilise la mutabilité des listes
      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(inpt,size[0],num_filters[0],'deconvDiscriminator'+ID+'0',start=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      for i in range(1,len(size)-1):
        couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[i],num_filters[i],'deconvDiscriminator'+ID+str(i))
        recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)

      couche,kernel,bias,kernel_saver,bias_saver,sortieImage = deconvLayer(couche,size[-1],num_filters[-1],'deconvDiscriminator'+ID+str(len(size)-1),end=True)
      recuperationDonneesCouches(kernel,bias,kernel_saver,bias_saver,sortieImage,kernelBiasList,saver,sortiesImages)
      return couche,kernelBiasList,saver,sortiesImages
    def discriminatorDense(inpt,neurones,ID):#ID car besoin nom unique pour chaque série de discriminator
        saver = []
        kernelBiasList = []
        sortiesImages = []
        couche = inpt
        for i,nb in enumerate(neurones):
          couche,kernel,bias,kernel_saver,bias_saver,sortieImage = denseLayer(couche,nb,'dense'+str(ID)+'_'+str(i+1))
          kernelBiasList.append(kernel)
          kernelBiasList.append(bias)
          saver.append(kernel_saver)
          saver.append(bias_saver)
          sortiesImages.append(sortieImage)
        return couche,kernelBiasList,saver,sortiesImages
      
    gen_input = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="input_noise")
    disc_input = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="input_clean")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    print('Taille de gen_input : '+str(gen_input.get_shape()))
    
    import numpy as np
    
    generator,gen_vars,gen_saver,gen_sortiesImages = generateur(disc_input,[[2]+[3],[3],[2]+[3]],[50,100,150],[4,4,2],['mean','mean','mean'])
    couche,kernelBiasList,saver,sortiesImages = discriminatorDense(generator,[150,300,150],'Lien')
    nbCouchesDsicriminateur = 15
    disc_faux,disc_vars,disc_saver,disc_faux_sortiesImages = discriminator(generator,np.linspace(int(generator.get_shape()[1]),199,nbCouchesDsicriminateur+1,dtype=np.int)[1:],np.linspace(int(generator.get_shape()[-1]),3,nbCouchesDsicriminateur,dtype=np.int),'Faux')
    
    gen_sortiesImages.append(tf.summary.image("inputGen/entree",tf.cast(gen_input,dtype=tf.uint8)))
    gen_sortiesImages.append(tf.summary.image("outputDisc/sortie",tf.cast(disc_faux,dtype=tf.uint8)))
    print('Taille de sortie disc_faux : ',end='')
    print(disc_faux.get_shape())
    print('Taille de sortie disc_input : ',end='')
    print(disc_input.get_shape())
    
    
    difference = disc_faux-gen_input
    disc_loss = tf.reduce_mean(difference)
    
    maximumGradient = tf.reduce_max(difference)
    minimumGradient = tf.reduce_min(difference)
    meanGradient = tf.reduce_mean(difference)
    maximumSortie = tf.reduce_max(disc_faux)
    minimumSortie = tf.reduce_min(disc_faux)
    meanSortie = tf.reduce_mean(disc_faux)
  
    loss_saver_disc = tf.summary.scalar("Discriminator_loss",disc_loss)
    
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5)
    
    saver =  disc_saver
    print("Trainable variables : "+str(tf.trainable_variables()))
    train_disc = optimizer_disc.minimize(disc_loss)
    init = tf.global_variables_initializer()
    summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
    tf_saver = tf.train.Saver()

    import os
    with tf.Session() as sess:
        def save(saver_biasKernel,gen_inpt,disc_inpt):
          for kernelBias in saver_biasKernel:
            summary_str = kernelBias.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt})
            summary_writer.add_summary(summary_str,i)
          return None
        if os.path.isdir('./checkpoints_'+nom) == True and forceInit == False:
          print(os.listdir('./checkpoints_'+nom))
          lastTrained = 0
          for elem in os.listdir('./checkpoints_'+nom):
            if elem[0:14] == 'checkpoint_at_' and int(elem[14]) > lastTrained:
              lastTrained = int(elem[14])
          tf_saver.restore(sess, 'checkpoints_'+nom+'/checkpoint_at_'+str(lastTrained)+'.ckpt')
        else :
          print('No previous training found...')
          init.run()
        print('Entrainement....')
        for i in range(0,101):
          gen_inpt, disc_inpt = next_batch(batch_size, noises,images,height)#Problèmes si gen_input
          _,dl = sess.run([train_disc,disc_loss], feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: True})
          summary_str = loss_saver_disc.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          summary_writer.add_summary(summary_str,i)
            
          save(disc_saver,gen_inpt,disc_inpt)
        
          maximumGradientValue = maximumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumGradientValue = minimumGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanGradientValue = meanGradient.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          maximumSortieValue = maximumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          minimumSortieValue = minimumSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
          meanSortieValue = meanSortie.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False})
        
          print('Step %i: Discriminator Loss: %f' % (i, dl))
          print('Différence : max %f, min %f, mean %f' % (maximumGradientValue, minimumGradientValue,meanGradientValue))
          print('Sortie : max %f, min %f, mean %f' % (maximumSortieValue, minimumSortieValue,meanSortieValue))
          if i % 4 == 0:
            os.remove('model/model'+nom+'.ckpt')
            tf_saver.save(sess,'model/model'+nom+'.ckpt')
          print('Step %i: Discriminator Loss: %f' % (i, dl))
        for sortieImage in gen_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        for sortieImage in disc_faux_sortiesImages:
          if sortieImage != None:
            try:
              summary_writer.add_summary(sortieImage.eval(feed_dict={gen_input: gen_inpt, disc_input: disc_inpt,training: False}))
            except Exception:
              traceback.format_exc()
        os.remove('model/model'+nom+'.ckpt')
        tf_saver.save(sess, 'model/model'+nom+'.ckpt')

### Essai 1

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 5 | 15 | 1 | Relu | \ |
| Conv | 10 | 15 | 1 | Relu | \ |
| Conv | 7 | 15 | 1 | Relu | \ |
| Conv | 7 | 5 | 1 | Relu | \ |
| Conv | 3 | 2 | 1 | Relu | \ |

### Essai 2

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 15 | 1 | Relu | \ |
| Conv | 6 | 12 | 1 | Relu | \ |
| Conv | 9 | 9 | 1 | Relu | \ |
| Conv | 12 | 6 | 1 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |

### Essai 3

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 15 | 1 | Relu | \ |
| Conv | 10 | 12 | 1 | Relu | \ |
| Conv | 17 | 9 | 1 | Relu | \ |
| Conv | 24 | 6 | 1 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |

### Essai 3'

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 15 | 1 | Relu | \ |
| Conv | 7 | 12 | 1 | Relu | \ |
| Conv | 11 | 9 | 1 | Relu | \ |
| Conv | 15 | 6 | 1 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |

### Essai 4

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 15 | 1 | Relu | \ |
| Conv | 10| 12 | 1 | Relu | \ |
| Conv | 17 | 9 | 1 | Relu | \ |
| Conv | 24 | 6 | 1 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |

### Essai 5

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 20 | 1 | Relu | \ |
| Conv | 3 | 18 | 1 | Relu | \ |
| Conv | 3 | 15 | 1 | Relu | \ |
| Conv | 7 | 15 | 1 | Relu | \ |
| Conv | 7 | 12 | 1 | Relu | \ |
| Conv | 7 | 12 | 1 | Relu | \ |
| Conv | 10 | 10 | 1 | Relu | \ |
| Conv | 10 | 10 | 1 | Relu | \ |
| Conv | 12 | 7 | 1 | Relu | \ |
| Conv | 12 | 7 | 1 | Relu | \ |
| Conv | 15 | 7 | 1 | Relu | \ |
| Conv | 15 | 3 | 1 | Relu | \ |
| Conv | 18 | 3 | 1 | Relu | \ |
| Conv | 20 | 3 | 1 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |

In [17]:
nb = 9
nom = ''
lastName = ''
for i in range(nb):
    if i %2 == 0:
        print()
        if i == nb-1:
            nom += "output = tf.layers.conv2d("+lastName+", filters=filtre["+str(i//2)+"], kernel_size=noyaux["
        elif i == 0:
            nom += "conv"+str(i//2+1)+" = tf.layers.conv2d(X, filters=filtre["+str(i//2)+"], kernel_size=noyaux["
            lastName = "conv"+str(i//2+1)
        else:
            nom += "conv"+str(i//2+1)+" = tf.layers.conv2d("+lastName+", filters=filtre["+str(i//2)+"], kernel_size=noyaux["
            lastName = "conv"+str(i//2+1)
        nom += str(i//2)+"],\n \t \t \t strides=1, padding='SAME',\n \t \t \t activation=tf.nn.relu, name='conv"
        nom += str(i//2+1)+"')"
    else:
        nom += 'maxPool' + str(i//2+1) + " = tf.layers.max_pooling2d("+lastName+", pool_size=(pool["+str(i//2)+"],pool["+str(i//2)+"]), strides=(pool["+str(i//2)+"],pool["+str(i//2)+"]), padding='same')"
        lastName = 'maxPool' + str(i//2+1)
    print(nom)
    nom = ''


conv1 = tf.layers.conv2d(X, filters=filtre[0], kernel_size=noyaux[0],
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv1')
maxPool1 = tf.layers.max_pooling2d(conv1, pool_size=(pool[0],pool[0]) strides=(pool[0],pool[0]), padding='same')

conv2 = tf.layers.conv2d(maxPool1, filters=filtre[1], kernel_size=noyaux[1],
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv2')
maxPool2 = tf.layers.max_pooling2d(conv2, pool_size=(pool[1],pool[1]) strides=(pool[1],pool[1]), padding='same')

conv3 = tf.layers.conv2d(maxPool2, filters=filtre[2], kernel_size=noyaux[2],
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv3')
maxPool3 = tf.layers.max_pooling2d(conv3, pool_size=(pool[2],pool[2]) strides=(pool[2],pool[2]), padding='same')

conv4 = tf.layers.conv2d(maxPool3, filters=filtre[3], kernel_size=noyaux[3],
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv4')
maxPool4 = tf.layers.max_pooling2d(conv4, po

### Essai 6

In [2]:
nb = 5
nbResize = 5
plage = 3,20
plageResize = 3,15
inputSize = 399
import numpy as np
chaineTableau = ''
chaineTf = ''
seprateur = '///'
def convTableau(indice,filtre,noyau,chaineTableau):
    chaineTableau += seprateur
    chaineTableau += "| Conv"+str(indice)+" | - | "+str(filtre)+" | "+ str(noyau) +" | 1 | Relu | \ |"
    return chaineTableau
def convTf(indice,filtre,noyau,chaineTf,first=False,last=False):
    chaineTf += seprateur
    global lastName
    if last == True:
        chaineTf += 'output'
        lastName = 'output'
    else:
        chaineTf += "conv"+str(indice)
    chaineTf += " = tf.layers.conv2d("
    if first == True:
        chaineTf += "X"
    else:
        chaineTf += lastName
    chaineTf+= ", filters="+str(filtre)+", kernel_size="+str(noyau)+",\n \t \t \t strides=1, padding='SAME',\n \t \t \t activation=tf.nn.relu, name='conv"+str(indice)+"')"
    lastName = "conv"+str(indice)
    chaineTf += seprateur+"print('Shape of "+lastName+" : '+str("+lastName+".get_shape()))"
    return chaineTf
def poolingTableau(indice,taille,pas,chaineTableau):
    chaineTableau += seprateur
    chaineTableau += "| Pool"+str(indice)+" | "+str(taille)+" | - | - | "+str(pas)+" | Relu |-|"
    return chaineTableau
def poolingTF(indice,taille,pas,chaineTf):
    chaineTf += seprateur
    global lastName
    chaineTf += 'maxPool' + str(indice) + " = tf.layers.max_pooling2d("+lastName+", pool_size=("+str(taille)+","+str(taille)+"), strides=("+str(taille)+","+str(taille)+"), padding='same')"
    lastName = 'maxPool' + str(indice)
    chaineTf += seprateur+"print('Shape of "+lastName+" : '+str("+lastName+".get_shape()))"
    chaineTf += seprateur
    return chaineTf
    
def resizeTableau(indice,size,chaineTableau):
    chaineTableau += seprateur
    chaineTableau += "| Resize"+str(indice)+" | "+str(size)+" | - | - | - | Relu | - |"
    return chaineTableau
def resizeTf(indice,size,chaineTf):
    chaineTf += seprateur
    chaineTf += seprateur
    global lastName
    chaineTf += "resized"+str(indice)+" = tf.image.resize_images("+lastName+", size=("+str(size)+","+str(size)+"), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)"
    lastName = "resized"+str(indice)
    chaineTf += seprateur+"print('Shape of "+lastName+" : '+str("+lastName+".get_shape()))"
    return chaineTf
def denseTableau(indice,nb,size,chaineTableau):
    chaineTableau += seprateur
    chaineTableau += "| Dense"+str(indice)+" | "+str(size)+" | - | - | - | Relu | "+str(nb)+" |"
    return chaineTableau
def denseTf(indice,nb,chaineTf):
    chaineTf += seprateur
    global lastName
    chaineTf += "dense"+str(indice)+" = tf.layers.dense("+lastName+","+str(nb)+")"
    lastName = "dense"+str(indice)
    chaineTf += seprateur+"print('Shape of "+lastName+" : '+str("+lastName+".get_shape()))"
    return chaineTf
def maxPoolingLinear(start,sizeInput,sizeOutput,nb,chaineTableau,chaineTf):
    global lastName
    filtre = np.linspace(plage[0],plage[1],nb,dtype=np.int)
    noyaux = np.linspace(plage[1],plage[0],nb,dtype=np.int)
    size = np.linspace(sizeInput,sizeOutput,nb+1,dtype=np.int)[1:]
    for i in range(len(size)):
        chaineTableau = resizeTableau(start+i,size[i],chaineTableau)
        chaineTableau = convTableau(start+i,filtre[i],noyaux[i],chaineTableau)
        chaineTf = resizeTf(start+i,size[i],chaineTf)
        chaineTf = convTf(start+i,filtre[i],noyaux[i],chaineTf)
    return chaineTableau,chaineTf
# filtre = np.linspace(plage[0],plage[1],nb//2+1 if nb % 2 == 1 else nb//2,dtype=np.int)
# noyaux = np.linspace(plage[1],plage[0],nb//2+1 if nb % 2 == 1 else nb//2,dtype=np.int)
# filtre = np.concatenate((filtre,np.flip(filtre,axis=0) if nb % 2 == 0 else np.flip(filtre[:-1],axis=0)),axis=0).tolist()
# noyaux = np.concatenate((noyaux,np.flip(noyaux,axis=0) if nb % 2 == 0 else np.flip(noyaux[:-1],axis=0)),axis=0).tolist()
filtre = np.linspace(3,3,nb,dtype=np.int)
noyaux = np.linspace(plage[1],plage[0],nb,dtype=np.int)
filtreResize = np.linspace(3,3,nbResize,dtype=np.int)
noyauxResize = np.linspace(plageResize[0],plageResize[1],nbResize,dtype=np.int)
maxpooling = False
poolingValue = 2
denseLayer = True
layers = [3 for i in range(15)]
print()
nom = ""
last = ''
lastName = 'X'
chaineTableau += "| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |\n| --- | --- | --- | --- | --- | --- | --- |"
currentSize = inputSize
chaineTableau = convTableau(0,filtre[0],noyaux[0],chaineTableau)
chaineTf = convTf(0,filtre[0],noyaux[0],chaineTf,first=True)
# if maxpooling == True:
#     currentSize //= poolingValue
# #     print(type(i),type(currentSize),type(poolingValue),type(chaineTableau))
#     chaineTableau = poolingTableau(i,currentSize,poolingValue,chaineTableau)
#     chaineTf = poolingTF(0,currentSize,poolingValue,chaineTf)

for i in range(1,len(filtre)-1):
    chaineTableau = convTableau(i,filtre[i],noyaux[i],chaineTableau)
    chaineTf = convTf(i,filtre[i],noyaux[i],chaineTf,chaineTf)
#     if maxpooling == True:
#         currentSize //= poolingValue
#         chaineTableau = poolingTableau(i,currentSize,poolingValue,chaineTableau)
#         chaineTf = poolingTF(i,currentSize,poolingValue,chaineTf)
chaineTableau = convTableau(len(filtre)-1,filtre[-1],noyaux[-1],chaineTableau)
chaineTf = convTf(len(filtre)-1,filtre[-1],noyaux[-1],chaineTf)
# if denseLayer == True:
#     for i in range(len(layers)):
#         chaineTableau = denseTableau(i,layers[i],currentSize,chaineTableau)
#         chaineTf = denseTf(i,layers[i],chaineTf)
# if maxpooling == True:
#     temp = maxPoolingLinear(len(filtre),currentSize,inputSize,5,chaineTableau,chaineTf)
#     chaineTableau = temp[0]
#     chaineTf = temp[1]

#     print(chaineTf)
# print(chaineTableau)
chaineTableau = chaineTableau.split(seprateur)
# print(chaineTableau)
chaineTf = chaineTf.split(seprateur)
# print(chaineTableau)
for elem in chaineTableau:
    print(elem)
for elem in chaineTf:
    print(elem)


| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Conv0 | - | 3 | 20 | 1 | Relu | \ |
| Conv1 | - | 3 | 15 | 1 | Relu | \ |
| Conv2 | - | 3 | 11 | 1 | Relu | \ |
| Conv3 | - | 3 | 7 | 1 | Relu | \ |
| Conv4 | - | 3 | 3 | 1 | Relu | \ |

conv0 = tf.layers.conv2d(X, filters=3, kernel_size=20,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv0')
print('Shape of conv0 : '+str(conv0.get_shape()))
conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=15,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv1')
print('Shape of conv1 : '+str(conv1.get_shape()))
conv2 = tf.layers.conv2d(conv1, filters=3, kernel_size=11,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv2')
print('Shape of conv2 : '+str(conv2.get_shape()))
conv3 = tf.layers.conv2d(conv2, filters=3, kernel_size=7,
 	 	 	 strides=1, padding='SAME',
 	 	 	 a

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 20 | 1 | Relu | \ |
| Conv | 5 | 17 | 1 | Relu | \ |
| Conv | 7 | 15 | 1 | Relu | \ |
| Conv | 10 | 12 | 1 | Relu | \ |
| Conv | 12 | 10 | 1 | Relu | \ |
| Conv | 15 | 7 | 1 | Relu | \ |
| Conv | 17 | 5 | 1 | Relu | \ |
| Conv | 20 | 3 | 1 | Relu | \ |
| Conv | 17 | 5 | 1 | Relu | \ |
| Conv | 15 | 7 | 1 | Relu | \ |
| Conv | 12 | 10 | 1 | Relu | \ |
| Conv | 10 | 12 | 1 | Relu | \ |
| Conv | 7 | 15 | 1 | Relu | \ |
| Conv | 5 | 17 | 1 | Relu | \ |
| Conv | 3 | 20 | 1 | Relu | \ |

## Essai 7

| Type de couche | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- |
| Conv | 3 | 15 | 1 | Relu | \ |
| Pool | / | / | 2 | Relu | \ |
| Conv | 7 | 12 | 1 | Relu | \ |
| Pool | / | / | 2 | Relu | \ |
| Conv | 11 | 9 | 1 | Relu | \ |
| Pool | / | / | 2 | Relu | \ |
| Conv | 15 | 6 | 1 | Relu | \ |
| Pool | / | / | 2 | Relu | \ |
| Conv | 3 | 3 | 1 | Relu | \ |
| Resize | - | - | - | - | - | - |
| Conv | 3 | 5 | 1 | Relu | \ |
| Resize | - | - | - | - | - | - |
| Conv | 3 | 10 | 1 | Relu | \ |
| Resize | - | - | - | - | - | - |
| Conv | 3 | 14 | 1 | Relu | \ |

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai7T'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

noyaux = [3,25,15,9,6,3]
filtre = [7, 9, 11, 15,19,7]
noyauxDecode = [5,10,14]
# filtreDecode = [3, 5, 7, 11,15,3]
pool = [2,2,2,2]

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    # 399*399*batch_size
    conv1 = tf.layers.conv2d(X, filters=filtre[0], kernel_size=noyaux[0],
 	 	 	    strides=1, padding='SAME',
 	 	 	    activation=tf.nn.relu, name='conv1')
    maxPool1 = tf.layers.max_pooling2d(conv1, pool_size=(pool[0],pool[0]), strides=(pool[0],pool[0]), padding='same')
    print(maxPool1.get_shape())
    # 199*199*batch_size
    conv2 = tf.layers.conv2d(maxPool1, filters=filtre[1], kernel_size=noyaux[1],
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv2')
    maxPool2 = tf.layers.max_pooling2d(conv2, pool_size=(pool[1],pool[1]), strides=(pool[1],pool[1]), padding='same')
    print(maxPool2.get_shape())
    # 99*99*batch_size
    conv3 = tf.layers.conv2d(maxPool2, filters=filtre[2], kernel_size=noyaux[2],
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv3')
    maxPool3 = tf.layers.max_pooling2d(conv3, pool_size=(pool[2],pool[2]), strides=(pool[2],pool[2]), padding='same')
    print(maxPool3.get_shape())
    # 49*49*batch_size
    conv4 = tf.layers.conv2d(maxPool3, filters=filtre[3], kernel_size=noyaux[3],
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv4')
    maxPool4 = tf.layers.max_pooling2d(conv4, pool_size=(pool[3],pool[3]), strides=(pool[3],pool[3]), padding='same')
    print(maxPool4.get_shape())
    # 24*24*batch_size
    encoded = tf.layers.conv2d(maxPool4, filters=filtre[4], kernel_size=noyaux[4],
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv5')
    
    upsample1 = tf.image.resize_images(encoded, size=(49,49), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print(upsample1.get_shape())
    # Now 49*49xbatch_size
    conv5 = tf.layers.conv2d(inputs=upsample1, filters=3, kernel_size=noyauxDecode[0], padding='same', activation=tf.nn.relu)
    print(conv5.get_shape())
    # Now 7x7xbatch_size
    upsample2 = tf.image.resize_images(conv4, size=(199,199), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print(upsample2.get_shape())
    # Now 199x199xbatch_size
    conv6 = tf.layers.conv2d(inputs=upsample2, filters=3, kernel_size=noyauxDecode[1], padding='same', activation=tf.nn.relu)
    print(conv6.get_shape())
    # Now 199x199xbatch_size
    upsample3 = tf.image.resize_images(conv5, size=(399,399), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print(upsample3.get_shape())
    # Now 399x399xbatch_size
    output = tf.layers.conv2d(inputs=upsample3, filters=3, kernel_size=noyauxDecode[2], padding='same', activation=tf.nn.relu)
    print(output.get_shape())
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver = tf.summary.scalar("Loss",loss)
        summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 100
    n_batches_per_epoch = 3
    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: y_batch, y: X_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver.eval(feed_dict={X: y_batch, y: X_batch})
                  summary_writer.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))
        summary_writer.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer.close()

## Essai 8

| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Conv0 | - | 3 | 20 | 1 | Relu | \ |
| Pool2 | 199 | - | - | 2 | Relu |-|
| Conv1 | - | 7 | 15 | 1 | Relu | \ |
| Pool1 | 99 | - | - | 2 | Relu |-|
| Conv2 | - | 11 | 11 | 1 | Relu | \ |
| Pool2 | 49 | - | - | 2 | Relu |-|
| Conv3 | - | 15 | 7 | 1 | Relu | \ |
| Pool3 | 24 | - | - | 2 | Relu |-|
| Conv4 | - | 20 | 3 | 1 | Relu | \ |
| Dense0 | 24 | - | - | - | Relu | 500 |
| Dense1 | 24 | - | - | - | Relu | 200 |
| Dense2 | 24 | - | - | - | Relu | 100 |
| Resize5 | 99 | - | - | - | Relu | - |
| Conv5 | - | 3 | 20 | 1 | Relu | \ |
| Resize6 | 174 | - | - | - | Relu | - |
| Conv6 | - | 7 | 15 | 1 | Relu | \ |
| Resize7 | 249 | - | - | - | Relu | - |
| Conv7 | - | 11 | 11 | 1 | Relu | \ |
| Resize8 | 324 | - | - | - | Relu | - |
| Conv8 | - | 15 | 7 | 1 | Relu | \ |
| Resize9 | 399 | - | - | - | Relu | - |
| Conv9 | - | 3 | 3 | 1 | Relu | \ |

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai8'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

noyaux = [3,25,15,9,6,3]
filtre = [7, 9, 11, 15,19,7]
noyauxDecode = [5,10,14]
# filtreDecode = [3, 5, 7, 11,15,3]
pool = [2,2,2,2]

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    # 399*399*batch_size
    conv0 = tf.layers.conv2d(X, filters=3, kernel_size=20,
 	 	 	 strides=1, padding='SAME',
 	 	 	 activation=tf.nn.relu, name='conv0')
    print('Shape of conv0 : '+str(conv0.get_shape()))
    maxPool0 = tf.layers.max_pooling2d(conv0, pool_size=(199,199), strides=(199,199), padding='same')
    print('Shape of maxPool0 : '+str(maxPool0.get_shape()))

    conv1 = tf.layers.conv2d(maxPool0, filters=7, kernel_size=15,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv1')
    print('Shape of conv1 : '+str(conv1.get_shape()))
    maxPool1 = tf.layers.max_pooling2d(conv1, pool_size=(99,99), strides=(99,99), padding='same')
    print('Shape of maxPool1 : '+str(maxPool1.get_shape()))

    conv2 = tf.layers.conv2d(maxPool1, filters=11, kernel_size=11,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv2')
    print('Shape of conv2 : '+str(conv2.get_shape()))
    maxPool2 = tf.layers.max_pooling2d(conv2, pool_size=(49,49), strides=(49,49), padding='same')
    print('Shape of maxPool2 : '+str(maxPool2.get_shape()))

    conv3 = tf.layers.conv2d(maxPool2, filters=15, kernel_size=7,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv3')
    print('Shape of conv3 : '+str(conv3.get_shape()))
    maxPool3 = tf.layers.max_pooling2d(conv3, pool_size=(24,24), strides=(24,24), padding='same')
    print('Shape of maxPool3 : '+str(maxPool3.get_shape()))

    conv4 = tf.layers.conv2d(maxPool3, filters=20, kernel_size=3,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv4')
    print('Shape of conv4 : '+str(conv4.get_shape()))
    dense0 = tf.layers.dense(conv4,500)
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,200)
    print('Shape of dense1 : '+str(dense1.get_shape()))
    dense2 = tf.layers.dense(dense1,100)
    print('Shape of dense2 : '+str(dense2.get_shape()))

    resized5 = tf.image.resize_images(dense2, size=(99,99), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print('Shape of resized5 : '+str(resized5.get_shape()))
    conv5 = tf.layers.conv2d(resized5, filters=3, kernel_size=20,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv5')
    print('Shape of conv5 : '+str(conv5.get_shape()))

    resized6 = tf.image.resize_images(conv5, size=(174,174), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print('Shape of resized6 : '+str(resized6.get_shape()))
    conv6 = tf.layers.conv2d(resized6, filters=7, kernel_size=15,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv6')
    print('Shape of conv6 : '+str(conv6.get_shape()))

    resized7 = tf.image.resize_images(conv6, size=(249,249), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print('Shape of resized7 : '+str(resized7.get_shape()))
    conv7 = tf.layers.conv2d(resized7, filters=11, kernel_size=11,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv7')
    print('Shape of conv7 : '+str(conv7.get_shape()))

    resized8 = tf.image.resize_images(conv7, size=(324,324), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print('Shape of resized8 : '+str(resized8.get_shape()))
    conv8 = tf.layers.conv2d(resized8, filters=15, kernel_size=7,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv8')
    print('Shape of conv8 : '+str(conv8.get_shape()))

    resized9 = tf.image.resize_images(conv8, size=(399,399), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    print('Shape of resized9 : '+str(resized9.get_shape()))
    output = tf.layers.conv2d(resized9, filters=3, kernel_size=3,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='output')
    print('Shape of output : '+str(output.get_shape()))
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver = tf.summary.scalar("Loss",loss)
        summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 100
    n_batches_per_epoch = 3
    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: y_batch, y: X_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver.eval(feed_dict={X: y_batch, y: X_batch})
                  summary_writer.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))
        summary_writer.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer.close()

## Essai 9

| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Dense0 | - | - | - | - | Relu | 800 |
| Dense1 | - | - | - | - | Relu | 700 |
| Dense2 | - | - | - | - | Relu | 500 |
| Dense3 | - | - | - | - | Relu | 200 |
| Dense4 | - | - | - | - | Relu | 500 |
| Dense5 | - | - | - | - | Relu | 700 |
| Dense6 | - | - | - | - | Relu | 800 |

## Essai 10

| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Dense0 | - | - | - | - | Relu | 800 |
| Dense1 | - | - | - | - | Relu | 700 |
| Dense2 | - | - | - | - | Relu | 500 |
| Dense3 | - | - | - | - | Relu | 200 |
| Dense4 | - | - | - | - | Relu | 500 |
| Dense5 | - | - | - | - | Relu | 700 |
| Dense6 | - | - | - | - | Relu | 800 |

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai10'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

noyaux = [3,25,15,9,6,3]
filtre = [7, 9, 11, 15,19,7]
noyauxDecode = [5,10,14]
# filtreDecode = [3, 5, 7, 11,15,3]
pool = [2,2,2,2]

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    # 399*399*batch_size
    dense0 = tf.layers.dense(y,3)
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,3)
    print('Shape of dense1 : '+str(dense1.get_shape()))
    dense2 = tf.layers.dense(dense1,3)
    print('Shape of dense2 : '+str(dense2.get_shape()))
    dense3 = tf.layers.dense(dense2,3)
    print('Shape of dense3 : '+str(dense3.get_shape()))
    dense4 = tf.layers.dense(dense3,3)
    print('Shape of dense4 : '+str(dense4.get_shape()))
    dense5 = tf.layers.dense(dense4,3)
    print('Shape of dense5 : '+str(dense5.get_shape()))
    output = tf.layers.dense(dense5,3)
    print('Shape of output : '+str(output.get_shape()))
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver = tf.summary.scalar("Loss",loss)
        summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 100
    n_batches_per_epoch = 3
    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: y_batch, y: X_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver.eval(feed_dict={X: y_batch, y: X_batch})
                  summary_writer.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))
        summary_writer.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer.close()

## Essai 10

| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Dense0 | 399 | - | - | - | Relu | 3 |
| Dense1 | 399 | - | - | - | Relu | 3 |
| Dense2 | 399 | - | - | - | Relu | 3 |
| Dense3 | 399 | - | - | - | Relu | 3 |
| Dense4 | 399 | - | - | - | Relu | 3 |
| Dense5 | 399 | - | - | - | Relu | 3 |
| Dense6 | 399 | - | - | - | Relu | 3 |
| Dense7 | 399 | - | - | - | Relu | 3 |
| Dense8 | 399 | - | - | - | Relu | 3 |
| Dense9 | 399 | - | - | - | Relu | 3 |
| Dense10 | 399 | - | - | - | Relu | 3 |
| Dense11 | 399 | - | - | - | Relu | 3 |
| Dense12 | 399 | - | - | - | Relu | 3 |
| Dense13 | 399 | - | - | - | Relu | 3 |
| Dense14 | 399 | - | - | - | Relu | 3 |

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai11'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

noyaux = [3,25,15,9,6,3]
filtre = [7, 9, 11, 15,19,7]
noyauxDecode = [5,10,14]
# filtreDecode = [3, 5, 7, 11,15,3]
pool = [2,2,2,2]

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    # 399*399*batch_size
    dense0 = tf.layers.dense(X,3)
    print('Shape of dense0 : '+str(dense0.get_shape()))
    dense1 = tf.layers.dense(dense0,3)
    print('Shape of dense1 : '+str(dense1.get_shape()))
    dense2 = tf.layers.dense(dense1,3)
    print('Shape of dense2 : '+str(dense2.get_shape()))
    dense3 = tf.layers.dense(dense2,3)
    print('Shape of dense3 : '+str(dense3.get_shape()))
    dense4 = tf.layers.dense(dense3,3)
    print('Shape of dense4 : '+str(dense4.get_shape()))
    dense5 = tf.layers.dense(dense4,3)
    print('Shape of dense5 : '+str(dense5.get_shape()))
    dense6 = tf.layers.dense(dense5,3)
    print('Shape of dense6 : '+str(dense6.get_shape()))
    dense7 = tf.layers.dense(dense6,3)
    print('Shape of dense7 : '+str(dense7.get_shape()))
    dense8 = tf.layers.dense(dense7,3)
    print('Shape of dense8 : '+str(dense8.get_shape()))
    dense9 = tf.layers.dense(dense8,3)
    print('Shape of dense9 : '+str(dense9.get_shape()))
    dense10 = tf.layers.dense(dense9,3)
    print('Shape of dense10 : '+str(dense10.get_shape()))
    dense11 = tf.layers.dense(dense10,3)
    print('Shape of dense11 : '+str(dense11.get_shape()))
    dense12 = tf.layers.dense(dense11,3)
    print('Shape of dense12 : '+str(dense12.get_shape()))
    dense13 = tf.layers.dense(dense12,3)
    print('Shape of dense13 : '+str(dense13.get_shape()))
    output = tf.layers.dense(dense13,3)
    print('Shape of output : '+str(output.get_shape()))
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.square(output - X))

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer()
        training_op = optimizer.minimize(loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver = tf.summary.scalar("Loss",loss)
        print(os.path.split(X.name))
#         weights = tf.get_default_graph().get_tensor_by_name(os.path.split(X.name)[1])
#         tf.summary.histogram('weightsX', weights)
        summary_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 300
    n_batches_per_epoch = 3
    import os
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op, feed_dict={X: y_batch, y: X_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver.eval(feed_dict={X: y_batch, y: X_batch})
                  summary_writer.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train))
        summary_writer.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer.close()

## Essai 12

| Type de couche | Taille | Nb de filtres | Taille du noyau | Pas | Fonction d'activation | Nb de neurones |
| --- | --- | --- | --- | --- | --- | --- |
| Conv0 | - | 3 | 20 | 1 | Relu | \ |
| Conv1 | - | 3 | 15 | 1 | Relu | \ |
| Conv2 | - | 3 | 11 | 1 | Relu | \ |
| Conv3 | - | 3 | 7 | 1 | Relu | \ |
| Conv4 | - | 3 | 3 | 1 | Relu | \ |

### Recopie pour apprentissage par couche avec convolution

In [2]:
repetition = ["""for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op"""+str(i)+""", feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver"""+str(i)+""".eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer"""+str(i)+""".add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train"""+str(i)+""" = loss"""+str(i)+""".eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train"""+str(i)+"""))""" for i in range(5)]
for element in repetition:
    print(element)

for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op1, feed_dict={X: X_batch, y: y_batch})
                if iteration %

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
nom = 'Essai12'

tf.reset_default_graph()
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}{}/".format(root_logdir,nom,now)
taining0 = True
taining1 = True
taining2 = True
taining3 = True
taining4 = True
    

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
    y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
    print(X.get_shape())
    
    
    conv0 = tf.layers.conv2d(X, filters=3, kernel_size=20,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv0', trainable=taining0)
    print('Shape of conv0 : '+str(conv0.get_shape()))
    conv1 = tf.layers.conv2d(conv0, filters=3, kernel_size=15,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv1', trainable=taining1)
    print('Shape of conv1 : '+str(conv1.get_shape()))
    conv2 = tf.layers.conv2d(conv1, filters=3, kernel_size=11,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv2', trainable=taining2)
    print('Shape of conv2 : '+str(conv2.get_shape()))
    conv3 = tf.layers.conv2d(conv2, filters=3, kernel_size=7,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='conv3', trainable=taining3)
    print('Shape of conv3 : '+str(conv3.get_shape()))
    output = tf.layers.conv2d(conv3, filters=3, kernel_size=3,
           strides=1, padding='SAME',
           activation=tf.nn.relu, name='output', trainable=taining4)
    print('Shape of output : '+str(output.get_shape()))
        
      
    with tf.name_scope('Optimizer'):
        optimizer = tf.train.AdamOptimizer()
    with tf.name_scope("phase0"):
        loss0 = tf.reduce_mean(tf.square(conv0 - y))
        training_op0 = optimizer.minimize(loss0)
    with tf.name_scope("phase1"):
        loss1 = tf.reduce_mean(tf.square(conv1 - y))
        training_op1 = optimizer.minimize(loss1)
    with tf.name_scope("phase2"):
        loss2 = tf.reduce_mean(tf.square(conv2 - y))
        training_op2 = optimizer.minimize(loss2)
    with tf.name_scope("phase3"):
        loss3 = tf.reduce_mean(tf.square(conv3 - y))
        training_op3 = optimizer.minimize(loss3)
    with tf.name_scope("phase4"):
        loss4 = tf.reduce_mean(tf.square(output - y))
        training_op4 = optimizer.minimize(loss4)
    
    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("enregistrement"):
        loss_saver0 = tf.summary.scalar("Loss0",loss0)
        loss_saver1 = tf.summary.scalar("Loss1",loss1)
        loss_saver2 = tf.summary.scalar("Loss2",loss2)
        loss_saver3 = tf.summary.scalar("Loss3",loss3)
        loss_saver4 = tf.summary.scalar("Loss4",loss4)
        kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
        kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
        kernel2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/kernel')[0]
        kernel3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/kernel')[0]
        kernel4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/kernel')[0]
        trainable = tf.trainable_variables()
        print(trainable[:2])
        
        bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
        bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
        bias2 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv2/bias')[0]
        bias3 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv3/bias')[0]
        bias4 = tf.get_collection(tf.GraphKeys.VARIABLES, 'output/bias')[0]
        print(os.path.split(X.name))
        summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
        summary_writer1 = tf.summary.FileWriter(logdir,tf.get_default_graph())
        summary_writer2 = tf.summary.FileWriter(logdir,tf.get_default_graph())
        summary_writer3 = tf.summary.FileWriter(logdir,tf.get_default_graph())
        summary_writer4 = tf.summary.FileWriter(logdir,tf.get_default_graph())
        saver = tf.train.Saver()
        
    with tf.name_scope('Avant-Apres_'+nom):
        orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
        fin = tf.summary.image('output',tf.cast(output,dtype=tf.uint8))
        

    n_epochs = 100
    n_batches_per_epoch = 3
    Lloss = []
    def arret(lastLoss, seuil, duree, decroissance = False):
        global Lloss
        if len(Lloss) < duree:
            Lloss.append(lastLoss)
            return False
        else:
            Lloss = Lloss[1:duree] + [lastLoss]
            L = Lloss[:]
            L.sort(reverse=True)
            if L[0] <= seuil:
              if decroissance == True:
                return True if L == Lloss else False
              else:
                return True
            return False
          
    import os
    with tf.Session() as sess:
        init.run()
        # Training 0
#         training0 = True
#         training1 = training2 = training3 = training4 = False
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
            if arret(loss_train0,800,6) == True:
              break
        # Training 1
        training1 = True
        training0 = training2 = training3 = training4 = False
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op1, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer1.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train1 = loss1.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train1))
            if arret(loss_train1,700,4) == True:
              break
        # Training 2
        training2 = True
        training1 = training0 = training3 = training4 = False
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op2, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver2.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer2.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train2 = loss2.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train2))
            if arret(loss_train2,500,4) == True:
              break
        # Training 3
        training3 = True
        training1 = training2 = training0 = training4 = False
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op3, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver3.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer3.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train3 = loss3.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train3))
            if arret(loss_train3,500,4) == True:
              break
        # Training 4
        training4 = True
        training1 = training2 = training3 = training0 = False
        for epoch in range(n_epochs):
            print("epoch : ",epoch)
            for iteration in range(n_batches_per_epoch):
                X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                sess.run(training_op4, feed_dict={X: X_batch, y: y_batch})
                if iteration % 5 == 0:
                  summary_str = loss_saver4.eval(feed_dict={X: X_batch, y: y_batch})
                  summary_writer4.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
            loss_train4 = loss4.eval(feed_dict={X: X_batch, y: y_batch})
            print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train4))
            if arret(loss_train4,200,4) == True:
              break
        summary_writer4.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
        summary_writer4.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
        saver.save(sess, 'model/model'+nom+'.ckpt')
      
summary_writer0.close()
summary_writer1.close()
summary_writer2.close()
summary_writer2.close()
summary_writer3.close()

### Essais C et Cdeux

In [ ]:
height = 399
width = 399
channels = 3

batch_size = 7
i = 0
a,b = 2,2
while a<=8:
  b = 2
  while b <= 8:
    print("Essai pour conv0 avec " + str(a) + " et conv1 avec " + str(b))
    nom = 'EssaiCdeux'+str(i)

    tf.reset_default_graph()
    from datetime import datetime

    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    logdir = "{}/run-{}{}/".format(root_logdir,nom,now)

    with tf.Graph().as_default():
        X = tf.placeholder(tf.float32, shape=[batch_size,height, width, channels],name="X")
        y = tf.placeholder(tf.float32, shape=[batch_size, height, width, channels], name="y")
        print(X.get_shape())


        conv0 = tf.layers.conv2d(X, filters=3, kernel_size=(a,a),
               strides=1, padding='SAME',
               activation=tf.nn.relu, name='conv0')
        print('Shape of conv0 : '+str(conv0.get_shape()))
        conv1 = tf.layers.conv2d(X, filters=3, kernel_size=(b,b),
               strides=1, padding='SAME',
               activation=tf.nn.relu, name='conv1')
        print('Shape of conv1 : '+str(conv1.get_shape()))


        with tf.name_scope('Optimizer'):
            optimizer = tf.train.AdamOptimizer()
        with tf.name_scope("phase0"):
            loss0 = tf.reduce_mean(tf.square(conv1 - y))
            training_op0 = optimizer.minimize(loss0)

        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("enregistrement"):
            loss_saver0 = tf.summary.scalar("Loss",loss0)
            kernel0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/kernel')[0]
            bias0 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv0/bias')[0]
            kernel1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/kernel')[0]
            bias1 = tf.get_collection(tf.GraphKeys.VARIABLES, 'conv1/bias')[0]
            kernel_saver0 = tf.summary.histogram("Kernel",kernel0)
            bias_saver0 = tf.summary.histogram("Bias",bias0)
            kernel_saver1 = tf.summary.histogram("Kernel",kernel1)
            bias_saver1 = tf.summary.histogram("Bias",bias1)
            print(os.path.split(X.name))
            summary_writer0 = tf.summary.FileWriter(logdir,tf.get_default_graph())
            saver = tf.train.Saver()

        with tf.name_scope('Avant-Apres_'+nom):
            orig = tf.summary.image('input',tf.cast(X,dtype=tf.uint8))
            fin = tf.summary.image('output',tf.cast(conv1,dtype=tf.uint8))


        n_epochs = 100
        n_batches_per_epoch = 3
        Lloss = []
        def arret(lastLoss, seuil, duree, decroissance = False):
            global Lloss
            if len(Lloss) < duree:
                Lloss.append(lastLoss)
                return False
            else:
                Lloss = Lloss[1:duree] + [lastLoss]
                L = Lloss[:]
                L.sort(reverse=True)
                if L[0] <= seuil:
                  if decroissance == True:
                    return True if L == Lloss else False
                  else:
                    return True
                return False

        import os
        with tf.Session() as sess:
            init.run()
            for epoch in range(n_epochs):
                print("epoch : ",epoch)
                for iteration in range(n_batches_per_epoch):
                    X_batch, y_batch = next_batch(batch_size, train_dataset_item,train_dataset_label)
                    sess.run(training_op0, feed_dict={X: X_batch, y: y_batch})
                    if iteration % 5 == 0:
                      summary_str = loss_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                      summary_strKernel = kernel_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                      summary_strBias = bias_saver0.eval(feed_dict={X: X_batch, y: y_batch})
                      summary_strKernel1 = kernel_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                      summary_strBias1 = bias_saver1.eval(feed_dict={X: X_batch, y: y_batch})
                      summary_writer0.add_summary(summary_str,epoch*n_batches_per_epoch+iteration)
                      summary_writer0.add_summary(summary_strKernel,epoch*n_batches_per_epoch+iteration)
                      summary_writer0.add_summary(summary_strBias,epoch*n_batches_per_epoch+iteration)
                      summary_writer0.add_summary(summary_strKernel1,epoch*n_batches_per_epoch+iteration)
                      summary_writer0.add_summary(summary_strBias1,epoch*n_batches_per_epoch+iteration)

                loss_train0 = loss0.eval(feed_dict={X: X_batch, y: y_batch})
                print("Pour l'époque {} la différence vaut {}".format(epoch,loss_train0))
                if arret(loss_train0,100,6) == True:
                  break
            summary_writer0.add_summary(orig.eval(feed_dict={X: X_batch, y: y_batch}))
            summary_writer0.add_summary(fin.eval(feed_dict={X: X_batch, y: y_batch}))
            saver.save(sess, 'model/model'+nom+'.ckpt')
    summary_writer0.close()
    b += 2
    i += 1
  a += 2

Avec une couche, il y a apprentissage au début avec les couches de plus petit noyau. </br>
Avec 2 couches, de même. </br>
Globalement à la fin le système reste imprécis.